# Hermeuptychia

## evaluate Wenlin's assembly 20181205

Based on Jing's description and Wenlin's notes, the final assemblies for 3303 (Hermeuptychia sosybius) and 3318 (Hermeuptychia intricata) is stored in `/work0/wenlin/sequencing/3303_round2/7_platanus/3303_careful_ass0.3_t1_scaffold.fa`, `/home/xcao/w/20181205Hermeuptychia/20181205wenlin/3303newAll_scaffold.fa` and `/work0/wenlin/sequencing/3318_with_3312_round2/10_improve_assembly/3318_careful_ass0.3_t1_merged_t2_gapClosed_v1.fa`. Copy the file to `/home/xcao/w/20181205Hermeuptychia/20181205wenlin/`.  
Evaluate the quality of the genomes with Quast

```
/home/xcao/p/quast/quast-5.0.2/quast.py -t 32 --eukaryote --large --k-mer-stats --circos  --gene-finding --rna-finding --conserved-genes-finding --use-all-alignments /home/xcao/w/20181205Hermeuptychia/20181205wenlin/3303_careful_ass0.3_t1_scaffold.fa /home/xcao/w/20181205Hermeuptychia/20181205wenlin/3303newAll_scaffold.fa  /home/xcao/w/20181205Hermeuptychia/20181205wenlin/3318_careful_ass0.3_t1_merged_t2_gapClosed_v1.fa -o /home/xcao/w/20181205Hermeuptychia/20181205evaluate_wenlin/ --no-gzip
```

## prepare reads for 3303
20181205

Based on job scripts `/work0/wenlin/sequencing/3303_round2/7_platanus/assemble.job`, 91 files were used as input.

### get some information about the input fastq files

#### run fastqc

```
files = open('/home/xcao/w/20181205Hermeuptychia/20181205Reads3303/wenlinInput/files.txt').read().split()
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181205Reads3303/wenlinInput/fastqc/'
fastqc = '/home/xcao/p/fastqc/FastQC/fastqc'
import os
file_cmds = '/home/xcao/w/20181205Hermeuptychia/scripts/20181205_fastqc.cmds'
open(file_cmds,'w').write('\n'.join('{fastqc} {file} -o {outfolder} -t 1'.format(fastqc=fastqc,outfolder=outfolder,file=file) for file in files))
os.system('python /home/xcao/p/xiaolongTools/multiThread.py 48 '+file_cmds)
os.system('cd  '+outfolder+'&& unzip "*.zip"')
```

#### summarize result

```
import glob
folder = '/home/xcao/w/20181205Hermeuptychia/20181205Reads3303/wenlinInput/fastqc/'
file_fastqc_data = glob.glob(folder+'**/fastqc_data.txt',recursive=True)
print(len(file_fastqc_data))

import os
import math
def fastqc2readsinfo(filename):
    l = open(filename).readlines()
    sample_id_full = os.path.basename(os.path.dirname(filename))
    sample_id = sample_id_full.split('_cor_')[0]
    if '_R1_' in sample_id_full:
        sample_reads_type = 'R1'
    elif '_R2_' in sample_id_full:
        sample_reads_type = 'R2'
    else:
        sample_reads_type = 'singleton'
    if 'combMate' in sample_id_full:
        sample_reads_type = 'combMate'+sample_reads_type
    if 'pairedM' in sample_id_full:
        sample_reads_type = 'pairedM'+sample_reads_type
    reads_count = l[6].split()[2]
    GC_percentage = int(l[9].split()[1])
    n = 0
    while n < len(l):
        if '>>Sequence Length Distribution' in l[n]:
            break
        else:
            n = n+1
    n = n+2
    reads_len_info = []
    while n < len(l):
        if l[n][0] != '>':
            _reads_len_range, _reads_count = l[n].strip().split('\t')
            if '-' in _reads_len_range:
                _rs,_re = _reads_len_range.split('-')
                _rl = (float(_rs) + float(_re))/2
            else:
                _rl = float(_reads_len_range)
            _reads_count = float(_reads_count)
            reads_len_info.append([_rl,_reads_count])
            n += 1
        else:
            break
    reads_count = sum(e[1] for e in reads_len_info)
    reads_len_mean = sum(e[0]*e[1] for e in reads_len_info)/reads_count
    reads_len_sd = math.sqrt(sum(number*number*count for number, count in reads_len_info) / reads_count - reads_len_mean * reads_len_mean)
    #print('total reads', reads_count,'reads length mean', reads_len_mean, 'reads length sd', reads_len_sd, GC_percentage)
    return sample_id, sample_reads_type, reads_count, reads_len_mean, reads_len_sd, GC_percentage,sample_id_full

import pandas as pd
df_fastqc = pd.DataFrame()
df2 = pd.DataFrame()
for f in file_fastqc_data:
    sample_id, sample_reads_type, reads_count, reads_len_mean, reads_len_sd, GC_percentage,sample_id_full= fastqc2readsinfo(f)
    df_fastqc.loc[sample_id,'reads_count_'+sample_reads_type] = reads_count
    df_fastqc.loc[sample_id,'GC_percentage_'+sample_reads_type] = GC_percentage
    df_fastqc.loc[sample_id,'reads_len_mean_'+sample_reads_type] = reads_len_mean
    df_fastqc.loc[sample_id,'reads_len_sd_'+sample_reads_type] = reads_len_sd
    df2.loc[sample_id_full, 'sample_id'] = sample_id
    df2.loc[sample_id_full, 'sample_reads_type'] = sample_reads_type
    df2.loc[sample_id_full, 'reads_count'] = reads_count
    df2.loc[sample_id_full, 'GC_percentage'] = GC_percentage
    df2.loc[sample_id_full, 'reads_len_mean'] = reads_len_mean
    df2.loc[sample_id_full, 'reads_len_sd'] = reads_len_sd
    
print(df_fastqc.shape)
df_fastqc.head()

df_fastqc.to_excel('/home/xcao/w/20181205Hermeuptychia/20181205Reads3303/wenlinInput/3303fasta_qc.xlsx')
df2.to_excel('/home/xcao/w/20181205Hermeuptychia/20181205Reads3303/wenlinInput/3303fasta_qcList.xlsx')


```

## test SPAdes with 3303
20181205  

Based on previous experience with bacteria genome, SPAdes works pretty well: correcting errors in reads, merge assemblies of different kmer length. The eukaryote genome might be too huge for it, but still want to test it.  
It seems the 

## run Qian's pipeline to Assemble 3303
20181208  
Understanding the pipelines.  

### get files that belongs to 3303

there are two batches of data, one is used by Qian, the second batch in in Wenlin's folder.  
Separate the files to 3 types: file_pairEnd, file_singleEnd, file_MatePair. 

Store the locations of these files in `/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/`

Since the files have duplicated names, create softlinks to them by renaming them with the folder name +filename
Store in `/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/raw`

```
import os
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/raw/'
file_locs = '''/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/file_MatePair
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/file_PairEnd
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/file_SingleEnd
'''.split()
files = []
for f in file_locs:
    files += open(f).read().split()
for file in files:
    basename = os.path.basename(file)
    foldername = os.path.dirname(file)
    foldername = os.path.basename(foldername)
    outname = foldername + basename
    commandline = 'ln -s {f} {outfolder}{outname}'.format(f=file,outfolder=outfolder, outname=outname)
    os.system(commandline)
```


after this step, the content in these files were changed to those soft link path.

#### repair one file
file cannot parse fastqc. Some lines contain errors. remove three lines.
```
filename = '/export/home11/congqian/satyrs/denovo/3303/2_filter/3303ac_single.fq'
from Bio import SeqIO
fo = open(filename)
n=0
line = fo.readline()
while line:
    if n%4 == 0:
        if line[0] != '@':
            print(line)
            break
    n += 1
    line = fo.readline()

filename = '/export/home11/congqian/satyrs/denovo/3303/2_filter/3303ac_single.fq'
fout = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/raw/2_filter3303ac_single.fq'
fout = open(fout,'w')
remove_lines = [62028608,62028609,62028610]
for n, line in enumerate(open(filename)):
    if n in remove_lines:
        print(n, line)
    else:
        fout.write(line)
fout.close()
```

#### run fastqc

```
import os,glob
folder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/raw/'
files = glob.glob(folder+'*')
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/fastqc/'
fastqc = '/home/xcao/p/fastqc/FastQC/fastqc'
file_cmds = '/home/xcao/w/20181205Hermeuptychia/scripts/20181205_fastqc.cmds'
open(file_cmds,'w').write('\n'.join('{fastqc} {file} -o {outfolder} -t 1'.format(fastqc=fastqc,outfolder=outfolder,file=file) for file in files))
os.system('python /home/xcao/p/xiaolongTools/multiThread.py 32 '+file_cmds)
os.system('cd  '+outfolder+'&& unzip "*.zip"')
```

#### summarize fastqc result

```
import glob
folder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/fastqc/'
file_fastqc_data = glob.glob(folder+'**/fastqc_data.txt',recursive=True)
print(len(file_fastqc_data))

import os
import math
def fastqc2readsinfo(filename):
    l = open(filename).readlines()
    sample_id_full = os.path.basename(os.path.dirname(filename))
    sample_id = sample_id_full.split('_cor_')[0]
    if '_R1_' in sample_id_full:
        sample_reads_type = 'R1'
    elif '_R2_' in sample_id_full:
        sample_reads_type = 'R2'
    else:
        sample_reads_type = 'singleton'
    if 'combMate' in sample_id_full:
        sample_reads_type = 'combMate'+sample_reads_type
    if 'pairedM' in sample_id_full:
        sample_reads_type = 'pairedM'+sample_reads_type
    reads_count = l[6].split()[2]
    GC_percentage = int(l[9].split()[1])
    n = 0
    while n < len(l):
        if '>>Sequence Length Distribution' in l[n]:
            break
        else:
            n = n+1
    n = n+2
    reads_len_info = []
    while n < len(l):
        if l[n][0] != '>':
            _reads_len_range, _reads_count = l[n].strip().split('\t')
            if '-' in _reads_len_range:
                _rs,_re = _reads_len_range.split('-')
                _rl = (float(_rs) + float(_re))/2
            else:
                _rl = float(_reads_len_range)
            _reads_count = float(_reads_count)
            reads_len_info.append([_rl,_reads_count])
            n += 1
        else:
            break
    reads_count = sum(e[1] for e in reads_len_info)
    reads_len_mean = sum(e[0]*e[1] for e in reads_len_info)/reads_count
    reads_len_sd = math.sqrt(sum(number*number*count for number, count in reads_len_info) / reads_count - reads_len_mean * reads_len_mean)
    #print('total reads', reads_count,'reads length mean', reads_len_mean, 'reads length sd', reads_len_sd, GC_percentage)
    return sample_id, sample_reads_type, reads_count, reads_len_mean, reads_len_sd, GC_percentage,sample_id_full

import pandas as pd
df_fastqc = pd.DataFrame()
df2 = pd.DataFrame()
for f in file_fastqc_data:
    sample_id, sample_reads_type, reads_count, reads_len_mean, reads_len_sd, GC_percentage,sample_id_full= fastqc2readsinfo(f)
    df_fastqc.loc[sample_id,'reads_count_'+sample_reads_type] = reads_count
    df_fastqc.loc[sample_id,'GC_percentage_'+sample_reads_type] = GC_percentage
    df_fastqc.loc[sample_id,'reads_len_mean_'+sample_reads_type] = reads_len_mean
    df_fastqc.loc[sample_id,'reads_len_sd_'+sample_reads_type] = reads_len_sd
    df2.loc[sample_id_full, 'sample_id'] = sample_id
    df2.loc[sample_id_full, 'sample_reads_type'] = sample_reads_type
    df2.loc[sample_id_full, 'reads_count'] = reads_count
    df2.loc[sample_id_full, 'GC_percentage'] = GC_percentage
    df2.loc[sample_id_full, 'reads_len_mean'] = reads_len_mean
    df2.loc[sample_id_full, 'reads_len_sd'] = reads_len_sd

print(df_fastqc.shape)
df_fastqc.head()

df2.to_excel('/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/3303_ori_fasta_qcList.xlsx')
```

### delox mate pair reads

```
filename = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/file_MatePair'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/3_delox/'
files = open(filename).read().split()
dc_pairs = {}
import os
delox = '/home/xcao/p/xiaolongTools/utils/seq/run_delox.py'
for file in files:
    basename = os.path.basename(file)
    foldername = os.path.dirname(file)
    foldername = os.path.basename(foldername)
    key = foldername+basename.split('_R')[0]
    if key not in dc_pairs:
        dc_pairs[key] = []
    dc_pairs[key].append(file)
for k,v in dc_pairs.items():
    commondline = 'python {delox} -f {read1} -r {read2} -p {outfolder}{prefix}'.format(delox=delox, read1 = v[0], read2=v[1],outfolder=outfolder, prefix=k)
    os.system(commondline)
```

### trim reads

```


commandline_pair = '''java -jar /home/xcao/p/trinity/trinityrnaseq-Trinity-v2.8.4/trinity-plugins/Trimmomatic-0.36/trimmomatic-0.36.jar PE -phred33 {input_forward} {input_reverse} {output_forward_paired} {output_forward_unpaired} {output_reverse_paired} {output_reverse_unpaired} ILLUMINACLIP:/home/xcao/p/trinity/trinityrnaseq-Trinity-v2.8.4/trinity-plugins/Trimmomatic-0.36/adapters/all.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36 -threads 4 -trimlog {trimlog}'''

commandline_single = '''java -jar /home/xcao/p/trinity/trinityrnaseq-Trinity-v2.8.4/trinity-plugins/Trimmomatic-0.36/trimmomatic-0.36.jar SE -phred33 {file_input} {file_output} ILLUMINACLIP:/home/xcao/p/trinity/trinityrnaseq-Trinity-v2.8.4/trinity-plugins/Trimmomatic-0.36/adapters/all.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36 -threads 4 -trimlog {trimlog}'''

import os
folder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/raw/'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/'
folder_delox = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/3_delox/'
file_locs = '''/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/file_MatePair
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/file_PairEnd
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/2_filter/file_SingleEnd
'''.split()

commandlines = []
file_singleEnd = '2_filter3303 2_filter3303ac'.split()
for f in file_singleEnd:
    file_input = os.path.join(folder, f+'_single.fq')
    file_output = os.path.join(outfolder,f+'_single.Trim.fq')
    trimlog = os.path.join(outfolder,f+'.trimlog')
    commandline = commandline_single.format(file_input=file_input, file_output=file_output, trimlog=trimlog)
    commandlines.append(commandline)

file_pairEnd = '2_filter3303_250b 2_filter3303_250s 2_filter3303lp_250 2_filter3303lp_250b 2_filter3303pb_250 2_filter3303_500s 2_filter3303lp_500 2_filter3303pb_500'.split()

for f in file_pairEnd:
    input_forward = os.path.join(folder,f+'_R1.fq')
    input_reverse = os.path.join(folder,f+'_R2.fq')
    output_forward_paired = os.path.join(outfolder,f+'_R1.TrimP.fq')
    output_forward_unpaired = os.path.join(outfolder,f+'_R1.TrimS.fq')
    output_reverse_paired = os.path.join(outfolder,f+'_R2.TrimP.fq')
    output_reverse_unpaired = os.path.join(outfolder,f+'_R2.TrimS.fq')
    trimlog = os.path.join(outfolder,f+'.trimlog')
    commandline = commandline_pair.format(input_forward=input_forward, input_reverse=input_reverse, output_forward_paired=output_forward_paired, output_forward_unpaired=output_forward_unpaired, output_reverse_paired=output_reverse_paired, output_reverse_unpaired=output_reverse_unpaired, trimlog=trimlog)
    commandlines.append(commandline)

import glob
files = glob.glob(folder_delox+'*')
dc_key2pair = {}
for f in files:
    basename = os.path.basename(f)
    if basename.endswith('.single.fastq'):
        file_input = f
        file_output = os.path.join(outfolder, basename+'.Trim.fq')
        trimlog = os.path.join(outfolder,basename+'.trimlog')
        commandline = commandline_single.format(file_input=file_input, file_output=file_output, trimlog=trimlog)
        commandlines.append(commandline)
    else:
        key = basename.split('_read')[0]
        if 'matepaired' in basename:
            key = key+'matepaired'
            if key not in dc_key2pair:
                dc_key2pair[key] = []
            dc_key2pair[key].append(f)
        else:
            key = key+'paired'
            if key not in dc_key2pair:
                dc_key2pair[key] = []
            dc_key2pair[key].append(f)

for key, values in dc_key2pair.items():
    input_forward = values[0]
    input_reverse = values[1]
    output_forward_paired = os.path.join(outfolder,key+'_R1.TrimP.fq')
    output_forward_unpaired = os.path.join(outfolder,key+'_R1.TrimS.fq')
    output_reverse_paired = os.path.join(outfolder,key+'_R2.TrimP.fq')
    output_reverse_unpaired = os.path.join(outfolder,key+'_R2.TrimS.fq')
    trimlog = os.path.join(outfolder,key+'.trimlog')
    commandline = commandline_pair.format(input_forward=input_forward, input_reverse=input_reverse, output_forward_paired=output_forward_paired, output_forward_unpaired=output_forward_unpaired, output_reverse_paired=output_reverse_paired, output_reverse_unpaired=output_reverse_unpaired, trimlog=trimlog)
    commandlines.append(commandline)

file_cmds = '/home/xcao/w/20181205Hermeuptychia/scripts/20181210_Trimmomatics.cmds'
open(file_cmds,'w').write('\n'.join(commandlines))
#os.system('python /home/xcao/p/xiaolongTools/multiThread.py 8 '+file_cmds)
```

### combine reads and run SPAdes

```
import os
import glob
folder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/'
files = glob.glob(folder+'*.fq')
print(len(files))

files_single = [e for e in files if 'single' in e or 'TrimS' in e]
files_mate = [e for e in files if 'matepaired' in e and 'TrimP' in e]
files_pair = [e for e in files if 'TrimP' in e and e not in files_mate]

outfile = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/single.fq'
command = ' '.join(['cat'] + files_single) + '>' + outfile
print(command)
#os.system(command)
files_single2 = [outfile]

files_pair2 = [e for e in files_pair if 'paired' not in e]
files_pair_frommate = [e for e in files_pair if 'paired' in e]
files_pair_frommate.sort()
outfile = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/paired_from_mate_R1.fq'
command = ' '.join(['cat'] + [e for e in files_pair_frommate if '_R1' in e]) + '>' + outfile
files_pair2.append(outfile)
print(command)
#os.system(command)
outfile = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/paired_from_mate_R2.fq'
command = ' '.join(['cat'] + [e for e in files_pair_frommate if '_R2' in e]) + '>' + outfile
print(command)
#os.system(command)
files_pair2.append(outfile)

matelen = [e+'K' for e in '2 6 8 3 14 15 20'.split()]
files_mate2 = []
for e in matelen:
    matelen_files = [i for i in files_mate if e in i]
    matelen_files.sort()
    print(e, len(matelen_files))
    outfile = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate'+e+'_R1.fq'
    files_mate2.append(outfile)
    command = ' '.join(['cat'] + [e for e in matelen_files if '_R1' in e]) + '>' + outfile
    #os.system(command)
    outfile = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate'+e+'_R2.fq'
    files_mate2.append(outfile)
    command = ' '.join(['cat'] + [e for e in matelen_files if '_R2' in e]) + '>' + outfile
    #os.system(command)


from collections import defaultdict
dc_mate = defaultdict(list)
dc_pair = defaultdict(list)
for e in files_mate2:
    key = e.replace('_R1','').replace('_R2','')
    dc_mate[key].append(e)
for e in files_pair2:
    key = e.replace('_R1','').replace('_R2','')
    dc_pair[key].append(e)

commands = ['python /home/xcao/p/SPAdes/SPAdes-3.13.0-Linux/bin/spades.py -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/ -t 32 -m 5200 ']
for n, e in enumerate(files_single2):
    command = '--s{n} {file}'.format(n=n+1, file=e)
    commands.append(command)
for n, e in enumerate(dc_mate.values()):
    command = '--mp{n}-1 {f1} --mp{n}-2 {f2}'.format(n=n+1,f1=e[0],f2=e[1])
    commands.append(command)
for n, e in enumerate(dc_pair.values()):
    command = '--pe{n}-1 {f1} --pe{n}-2 {f2}'.format(n=n+1,f1=e[0],f2=e[1])
    commands.append(command)
print(' '.join(commands))

fout = open('/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/files.tab','w')
for e in files_single2:
    fout.write(os.path.basename(e)+'\t'+e+'\n')
for k, e in dc_mate.items():
    fout.write(os.path.basename(k)+'\t'+','.join(e)+'\n')
for k,e in dc_pair.items():
    fout.write(os.path.basename(k)+'\t'+','.join(e)+'\n')
fout.close()

```

### improve genome with redundance for 3303
20181213


should have use only mate pairs
```
/home/xcao/p/anaconda2/bin/python2 /home/xcao/p/redundans/redundans-master/redundans.py -t 32 -m 240 --iters 2 --usebwa -o /home/xcao/w/20181205Hermeuptychia/20181212redundans/3303 -f /home/xcao/w/20181205Hermeuptychia/20181205wenlin/3303_careful_ass0.3_t1_contig.fa -i /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate2K_R1.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate2K_R2.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate6K_R1.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate6K_R2.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate8K_R1.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate8K_R2.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate3K_R1.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate3K_R2.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate14K_R1.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate14K_R2.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate15K_R1.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate15K_R2.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate20K_R1.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/mate20K_R2.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303_500s_R2.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303_500s_R1.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303lp_500_R1.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303lp_500_R2.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303_250b_R2.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303_250b_R1.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303_250s_R2.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303_250s_R1.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303pb_250_R1.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303pb_250_R2.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303lp_250_R2.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303lp_250_R1.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303pb_500_R1.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303pb_500_R2.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303lp_250b_R1.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4_trim/2_filter3303lp_250b_R2.TrimP.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/paired_from_mate_R1.fq  /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/paired_from_mate_R2.fq

```

### SPAdes run failed. De novo assembly with redundance for 3303

```
nohup /home/xcao/p/anaconda2/bin/python2 /home/xcao/p/redundans/redundans-master/redundans.py -t 32 -m 690 --iters 4 --tmp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/tmp/ --usebwa -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/6_redundans  -i /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250b_R_unpaired.00.9_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250b_R1.TrimP.00.9_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250b_R2.TrimP.00.9_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250s_R_unpaired.00.10_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250s_R1.TrimP.00.10_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250s_R2.TrimP.00.10_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_500s_R_unpaired.00.7_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_500s_R1.TrimP.00.7_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_500s_R2.TrimP.00.7_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250_R_unpaired.00.12_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250_R1.TrimP.00.12_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250_R2.TrimP.00.12_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250b_R_unpaired.00.14_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250b_R1.TrimP.00.14_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250b_R2.TrimP.00.14_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_500_R_unpaired.00.8_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_500_R1.TrimP.00.8_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_500_R2.TrimP.00.8_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_250_R_unpaired.00.11_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_250_R1.TrimP.00.11_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_250_R2.TrimP.00.11_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_500_R_unpaired.00.13_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_500_R1.TrimP.00.13_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_500_R2.TrimP.00.13_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate2K_R_unpaired.00.0_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate2K_R1.00.0_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate2K_R2.00.0_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate3K_R_unpaired.00.3_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate3K_R1.00.3_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate3K_R2.00.3_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate6K_R_unpaired.00.1_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate6K_R1.00.1_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate6K_R2.00.1_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate8K_R_unpaired.00.2_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate8K_R1.00.2_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate8K_R2.00.2_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate14K_R_unpaired.00.4_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate14K_R1.00.4_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate14K_R2.00.4_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate15K_R_unpaired.00.5_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate15K_R1.00.5_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate15K_R2.00.5_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate20K_R_unpaired.00.6_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate20K_R1.00.6_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate20K_R2.00.6_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/paired_from_mate_R_unpaired.00.15_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/paired_from_mate_R1.00.15_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/paired_from_mate_R2.00.15_0.cor.fastq.gz \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/single.00.16_0.cor.fastq.gz >/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/6_redundans.logs &

```

### de novo assembly with platanus
20181213 de novo assembly with redundance failed. Try to run Plantunus directly

```
nohup /home/xcao/p/redundans/redundans-master/bin/platanus assemble -t 32 -tmp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/tmp/ -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/6_platanus/ -m 768 -u 0.3 -a 5 -d 0.3 -f /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250b_R_unpaired.00.9_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250b_R1.TrimP.00.9_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250b_R2.TrimP.00.9_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250s_R_unpaired.00.10_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250s_R1.TrimP.00.10_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_250s_R2.TrimP.00.10_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_500s_R_unpaired.00.7_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_500s_R1.TrimP.00.7_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303_500s_R2.TrimP.00.7_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250_R_unpaired.00.12_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250_R1.TrimP.00.12_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250_R2.TrimP.00.12_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250b_R_unpaired.00.14_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250b_R1.TrimP.00.14_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_250b_R2.TrimP.00.14_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_500_R_unpaired.00.8_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_500_R1.TrimP.00.8_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303lp_500_R2.TrimP.00.8_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_250_R_unpaired.00.11_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_250_R1.TrimP.00.11_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_250_R2.TrimP.00.11_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_500_R_unpaired.00.13_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_500_R1.TrimP.00.13_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/2_filter3303pb_500_R2.TrimP.00.13_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate2K_R_unpaired.00.0_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate2K_R1.00.0_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate2K_R2.00.0_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate3K_R_unpaired.00.3_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate3K_R1.00.3_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate3K_R2.00.3_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate6K_R_unpaired.00.1_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate6K_R1.00.1_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate6K_R2.00.1_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate8K_R_unpaired.00.2_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate8K_R1.00.2_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate8K_R2.00.2_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate14K_R_unpaired.00.4_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate14K_R1.00.4_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate14K_R2.00.4_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate15K_R_unpaired.00.5_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate15K_R1.00.5_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate15K_R2.00.5_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate20K_R_unpaired.00.6_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate20K_R1.00.6_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/mate20K_R2.00.6_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/paired_from_mate_R_unpaired.00.15_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/paired_from_mate_R1.00.15_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/paired_from_mate_R2.00.15_0.cor.fastq \
/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/single.00.16_0.cor.fastq >/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/6_platanus.logs &
```

## run Qian's pipeline for Assemble 3318
20181213

### improve genome with redundans for 3318 of wenlin
```
nohup /home/xcao/p/anaconda2/bin/python2 /home/xcao/p/redundans/redundans-master/redundans.py -t 32 -m 240 --iters 2 --tmp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/tmp/ --usebwa -o /home/xcao/w/20181205Hermeuptychia/20181212redundans/3318 -f /home/xcao/w/20181205Hermeuptychia/20181205wenlin/3318_careful_ass0.3_t1_contig.fa -i /work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_3KM_cor_R1.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_3KM_cor_R2.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_6KM_merged_R1.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_6KM_merged_R2.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_8KM_merged_R1.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_8KM_merged_R2.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_14KM_merged_R1.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_14KM_merged_R2.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_15KM_merged_R1.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_15KM_merged_R2.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_20KM_merged_R1.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318_20KM_merged_R2.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318ac_2KM_cor_R1.fastq \
/work0/wenlin/sequencing/3318_with_3312_round2/7.1_fastq_merged4scaffolding/3318ac_2KM_cor_R2.fastq >/home/xcao/w/20181205Hermeuptychia/20181212redundans/3318.log &

```

#### quast check the difference
20181214

```
/home/xcao/p/quast/quast-5.0.2/quast.py -t 32 --eukaryote --large --k-mer-stats --circos  --conserved-genes-finding /home/xcao/w/20181205Hermeuptychia/20181205wenlin/3318_careful_ass0.3_t1_contig.fa /home/xcao/w/20181205Hermeuptychia/20181205wenlin/3318_careful_ass0.3_t1_merged_t2_gapClosed_v1.fa /home/xcao/w/20181205Hermeuptychia/20181205wenlin/3318_redundans.scaffold.fa  -o /home/xcao/w/20181205Hermeuptychia/20181205evaluate_wenlin/20181214Improved3318 --no-gzip
```

it seems that Wenlin's result is better, based on N50
```
1  3318_careful_ass0.3_t1_contig, N50 = 5041, L50 = 22965, Total length = 343332627, GC % = undefined, # N's per 100 kbp =  0.00
    2  3318_careful_ass0.3_t1_merged_t2_gapClosed_v1, N50 = 304870, L50 = 811, Total length = 922282021, GC % = undefined, # N's per 100 kbp =  7396.64
    3  3318_redundans.scaffold, N50 = 13277, L50 = 15667, Total length = 713010486, GC % = undefined, # N's per 100 kbp =  2783.77

```

## Annotation

### run Trinity


Trinity doesn't run well
`Thursday, December 13, 2018: 02:13:11	CMD: cat 3352_RNA_R2.fq.PwU.qtrim.fq | seqtk-trinity seq -A - >> right.fa
Error, not recognizing read name formatting: [DJDP2NM1:317:HKMT5ADXX:2:1101:1237:2193]`

correct read names

```
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/'
files_left = '''/alea/home11/congqian/satyrs/RNAseq/3072_RNA_R1.fq /alea/home11/congqian/satyrs/RNAseq/3305_RNA_R1.fq /alea/home11/congqian/satyrs/RNAseq/3320_RNA_R1.fq /alea/home11/congqian/satyrs/RNAseq/3352_RNA_R1.fq /alea/local/Qian_biohpc_backup/project/96_RNA/step2_fix/3305_R1.fq /alea/local/Qian_biohpc_backup/project/96_RNA/step2_fix/3320_R1.fq /alea/local/Qian_biohpc_backup/project/96_RNA/step2_fix/3352_R1.fq /alea/local/Qian_biohpc_backup/project/96_RNA/step2_fix/4003_R1.fq'''.split()
files_right = '''/alea/home11/congqian/satyrs/RNAseq/3072_RNA_R2.fq /alea/home11/congqian/satyrs/RNAseq/3305_RNA_R2.fq /alea/home11/congqian/satyrs/RNAseq/3320_RNA_R2.fq /alea/home11/congqian/satyrs/RNAseq/3352_RNA_R2.fq /alea/local/Qian_biohpc_backup/project/96_RNA/step2_fix/3305_R2.fq /alea/local/Qian_biohpc_backup/project/96_RNA/step2_fix/3320_R2.fq /alea/local/Qian_biohpc_backup/project/96_RNA/step2_fix/3352_R2.fq /alea/local/Qian_biohpc_backup/project/96_RNA/step2_fix/4003_R2.fq'''.split()

import os
from multiprocessing import Pool

def correctName(f,addon='/1'):
    outfile = os.path.join(outfolder,os.path.basename(f))
    fout = open(outfile,'w')
    for n,l in enumerate(open(f)):
        if n%4 == 0:
            fout.write(l.replace(' ','_').replace('\n',addon+'\n'))
        else:
            fout.write(l)
    fout.close()


pool = Pool(16)
commands = [[e,'/1'] for e in files_left] + [[e,'/2'] for e in files_right]
pool.starmap(correctName,commands)
pool.close()
```

```
nohup /home/xcao/p/trinity/trinityrnaseq-Trinity-v2.8.4/Trinity  --seqType fq  --max_memory 240G --CPU 32 --trimmomatic --no_normalize_reads --output /home/xcao/w/20181205Hermeuptychia/20181212Annotation/Trinity/ \
    --left /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3072_RNA_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3305_RNA_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3320_RNA_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3352_RNA_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3305_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3320_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3352_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/4003_R1.fq \
    --right /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3072_RNA_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3305_RNA_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3320_RNA_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3352_RNA_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3305_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3320_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3352_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/4003_R2.fq \
    >/home/xcao/w/20181205Hermeuptychia/20181212Annotation/Trinity/trinity.log &
```

#### correct trinity errors

change `/home/xcao/p/trinity/trinityrnaseq-Trinity-v2.8.4/util/support_scripts/../../Trinity --single "/home/xcao/w/20181205Hermeuptychia/20181212Annotation/Trinity/read_partitions/Fb_0/CBin_0/c4.trinity.reads.fa" --output "/home/xcao/w/20181205Hermeuptychia/20181212Annotation/Trinity/read_partitions/Fb_0/CBin_0/c4.trinity.reads.fa.out" --CPU 1 --max_memory 1G --run_as_paired --seqType fa --trinity_complete --full_cleanup /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3305_RNA_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3320_RNA_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3352_RNA_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3305_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3320_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3352_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/4003_R1.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3305_RNA_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3320_RNA_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3352_RNA_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3305_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3320_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/3352_R2.fq /home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/4003_R2.fq` to `/home/xcao/p/trinity/trinityrnaseq-Trinity-v2.8.4/util/support_scripts/../../Trinity --single "/home/xcao/w/20181205Hermeuptychia/20181212Annotation/Trinity/read_partitions/Fb_0/CBin_0/c4.trinity.reads.fa" --output "/home/xcao/w/20181205Hermeuptychia/20181212Annotation/Trinity/read_partitions/Fb_0/CBin_0/c4.trinity.reads.fa.out" --CPU 1 --max_memory 1G --run_as_paired --seqType fa --trinity_complete --full_cleanup`

```
import os
filein = '/home/xcao/w/20181205Hermeuptychia/20181212Annotation/Trinity/recursive_trinity.cmds'
os.rename(filein, filein+'.backup')
fout = open(filein,'w')
for line in open(filein+'.backup'):
    fout.write(line.split('--full_cleanup')[0]+' --full_cleanup\n')
fout.close()
```

#### run Trinity, genome guided 3303
20181214

```
nohup /home/xcao/p/trinity/trinityrnaseq-Trinity-v2.8.4/Trinity   --max_memory 240G --CPU 32  --no_normalize_reads --output /home/xcao/w/20181205Hermeuptychia/20181212Annotation/Trinity_3303GenomeGuided/ --genome_guided_bam /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/merged.bam --genome_guided_max_intron 100000 >/home/xcao/w/20181205Hermeuptychia/20181212Annotation/Trinity_3303GenomeGuided/trinity.log &
```

###  step 7 align reads to wenlin's 3303
20181213  
/home/xcao/w/20181205Hermeuptychia/20181205wenlin/3303_careful_ass0.3_t1_scaffold.fa

#### filter and keep scaffolds longer than 200bp

```
filein = '/home/xcao/w/20181205Hermeuptychia/20181205wenlin/3303_careful_ass0.3_t1_scaffold.fa'
fileout = '/home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa'

from Bio import SeqIO

total_scaffold = 0
keep_scaffold = 0
fout = open(fileout,'w')
for s in SeqIO.parse(filein,'fasta'):
    total_scaffold += 1
    if len(s.seq)>=200:
        fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
        keep_scaffold += 1
fout.close()
print(total_scaffold,'scaffolds before. After filtering:', keep_scaffold)
```

#### bwa index
```
bwa index /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa
```

#### align reads

run scripts for generate SPAdes command line. we got dc_single, dc_pair and dc_mate. Generate bwa running scripts based on that. save a file with like
`ID seq1,seq2`
```
import os

commandline = '/home/xcao/p/bin/bwa mem -t 48 /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa {filein} >{fileout}.sam && /home/xcao/p/bin/samtools sort -@48 -o {fileout} {fileout}.sam  && rm {fileout}.sam '
qsub = '''#!/bin/bash
#$ -S /bin/sh
#$ -cwd
#$ -j y
source /home/xcao/.bashrc
'''

files_info = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/files.tab'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/'
f = open(files_info)
commandlines = []
for line in f:
    k,v = line.split()
    fileout = os.path.join(outfolder,k+'.bam')
    filein = v.replace(',',' ')
    c = commandline.format(filein=filein, fileout=fileout)
    commandlines.append(c)
    open(os.path.join(outfolder,'cmds/qsub_'+k),'w').write(qsub+c)
f.close()
#open(os.path.join(outfolder,'commands'),'w').write('\n'.join(commandlines))


```

```
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_2_filter3303_250b.TrimP.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_2_filter3303_250s.TrimP.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_2_filter3303_500s.TrimP.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_2_filter3303lp_250.TrimP.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_2_filter3303lp_250b.TrimP.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_2_filter3303lp_500.TrimP.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_2_filter3303pb_250.TrimP.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_2_filter3303pb_500.TrimP.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_mate2K.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_mate3K.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_mate6K.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_mate8K.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_mate14K.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_mate15K.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_mate20K.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_paired_from_mate.fq
qsub -pe mpi 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/cmds/qsub_single.fq
```

#### jing's scripts

index fasta file
```
samtools faidx /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa
```

```
samtools faidx /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa
java -jar /home/xcao/p/picard/picard2.18.20.jar CreateSequenceDictionary R=/home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads
samtools merge merged.tmp.bam 2_filter3303_250b.TrimP.fq.bam 2_filter3303_250s.TrimP.fq.bam 2_filter3303_500s.TrimP.fq.bam 2_filter3303lp_250.TrimP.fq.bam 2_filter3303lp_250b.TrimP.fq.bam 2_filter3303lp_500.TrimP.fq.bam 2_filter3303pb_250.TrimP.fq.bam 2_filter3303pb_500.TrimP.fq.bam mate2K.fq.bam mate3K.fq.bam mate6K.fq.bam mate8K.fq.bam mate14K.fq.bam mate15K.fq.bam mate20K.fq.bam paired_from_mate.fq.bam single.fq.bam
samtools sort merged.tmp.bam -o merged.bam
rm merged.tmp.bam
java -Djava.io.tmpdir=merged_tmp -jar /home/xcao/p/picard/picard2.18.20.jar MarkDuplicates INPUT=merged.bam OUTPUT=merged_dedup.bam METRICS_FILE=merged.metrics.txt >& merged.markdup.log
java -Djava.io.tmpdir=merged_tmp -jar /home/xcao/p/picard/picard2.18.20.jar  AddOrReplaceReadGroups  INPUT=merged_dedup.bam  OUTPUT=merged_addrg.bam SORT_ORDER=coordinate RGID=group1 RGLB=lib1 RGPL=illumina RGPU=unit1 RGSM=merged CREATE_INDEX=True >& merged.add_readgroup.log
java -Djava.io.tmpdir=merged_tmp -jar /home/xcao/p/picard/picard2.18.20.jar  BuildBamIndex INPUT=merged_addrg.bam >& merged.index.log
nohup java -Djava.io.tmpdir=merged_tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -T RealignerTargetCreator -R /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa -I merged_addrg.bam -o merged_realignment_targets.list >& merged_prepare_realign.log &
nohup java -Djava.io.tmpdir=merged_tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -T IndelRealigner  -R /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa -I  merged_addrg.bam -targetIntervals merged_realignment_targets.list -o  merged_realigned.bam >& 1185.realign.log &&\
nohup java -Djava.io.tmpdir=merged_tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -T  UnifiedGenotyper -nct 32 -R /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa  -stand_call_conf 30  -I merged_realigned.bam -o merged_snp.vcf --heterozygosity 0.02 --indel_heterozygosity 0.0025 --output_mode EMIT_ALL_SITES >& merged.snp.log &
rm -rf merged_tmp
```

#### run for 3318

```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/
cp /home/xcao/w/20181205Hermeuptychia/20181205wenlin/3318_careful_ass0.3_t1_merged_t2_gapClosed_v1.fa ./3318_genome.fa
bwa index 3318_genome.fa
```

```
filein = '/home/xcao/w/20181205Hermeuptychia/20181205wenlin/3318_careful_ass0.3_t1_merged_t2_gapClosed_v1.fa'
fileout = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/3318_genome.fa'

from Bio import SeqIO

total_scaffold = 0
keep_scaffold = 0
fout = open(fileout,'w')
for s in SeqIO.parse(filein,'fasta'):
    total_scaffold += 1
    if len(s.seq)>=200:
        fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
        keep_scaffold += 1
fout.close()
print(total_scaffold,'scaffolds before. After filtering:', keep_scaffold)

#conclusion: all scaffolds for 3318 longer than 200bp



import os

genome_ref = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/3318_genome.fa'
files_info = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/files.tab'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/'

commandline = 'cd {outfolder} && /home/xcao/p/bin/bwa mem -t 48 {genome_ref} {filein} >{fileout}.sam && /home/xcao/p/bin/samtools sort -@48 -o {fileout} {fileout}.sam  && rm {fileout}.sam '
qsub = '''#!/bin/bash
#$ -S /bin/sh
#$ -cwd
#$ -j y
source /home/xcao/.bashrc
'''


if not os.path.exists(outfolder+'cmds'):
    os.makedirs(outfolder+'cmds')
f = open(files_info)
commandlines = []
for line in f:
    k,v = line.split()
    fileout = os.path.join(outfolder,k+'.bam')
    filein = v.replace(',',' ')
    c = commandline.format(filein=filein, fileout=fileout, outfolder=outfolder, genome_ref=genome_ref)
    commandlines.append(c)
    print(c)
    open(os.path.join(outfolder,'cmds/qsub_'+k),'w').write(qsub+c)
f.close()
open(os.path.join(outfolder,'commands'),'w').write('\n'.join(commandlines))



```

```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads
samtools merge merged.tmp.bam 2KM.bam 3KM.bam 6KM.bam 8KM.bam 14KM.bam 15KM.bam 20KM.bam 200bp.bam 250bp.bam 500bp.bam
samtools sort merged.tmp.bam -o merged.bam
rm merged.tmp.bam
```

### step 8 improve assembly

some codes
```
import os

commandline = '/home/xcao/p/bin/bwa mem -t 48 /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa {filein} >{fileout}.sam && /home/xcao/p/bin/samtools sort -@48 -o {fileout} {fileout}.sam  && rm {fileout}.sam '
qsub = '''#!/bin/bash
#$ -S /bin/sh
#$ -cwd
#$ -j y
source /home/xcao/.bashrc
'''

files_info = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/files.tab'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/'
f = open(files_info)
commandlines = []
for line in f:
    k,v = line.split()
    fileout = os.path.join(outfolder,k+'.bam')
    filein = v.replace(',',' ')
    c = commandline.format(filein=filein, fileout=fileout)
    commandlines.append(c)
    open(os.path.join(outfolder,'cmds/qsub_'+k),'w').write(qsub+c)
f.close()

commandline = '''/home/xcao/p/bin/samtools merge -@ 48 {outfolder}merged.tmp.bam {outfolder}*.bam
/home/xcao/p/bin/samtools sort -@ 48 {outfolder}merged.tmp.bam -o {outfolder}merged.bam
rm {outfolder}merged.tmp.bam

java -Djava.io.tmpdir={outfolder}merged_tmp -jar /home/xcao/p/picard/picard2.18.20.jar MarkDuplicates INPUT={outfolder}merged.bam OUTPUT={outfolder}merged_dedup.bam METRICS_FILE={outfolder}merged.metrics.txt >& {outfolder}merged.markdup.log
java -Djava.io.tmpdir={outfolder}merged_tmp -jar /home/xcao/p/picard/picard2.18.20.jar  AddOrReplaceReadGroups  INPUT={outfolder}merged_dedup.bam  OUTPUT={outfolder}merged_addrg.bam SORT_ORDER=coordinate RGID=group1 RGLB=lib1 RGPL=illumina RGPU=unit1 RGSM={outfolder}merged CREATE_INDEX=True >& {outfolder}merged.add_readgroup.log
java -Djava.io.tmpdir={outfolder}merged_tmp -jar /home/xcao/p/picard/picard2.18.20.jar  BuildBamIndex INPUT={outfolder}merged_addrg.bam >& {outfolder}merged.index.log

java -Djava.io.tmpdir={outfolder}merged_tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -T RealignerTargetCreator -R /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa -I {outfolder}merged_addrg.bam -o {outfolder}merged_realignment_targets.list >& {outfolder}merged_prepare_realign.log
java -Djava.io.tmpdir={outfolder}merged_tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -T IndelRealigner -R /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa -I  {outfolder}merged_addrg.bam -targetIntervals {outfolder}merged_realignment_targets.list -o  {outfolder}merged_realigned.bam >& {outfolder}merged.realign.log
java -Djava.io.tmpdir={outfolder}merged_tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -nct 32 -R /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa -T  UnifiedGenotyper -stand_call_conf 30 -stand_emit_conf 10 -I {outfolder}merged_realigned.bam -o {outfolder}merged_snp.vcf --heterozygosity 0.02 --indel_heterozygosity 0.0025 --output_mode EMIT_ALL_SITES >& {outfolder}merged.snp.log
rm -rf {outfolder}merged_tmp
'''.format(outfolder=outfolder)
print(commandline)

import glob
import os
files = glob.glob('/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/5_SPAdes/corrected/*.gz')
for f in files:
    print('pigz -d -p 32 ',f)







```

#### get reads count per scaffold
```
source activate bio
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/
cp /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa ./
samtools index ../7_align_reads/merged.bam
samtools idxstats ../7_align_reads/merged.bam >merged.idxstats
```

#### get coverage and plot to select depth min
set depth min to 30
```
python /home/xcao/p/xiaolongTools/utils/annotation/get_coverage_histogram.py -g /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa  -r /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.idxstats
```

#### run blast to compare the genome with itself
```
makeblastdb -in wenlin3303min200.fa -input_type fasta -dbtype nucl -parse_seqid
blastn -query wenlin3303min200.fa -db wenlin3303min200.fa -out all.blast2 -outfmt '6 qseqid sseqid pident evalue qlen qstart qend slen sstart send' -num_threads 48 -word_size 36 -evalue 0.0000000001
```

It is slow to run in single node. split to 14 parts
```
python /home/xcao/p/xiaolongTools/utils/seq/splitFasta2Nparts.py -s /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200.fa -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/blast/seq -N 14
```

run blast
```
workfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/'
txt = '''source activate bio && cd {workfolder} &&blastn -query ./blast/seq{n} -db wenlin3303min200.fa -out ./blast/seq{n}.blast -outfmt '6 qseqid sseqid pident evalue qlen qstart qend slen sstart send' -num_threads 48 -word_size 36 -evalue 0.0000000001 &'''
fout = open(workfolder+'blast.cmds','w')
fout.write('\n'.join([txt.format(workfolder=workfolder, n=n) for n in range(14)]))
fout.close()
```
combine
```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/ && cat ./blast/*.blast >all.blast &
```

#### filter the genome
```
file_genome="/home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa"
file_stats="/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.stats"
file_blast = "/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/all.blast"
depth_min=100
outfile="/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa"
identity_min = 90

 python /home/xcao/p/xiaolongTools/utils/annotation/improve_scaffold_threads.py -i /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa -s /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.stats -b /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/all.blast -d 100 -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa -m 90 -t 32
 
python /home/xcao/p/xiaolongTools/utils/annotation/improve_scaffold_threads.py -i /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa -s /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.stats -b /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/all.blast -d 120 -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter2.fa -m 85 -t 32 & 

python /home/xcao/p/xiaolongTools/utils/annotation/improve_scaffold_threads.py -i /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa -s /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.stats -b /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/all.blast -d 100 -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter3.fa -m 95 -t 32 -M 500 & 

#get depth hist
python /home/xcao/p/xiaolongTools/utils/annotation/get_coverage_histogram.py -g /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa  -r /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.idxstats -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.filter

python /home/xcao/p/xiaolongTools/utils/annotation/get_coverage_histogram.py -g /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa  -r /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.idxstats -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.filter
```

#### get depth for each base
```
samtools depth -a /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/merged.bam >/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/9_identify_repeats_by_coverage/merged.depth &
```

#### run for 3318

get reads count per scaffold
```
source activate bio
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/
ln -s ../7_align_reads/3318_genome.fa ./
samtools index ../7_align_reads/merged.bam
samtools idxstats ../7_align_reads/merged.bam >merged.idxstats
```
get coverage and plot to select depth min
```
python /home/xcao/p/xiaolongTools/utils/annotation/get_coverage_histogram.py -g 3318_genome.fa  -r merged.idxstats
```
run blast to compare the genome with itself
```
makeblastdb -in 3318_genome.fa -input_type fasta -dbtype nucl 
blastn -query 3318_genome.fa -db 3318_genome.fa -out all.blast -outfmt '6 qseqid sseqid pident evalue qlen qstart qend slen sstart send' -num_threads 48 -word_size 36 -evalue 0.0000000001 &
```


split to 14 parts
```
python /home/xcao/p/xiaolongTools/utils/seq/splitFasta2Nparts.py -s /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/3318_genome.fa -o /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/blast/seq -N 14
```

run blast
```
workfolder = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/'
txt = '''source activate bio && cd {workfolder} &&blastn -query ./blast/seq{n} -db 3318_genome.fa -out ./blast/seq{n}.blast -outfmt '6 qseqid sseqid pident evalue qlen qstart qend slen sstart send' -num_threads 48 -word_size 36 -evalue 0.0000000001 &'''
fout = open(workfolder+'blast.cmds','w')
fout.write('\n'.join([txt.format(workfolder=workfolder, n=n) for n in range(14)]))
fout.close()
```
combine
```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/ && cat ./blast/*.blast >all.blast &
```

get depth for each base
```
samtools depth -a /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/merged.bam >/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318//9_identify_repeats_by_coverage/merged.depth &
```

filter the genome
```
python /home/xcao/p/xiaolongTools/utils/annotation/improve_scaffold_threads.py -i /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/3318_genome.fa -s /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/merged.stats -b /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/all.blast -d 120 -o /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/3318_genomefilter1.fa -m 90  -t 32 &

python /home/xcao/p/xiaolongTools/utils/annotation/improve_scaffold_threads.py -i /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/3318_genome.fa -s /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/merged.stats -b /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/all.blast -d 130 -o /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/3318_genomefilter2.fa -m 85  -t 32 &

 python /home/xcao/p/xiaolongTools/utils/annotation/improve_scaffold_threads.py -i /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/3318_genome.fa -s /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/merged.stats -b /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/all.blast -d 120 -o /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/3318_genomefilter3.fa -m 95  -t 32 -M 500 &
 
#get depth hist
python /home/xcao/p/xiaolongTools/utils/annotation/get_coverage_histogram.py -g /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa  -r /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.idxstats -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.filter


python /home/xcao/p/xiaolongTools/utils/annotation/get_coverage_histogram.py -g /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter2.fa  -r /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.idxstats -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/merged.filter2
```

### Step 9 identify repeats by coverage



```
python2 partition_scafs_step1.py /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa.scaf_size ../8_improve_assembly/wenlin3303min200filter.fa  > partition_scafs_1
python2 partition_scafs_step2.py partition_scafs_1 > partition_scafs_2
scp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/merged.bam ks2073@ls5.tacc.utexas.edu:/scratch/05920/ks2073/20181206Junonia/9_identify_repeats_by_coverage/
scp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/7_align_reads/merged.bam.bai ks2073@ls5.tacc.utexas.edu:/scratch/05920/ks2073/20181206Junonia/9_identify_repeats_by_coverage/
scp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/9_identify_repeats_by_coverage/get_coverage_cmds ks2073@ls5.tacc.utexas.edu:/scratch/05920/ks2073/20181206Junonia/9_identify_repeats_by_coverage/
mkdir run
mkdir run_result
python 
split get_coverage_cmds ./run/cmd_ -d -l 50000 -a 10

```




after finish running the commands, combine the result
```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/9_identify_repeats_by_coverage
cat ./run_result/* > get_coverage_result
paste partition_scafs_2 get_coverage_result > segment_readcount_100
python2 get_readcount_histogram.py segment_readcount_100 > segment_readcount_100.hist
#check the segment_readcount_100.hist to get the peak coverage and modify the paramter in get_potential_repeats_step1.py
cat segment_readcount_100 | awk '{printf "%s\t%s\t%s\t%s\n",$1,$2,$3,$3*100/$2}' > segment_coverage_100
python2 get_potential_repeats_step1.py > potential_repeats_1
python2 get_potential_repeats_step2.py potential_repeats_1 ../8_improve_assembly/wenlin3303min200.fa
cat potential_repeats/scaffold* > repeats.fa #too many input, do not work
python combine_files.py
```

combine_files.py
```
import glob
import os
files = glob.glob('./potential_repeats/scaffold*')
print('number of files to combine', len(files))
results = [open(f).read() for f in files]
open('repeats.fa','w').write(''.join(results))
```

#### 3318

```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/9_identify_repeats_by_coverage
python /home/xcao/p/xiaolongTools/utils/annotation/get_genome_scaf_size.py ../8_improve_assembly/3318_genomefilter1.fa 
python2 /home/xcao/p/xiaolongTools/utils/annotation/partition_scafs_step1.py genome.scf_size ../8_improve_assembly/3318_genomefilter1.fa  > partition_scafs_1
python2 /home/xcao/p/xiaolongTools/utils/annotation/partition_scafs_step2.py partition_scafs_1 > partition_scafs_2
python /home/xcao/p/xiaolongTools/utils/annotation/get_segment_coverage_with_depth_file.py partition_scafs_2 merged.depth
python2 /home/xcao/p/xiaolongTools/utils/annotation/get_readcount_histogram.py segment_coverage_100 > segment_coverage_100.hist

#set CUTOFF to 530 in get_potential_repeats_step1.py
python2 get_potential_repeats_step1.py > potential_repeats_1
python2 get_potential_repeats_step2.py potential_repeats_1 ../8_improve_assembly/3318_genomefilter1.fa
python combine_files.py
```

###  step 10 run RepeatModeler
```
scp /home/xcao/w/genomes/Hermeuptychia/wenlin3303min200.fa s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/2018Hermeuptychia/20181219RepeatModeler/

#bioHPC
#!/bin/bash
#SBATCH --job-name=repeatModeler
#SBATCH --partition=32GB
#SBATCH --nodes=1
#SBATCH --ntasks=32
#SBATCH --time=10-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL
cd /work/biophysics/s185491/2018Hermeuptychia/20181219RepeatModeler/
module add repeatModeler/1.0.11
module add repeatMasker/4.0.7
ln -s /usr/lib64/libgnutls.so.28.43.0 /home2/s185491/p/lib/libgnutls.so.26
export LD_LIBRARY_PATH=/home2/s185491/p/lib/
/cm/shared/apps/repeatModeler/1.0.11/1.0.11/RepeatModeler-open-1.0.11/BuildDatabase -name 3303 wenlin3303min200.fa
/cm/shared/apps/repeatModeler/1.0.11/1.0.11/RepeatModeler-open-1.0.11/RepeatModeler  -database 3303 -engine ncbi -pa 31


```

for 3318
```
scp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/3318_genome.fa s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/2018Hermeuptychia/20181219RepeatModeler/

#bioHPC
#!/bin/bash
#SBATCH --job-name=repeatModeler
#SBATCH --partition=32GB
#SBATCH --nodes=1
#SBATCH --ntasks=32
#SBATCH --time=10-00:00:00
#SBATCH --output=0.%j.out
#SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL
cd /work/biophysics/s185491/2018Hermeuptychia/20181219RepeatModeler/
module add repeatModeler/1.0.11
module add repeatMasker/4.0.7
ln -s /usr/lib64/libgnutls.so.28.43.0 /home2/s185491/p/lib/libgnutls.so.26
export LD_LIBRARY_PATH=/home2/s185491/p/lib/
/cm/shared/apps/repeatModeler/1.0.11/1.0.11/RepeatModeler-open-1.0.11/BuildDatabase -name 3318 3318_genome.fa
/cm/shared/apps/repeatModeler/1.0.11/1.0.11/RepeatModeler-open-1.0.11/RepeatModeler  -database 3318 -engine ncbi -pa 31


```


###  step 11_classify_repeats

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/11_classify_repeats
cp ../9_identify_repeats_by_coverage/repeats.fa ./repeats_by_coverage.fa
cp ../10_identify_repeats_by_repeatmodeller/3303-families.fa ./repeats_by_repeatmodeller.fa
split -l 7000 -d repeats_by_coverage.fa repeats_
# split repeats_by_repeatmodeller.fa manually and submit through the webstie http://www.girinst.org/censor/ 

wget -O repeats_00.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=11429&lib=root"      &
wget -O repeats_01.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=31832&lib=root"      &
wget -O repeats_02.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=32555&lib=root"      &
wget -O repeats_03.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=33141&lib=root"      &
wget -O repeats_04.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=34724&lib=root"      &
wget -O repeats_05.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=36084&lib=root"      &
wget -O repeats_06.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=37207&lib=root"      &
wget -O repeats_07.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=112585&lib=root"      &
wget -O repeats_08.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=112593&lib=root"      &
wget -O repeats_09.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=113052&lib=root"      &
wget -O repeats_10.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=113243&lib=root"      &
wget -O repeats_11.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=114978&lib=root"      &
wget -O repeats_12.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=12106&lib=root"      &
wget -O repeats_13.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=58051&lib=root"      &
wget -O repeats_14.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=58869&lib=root"      &
wget -O repeats_15.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=59701&lib=root"      &
wget -O repeats_16.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=60263&lib=root"      &
wget -O repeats_17.html    "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=60833&lib=root"      &

cat *.html >repeats_results.html
grep '^<table' repeats_results.html | grep 'MAP-NAME' > repeats_results_step0
python2 parse_CENSOR_step1.py repeats_results_step0 > repeats_results_step1
python2 parse_CENSOR_step2.py repeats_results_step1 > repeats_results_step2
cat repeats_by_coverage.fa repeats_by_repeatmodeller.fa >repeats_both.fa

python2 count_aa.py repeats_both.fa > repeats.size
python2 parse_CENSOR_step3.py repeats_results_step2 repeats.size > repeats_results_step3
python2 rename_repeats.py repeats_both.fa repeats_results_step3 3303_repeats.fa 3303_lib.fa Psen
cp /export/home5/qc_4/protein_annotation_glaucus/get_repeat_lib/common_lib.fa ./
cat 3303_lib.fa common_lib.fa > common_lib_new.fa
cat 3303_repeats.fa common_lib.fa > 3303_replib_final.fa

```

#### 3318
```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/11_classify_repeats/
cp ../9_identify_repeats_by_coverage/repeats.fa ./repeats_by_coverage.fa
cp ../10_identify_repeats_by_repeatmodeller/3318-families.fa ./repeats_by_repeatmodeller.fa
split -l 6000 -d repeats_by_coverage.fa repeats_

wget -O repeats_00.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=73174&lib=root" &
wget -O repeats_01.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=73197&lib=root" &
wget -O repeats_02.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=73410&lib=root" &
wget -O repeats_03.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=73835&lib=root" &
wget -O repeats_04.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=74226&lib=root" &
wget -O repeats_05.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=74875&lib=root" &
wget -O repeats_06.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=80225&lib=root" &
wget -O repeats_07.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=84706&lib=root" &
wget -O repeats_08.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=85666&lib=root" &
wget -O repeats_09.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=86231&lib=root" &
wget -O repeats_10.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=86560&lib=root" &
wget -O repeats_11.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=87427&lib=root" &
wget -O repeats_12.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=87822&lib=root" &
wget -O repeats_13.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=88463&lib=root" &
wget -O repeats_14.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=95194&lib=root" &
wget -O repeats_15.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=95725&lib=root" &
wget -O repeats_16.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=96178&lib=root" &
wget -O repeats_17.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=96834&lib=root" &
wget -O repeats_18.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=97883&lib=root" &
wget -O repeats_19.html  "https://www.girinst.org/cgi-bin/censor/show_results.cgi?id=98282&lib=root" &

cat *.html >repeats_results.html
grep '^<table' repeats_results.html | grep 'MAP-NAME' > repeats_results_step0
python2 parse_CENSOR_step1.py repeats_results_step0 > repeats_results_step1
python2 parse_CENSOR_step2.py repeats_results_step1 > repeats_results_step2
cat repeats_by_coverage.fa repeats_by_repeatmodeller.fa >repeats_both.fa

python2 count_aa.py repeats_both.fa > repeats.size
python2 parse_CENSOR_step3.py repeats_results_step2 repeats.size > repeats_results_step3
python2 rename_repeats.py repeats_both.fa repeats_results_step3 3318_repeats.fa 3318_lib.fa Psen
cp /export/home5/qc_4/protein_annotation_glaucus/get_repeat_lib/common_lib.fa ./
cat 3318_lib.fa common_lib.fa > common_lib_new.fa
cat 3318_repeats.fa common_lib.fa > 3318_replib_final.fa

```

### step 12_mask_repeats

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/12_mask_repeats
cp /export/home11/congqian/3314/12_mask_repeats/*.py ./
cp ../11_classify_repeats/3303_replib_final.fa ./
cp ../8_improve_assembly/wenlin3303min200filter.fa ./
/export/home5/qc_4/software/repeatmasker/RepeatMasker -lib 3303_replib_final.fa -pa 32 -div 30 wenlin3303min200filter.fa >& repeatmasker1.log
/export/home5/qc_4/software/repeatmasker/RepeatMasker -species all -pa 32 -div 30 wenlin3303min200filter.fa.masked >& repeatmasker2.log
cat wenlin3303min200filter.fa.masked.out >> wenlin3303min200filter.fa.out
python2 merge_repeats_step1.py wenlin3303min200filter.fa.out 3303_repeats_step1
python2 merge_repeats_step2.py 3303_repeats_step1 3303_repeats.gff
python2 get_repeat_counts_and_covs.py 3303_repeats.gff > 3303_repeats.sum

```

#### 3318
```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/12_mask_repeats
cp /export/home11/congqian/3314/12_mask_repeats/*.py ./
cp ../11_classify_repeats/3318_replib_final.fa ./
cp ../8_improve_assembly/3318_genomefilter1.fa ./
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/RepeatMasker -lib 3318_replib_final.fa -pa 32 -div 30 3318_genomefilter1.fa >& repeatmasker1.log
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/RepeatMasker -species all -pa 32 -div 30 3318_genomefilter1.fa.masked >& repeatmasker2.log
cat 3318_genomefilter1.fa.masked.out >>3318_genomefilter1.fa.out
python2 merge_repeats_step1.py 3318_genomefilter1.fa.out 3318_repeats_step1
python2 merge_repeats_step2.py 3318_repeats_step1 3318_repeats.gff
python2 get_repeat_counts_and_covs.py 3318_repeats.gff > 3318_repeats.sum

```

### step 13_align_reads_v1

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/13_align_reads_v1/
ln -s ../8_improve_assembly/wenlin3303min200filter.fa ./
bwa index wenlin3303min200filter.fa
```

```
import os

commandline = 'cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/13_align_reads_v1/ && /home/xcao/p/bin/bwa mem -t 48 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/13_align_reads_v1/wenlin3303min200filter.fa {filein} >{fileout}.sam && /home/xcao/p/bin/samtools sort -@48 -o {fileout} {fileout}.sam  && rm {fileout}.sam '
qsub = '''#!/bin/bash
#$ -S /bin/sh
#$ -cwd
#$ -j y
source /home/xcao/.bashrc
'''

files_info = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/4.1InputSPAdes/files.tab'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/13_align_reads_v1/'
if not os.path.exists(outfolder+'cmds'):
    os.makedirs(outfolder+'cmds')
f = open(files_info)
commandlines = []
for line in f:
    k,v = line.split()
    fileout = os.path.join(outfolder,k+'.bam')
    filein = v.replace(',',' ')
    c = commandline.format(filein=filein, fileout=fileout)
    commandlines.append(c)
    print(c)
    open(os.path.join(outfolder,'cmds/qsub_'+k),'w').write(qsub+c)
f.close()
open(os.path.join(outfolder,'commands'),'w').write('\n'.join(commandlines))
```

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/13_align_reads_v1 && \
samtools merge merged.tmp.bam 2_filter3303_250b.TrimP.fq.bam 2_filter3303_250s.TrimP.fq.bam 2_filter3303_500s.TrimP.fq.bam 2_filter3303lp_250.TrimP.fq.bam 2_filter3303lp_250b.TrimP.fq.bam 2_filter3303lp_500.TrimP.fq.bam 2_filter3303pb_250.TrimP.fq.bam 2_filter3303pb_500.TrimP.fq.bam mate2K.fq.bam mate3K.fq.bam mate6K.fq.bam mate8K.fq.bam mate14K.fq.bam mate15K.fq.bam mate20K.fq.bam paired_from_mate.fq.bam single.fq.bam &&\
samtools sort -@ 32 merged.tmp.bam -o merged.bam &&\
rm merged.tmp.bam &
```

#### 3318
```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/13_align_reads_v1
ln -s ../8_improve_assembly/3318_genomefilter1.fa ./genome.fa
bwa index genome.fa
```
```
import os

commandline = 'cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/13_align_reads_v1/ && /home/xcao/p/bin/bwa mem -t 48 /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/13_align_reads_v1/genome.fa {filein} >{fileout}.sam && /home/xcao/p/bin/samtools sort -@48 -o {fileout} {fileout}.sam  && rm {fileout}.sam '
qsub = '''#!/bin/bash
#$ -S /bin/sh
#$ -cwd
#$ -j y
source /home/xcao/.bashrc
'''

files_info = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/7_align_reads/files.tab'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/13_align_reads_v1/'
if not os.path.exists(outfolder+'cmds'):
    os.makedirs(outfolder+'cmds')
f = open(files_info)
commandlines = []
for line in f:
    k,v = line.split()
    fileout = os.path.join(outfolder,k+'.bam')
    filein = v.replace(',',' ')
    c = commandline.format(filein=filein, fileout=fileout)
    commandlines.append(c)
    print(c)
    open(os.path.join(outfolder,'cmds/qsub_'+k),'w').write(qsub+c)
f.close()
open(os.path.join(outfolder,'commands'),'w').write('\n'.join(commandlines))
```


```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/13_align_reads_v1 && \
samtools merge merged.tmp.bam 2KM.bam 3KM.bam 6KM.bam 8KM.bam 14KM.bam 15KM.bam 20KM.bam 200bp.bam 250bp.bam 500bp.bam  &&\
samtools sort -@ 32 merged.tmp.bam -o merged.bam &&\
rm merged.tmp.bam &
```

#### samtools depth

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/13_align_reads_v1 && samtools index merged.bam && samtools depth -a merged.bam >merged.depth &
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/13_align_reads_v1 && samtools index merged.bam &&  samtools depth -a merged.bam >merged.depth &
```

### steo 14 annotate_transcript

trinity is finished in previous step

#### StringTie Histat

```
# activate bio to use histat and stringtie
source activate bio
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/hisat2-build -f /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa -p 32 3303histat
```

histat
```

files_info = '/home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/files.tab'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/'
histat = '/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/hisat2'
genome_ref = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/3303histat'
commandline = 'cd {outfolder} && {histat} -p 48 -x {genome_ref} -1 {reads1} -2 {reads2} -S {fileout}.sam && /home/xcao/p/bin/samtools sort -@48 -o {fileout} {fileout}.sam  && rm {fileout}.sam >{fileout}.log'
if not os.path.exists(outfolder+'cmds'):
    os.makedirs(outfolder+'cmds')
f = open(files_info)
commandlines = []
for line in f:
    k,v = line.split()
    fileout = os.path.join(outfolder,k+'.bam')
    reads1, reads2 = v.strip().split(',')
    c = commandline.format(histat=histat, reads1=reads1, reads2=reads2,fileout=fileout, outfolder=outfolder, genome_ref=genome_ref)
    commandlines.append(c)
    print(c)
    open(os.path.join(outfolder,'cmds/qsub_'+k),'w').write(qsub+c)
f.close()
open(os.path.join(outfolder,'commands'),'w').write('\n'.join(commandlines))

```

stringtie

```
files_info = '/home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/files.tab'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/'
stringtie = '/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/stringtie'
genome_ref = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/3303histat'
commandline = 'cd {outfolder} && {stringtie} {filein} -o {fileout} -p 48 &'
f = open(files_info)
commandlines = []
for line in f:
    k,v = line.split()
    fileout = os.path.join(outfolder,k+'.gtf')
    filein = os.path.join(outfolder,k+'.bam')
    c = commandline.format(stringtie=stringtie, filein=filein,fileout=fileout, outfolder=outfolder, genome_ref=genome_ref)
    commandlines.append(c)
    print(c)
f.close()
open(os.path.join(outfolder,'commands_stringtie'),'w').write('\n'.join(commandlines))
```

```
ls /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/*.gtf >/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/gtf_list
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/stringtie --merge -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/merged.gtf /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/gtf_list
```

get sequence
```
gffread -g /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa -w /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/stringtie.transcript.fa /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie_hisat/merged.gtf
```

#### PASA
```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript
cat Trinity.fasta Trinity-GG.fasta >Trinity.all.fa

```

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript
cat Trinity.fasta Trinity-GG.fasta >Trinity.all.fa
perl /home/xcao/p/pasa/PASApipeline-v2.3.3/bin/seqclean Trinity.all.fa -c 16 #max 16 for -c option
perl /home/xcao/p/pasa/PASApipeline-v2.3.3/misc_utilities/accession_extractor.pl < Trinity.all.fa > tdn.accs
docker run --rm -it -v /tmp:/tmp -v /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript:/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript      pasapipeline/pasapipeline:latest      bash -c 'cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript && /usr/local/src/PASApipeline/Launch_PASA_pipeline.pl      -c alignAssembly.config -C -R --ALIGNER blat,gmap -g wenlin3303min200filter.fa -t Trinity.all.fa.clean -T -u Trinity.all.fa --TDN tdn.accs --trans_gtf stringtie.merged.gtf --CPU 32'
```

#### StringTieHistat 3318

```
# activate bio to use histat and stringtie
source activate bio
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/hisat2-build -f /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/3318_genomefilter1.fa -p 32 3318histat
```

histat
```

files_info = '/home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/files.tab'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/'
histat = '/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/hisat2'
genome_ref = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/3318histat'
commandline = 'cd {outfolder} && {histat} -p 48 -x {genome_ref} -1 {reads1} -2 {reads2} -S {fileout}.sam && /home/xcao/p/bin/samtools sort -@48 -o {fileout} {fileout}.sam  && rm {fileout}.sam '
if not os.path.exists(outfolder+'cmds'):
    os.makedirs(outfolder+'cmds')
f = open(files_info)
commandlines = []
for line in f:
    k,v = line.split()
    fileout = os.path.join(outfolder,k+'.bam')
    reads1, reads2 = v.strip().split(',')
    c = commandline.format(histat=histat, reads1=reads1, reads2=reads2,fileout=fileout, outfolder=outfolder, genome_ref=genome_ref)
    commandlines.append(c)
    print(c)
    open(os.path.join(outfolder,'cmds/qsub_'+k),'w').write(qsub+c)
f.close()
open(os.path.join(outfolder,'commands'),'w').write('\n'.join(commandlines))

```

stringtie

```
files_info = '/home/xcao/w/20181205Hermeuptychia/20181212Annotation/RNA/files.tab'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/'
stringtie = '/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/stringtie'
genome_ref = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/3318histat'
commandline = 'cd {outfolder} && {stringtie} {filein} -o {fileout} -p 48 &'
f = open(files_info)
commandlines = []
for line in f:
    k,v = line.split()
    fileout = os.path.join(outfolder,k+'.gtf')
    filein = os.path.join(outfolder,k+'.bam')
    c = commandline.format(stringtie=stringtie, filein=filein,fileout=fileout, outfolder=outfolder, genome_ref=genome_ref)
    commandlines.append(c)
    print(c)
f.close()
open(os.path.join(outfolder,'commands_stringtie'),'w').write('\n'.join(commandlines))
```

```
ls /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/*.gtf >/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/gtf_list
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/stringtie --merge -o /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/merged.gtf /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/gtf_list
```

#### PASA 3318

```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie_hisat/merged.gtf stringtie.merged.gtf
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/3318_genomefilter1.fa ./
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/Trinity* ./
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/alignAssembly.config ./

perl /home/xcao/p/pasa/PASApipeline-v2.3.3/bin/seqclean Trinity.all.fa -c 16
perl /home/xcao/p/pasa/PASApipeline-v2.3.3/misc_utilities/accession_extractor.pl < Trinity.all.fa > tdn.accs
docker run --rm -it -v /tmp:/tmp -v /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript://home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript      pasapipeline/pasapipeline:latest      bash -c 'cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript && /usr/local/src/PASApipeline/Launch_PASA_pipeline.pl      -c alignAssembly.config -C -R --ALIGNER blat,gmap -g 3318_genomefilter1.fa -t Trinity.all.fa.clean -T -u Trinity.all.fa --TDN tdn.accs --trans_gtf stringtie.merged.gtf --CPU 32'
```

### step 15 annotate homologs

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/15_annotate_homolog/
ln -s /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa genome.fa
export ALN_TAB=/home/xcao/p/spaln/spaln2.3.2b
export ALN_DBS=/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/15_annotate_homolog/
/home/xcao/p/spaln/spaln2.3.2b/perl/makeidx.pl genome.fa
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Drosophila/dmel-all-translation-r6.24.fasta >spaln.Drosophila_melanogaster.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Bombyx_mori/GCF_000151625.1_ASM15162v1_protein.faa >spaln.Bombyx_mori.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Anopheles_gambiae/Anopheles-gambiae-PEST_PEPTIDES_AgamP4.10.fa >spaln.Anopheles_gambiae.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Manduca_sexta/MCOT30303.protein.fa >spaln.Manduca_sexta.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Danaus_plexippus/Dp_geneset_OGS2_pep.fasta >spaln.Danaus_plexippus.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Heliconius_melpomene/Heliconius_melpomene_melpomene_Hmel2.5.proteins.fa >spaln.Heliconius_melpomene.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Lerema_accius/Lerema_accius_v1.1_-_proteins.fa >spaln.Lerema_accius.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Papilio_polytes/Papilio_polytes_Ppol_1.0_Refseq_-_proteins.fa >spaln.Papilio_polytes.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Papilio_xuthus/Papilio_xuthus_Pxut_1.0_Refseq_-_proteins.fa >spaln.Papilio_xuthus.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Spodoptera_litura/GCF_002706865.1_ASM270686v1_protein.faa >spaln.Spodoptera_litura.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Megathymus_ursus_violae/mvi_1504_pr.fa >spaln.Megathymus_ursus_violae.gff &


```


```
workfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/15_annotate_homolog/'
query_genome = 'genome.fa'
txt = '''/home/xcao/p/bin/spaln -Q7  -O0 {query_genome} {ref_genome} >{output} &'''

dc_ref = {}
dc_ref['Drosophila_melanogaster'] = '/home/xcao/w/genomes/Drosophila/dmel-all-translation-r6.24.fasta'
dc_ref['Bombyx_mori'] = '/home/xcao/w/genomes/Bombyx_mori/GCF_000151625.1_ASM15162v1_protein.faa'
dc_ref['Anopheles_gambiae'] = '/home/xcao/w/genomes/Anopheles_gambiae/Anopheles-gambiae-PEST_PEPTIDES_AgamP4.10.fa'
dc_ref['Manduca_sexta'] = '/home/xcao/w/genomes/Manduca_sexta/MCOT30303.protein.fa'
dc_ref['Danaus_plexippus'] = '/home/xcao/w/genomes/Danaus_plexippus/Dp_geneset_OGS2_pep.fasta'
dc_ref['Heliconius_melpomene'] = '/home/xcao/w/genomes/Heliconius_melpomene/Heliconius_melpomene_melpomene_Hmel2.5.proteins.fa'
dc_ref['Lerema_accius'] = '/home/xcao/w/genomes/Lerema_accius/Lerema_accius_v1.1_-_proteins.fa'
dc_ref['Papilio_polytes'] = '/home/xcao/w/genomes/Papilio_polytes/Papilio_polytes_Ppol_1.0_Refseq_-_proteins.fa'
dc_ref['Papilio_xuthus'] = '/home/xcao/w/genomes/Papilio_xuthus/Papilio_xuthus_Pxut_1.0_Refseq_-_proteins.fa'
dc_ref['Spodoptera_litura'] = '/home/xcao/w/genomes/Spodoptera_litura/GCF_002706865.1_ASM270686v1_protein.faa'
dc_ref['Megathymus_ursus_violae'] = '/home/xcao/w/genomes/Megathymus_ursus_violae/mvi_1504_pr.fa'

for species, ref_genome in dc_ref.items():
    print(txt.format(workfolder=workfolder, query_genome=query_genome, ref_genome=ref_genome, output = 'spaln.'+species+'.gff'))
```

#### combine and rename geneIDs
```
files = '''spaln.Anopheles_gambiae.gff
spaln.Bombyx_mori.gff
spaln.Danaus_plexippus.gff
spaln.Drosophila_melanogaster.gff
spaln.Heliconius_melpomene.gff
spaln.Lerema_accius.gff
spaln.Manduca_sexta.gff
spaln.Megathymus_ursus_violae.gff
spaln.Papilio_polytes.gff
spaln.Papilio_xuthus.gff
spaln.Spodoptera_litura.gff
'''.split()
folder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/15_annotate_homolog/'
fout = open(folder+'spaln.all.rename.gff','w')
for f in files:
    txt = open(folder + f).read()
    species = f.split('.')[1]
    txt = txt.replace('ID=','ID='+species+'_')
    txt = txt.replace('Parent=','Parent='+species+'_')
    fout.write(txt)
fout.close()
```

#### 3318

```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/15_annotate_homolog/
ln -s /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/3318_genomefilter1.fa genome.fa
export ALN_TAB=/home/xcao/p/spaln/spaln2.3.2b/table/
export ALN_DBS=/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/15_annotate_homolog
/home/xcao/p/spaln/spaln2.3.2b/perl/makeidx.pl genome.fa
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Drosophila/dmel-all-translation-r6.24.fasta >spaln.Drosophila_melanogaster.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Bombyx_mori/GCF_000151625.1_ASM15162v1_protein.faa >spaln.Bombyx_mori.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Anopheles_gambiae/Anopheles-gambiae-PEST_PEPTIDES_AgamP4.10.fa >spaln.Anopheles_gambiae.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Manduca_sexta/MCOT30303.protein.fa >spaln.Manduca_sexta.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Danaus_plexippus/Dp_geneset_OGS2_pep.fasta >spaln.Danaus_plexippus.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Heliconius_melpomene/Heliconius_melpomene_melpomene_Hmel2.5.proteins.fa >spaln.Heliconius_melpomene.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Lerema_accius/Lerema_accius_v1.1_-_proteins.fa >spaln.Lerema_accius.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Papilio_polytes/Papilio_polytes_Ppol_1.0_Refseq_-_proteins.fa >spaln.Papilio_polytes.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Papilio_xuthus/Papilio_xuthus_Pxut_1.0_Refseq_-_proteins.fa >spaln.Papilio_xuthus.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Spodoptera_litura/GCF_002706865.1_ASM270686v1_protein.faa >spaln.Spodoptera_litura.gff &
/home/xcao/p/bin/spaln -Q7  -O0 genome.fa /home/xcao/w/genomes/Megathymus_ursus_violae/mvi_1504_pr.fa >spaln.Megathymus_ursus_violae.gff &

```

#### with qian's script
```
ls -1d /work/biophysics/jshen3/annotation/3639/15_annotate_homolog/exonerate/*_prots | awk '{printf "cp -r %s . &\n", $1}' | bash
wait
ls -1 *_prots/*.fa | awk '{split($1, a, "/"); split(a[1], b, "_"); printf "exonerate --model protein2genome --refine region -q %s -t genome.fa -Q protein -T dna --showtargetgff yes --showalignment yes --percent 30 > %s_aligns/%s.exonerate\n", $1, b[1], a[2]}' > input
wc -l input
split -l 7673 input input_ # Divide input file into 15 parts to allocate to 15 jobs, get the number by divide input lines to 15.
ls -1d *_prots | awk '{split($1, a, "_"); printf "mkdir %s_aligns\n", a[1]}' | bash
ls -1 input_?? | awk '{split($1, a, "_"); printf "%s\n", a[2]}' > list
python2 prepare_exenorate_jobs.py genome
ls -1 genome_??.job | awk '{printf "sbatch %s\n", $1}' | bash
ls -1d *_aligns | awk '{split($1, a, "_"); printf "cat %s/*.exonerate > %s.exonerate &\n", $1, a[1]}' | bash
wait
ls -1d *_aligns | awk '{split($1, a, "_"); printf "python2 process_exonerate_out_to_prediction.py %s.exonerate homolog_%s.gff &\n", a[1], a[1]}' | bash
wait
ls -1 *_prots/*.fa | awk '{split($1, a, "_"); printf "python2 fasta_letter_count.py %s >> %s_lens \n", $1, a[1]}' | bash
wait
ls -1d *_aligns | awk '{split($1, a, "_"); printf "python2 select_complete_aligns.py %s &\n", a[1]}' | bash
wait

```

### step 16_prepare_train_data

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/16_prepare_train_data/
cp /export/home11/congqian/3314/16_prepare_train_data/*.py ./

cp ../14_annotate_transcript/stringtie.merged.gtf ./
cp ../14_annotate_transcript/sample_mydb_pasa.sqlite.pasa_assemblies.gtf ./3303_pasa.gtf
cp ../15_annotate_homolog/*.gff ./
cat spaln.*.gff > spaln.homolog.gff
cat stringtie.merged.gtf 3303_pasa.gtf > genome_based.gff

#python rewrite_pasa_gff.py 3303_pasa.gtf > 3303_pasa.gff #this one not needed here
#run step16_prepare_training_data.py in ipython, and change some parts accordingly
```

final result is   
good_transcript_genome_homology.gff

#### 3318

```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/16_prepare_train_data
cp /export/home11/congqian/3314/16_prepare_train_data/*.py ./
cp ../14_annotate_transcript/stringtie.merged.gtf ./
cp ../14_annotate_transcript/3318_genomefilter1.fa.sqlite.pasa_assemblies.gtf ./3318_pasa.gtf
cp ../15_annotate_homolog/*.gff ./
cat spaln.*.gff > spaln.homolog.gff
cat 3318_pasa.gtf stringtie.merged.gtf >genome_based.gff
#run step16_prepare_training_data.py in ipython, and change some parts accordingly

```

final result is   
good_transcript_genome_homology.gff

#### with Qian's scripts

```
gffread ../14_annotate_transcript/stringtie.merged.gtf -o stringtie.gff3
gffread /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/sample_mydb_pasa.sqlite.pasa_assemblies.gtf -o pasa.gff3
gffread /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/3318_genomefilter1.fa.sqlite.pasa_assemblies.gtf -o pasa.gff3
python2 rewrite_pasa_gff.py pasa.gff3 > pasa.gff
cp ../15_annotate_homolog/*_good.gff ./
sed -i 's/gene/transcript/g' *_good.gff

#Note: memory consumption huge.
python2 select_good_transcripts_step1.py bmo_good.gff pasa.gff > good_transcripts_bmo_pasa &
python2 select_good_transcripts_step1.py bmo_good.gff stringtie.gff3 > good_transcripts_bmo_tophat &
python2 select_good_transcripts_step1.py cne_good.gff pasa.gff > good_transcripts_cne_pasa &
python2 select_good_transcripts_step1.py cne_good.gff stringtie.gff3 > good_transcripts_cne_tophat &
python2 select_good_transcripts_step1.py dme_good.gff pasa.gff > good_transcripts_dme_pasa &
python2 select_good_transcripts_step1.py dme_good.gff stringtie.gff3 > good_transcripts_dme_tophat &
python2 select_good_transcripts_step1.py dpl_good.gff pasa.gff > good_transcripts_dpl_pasa &
python2 select_good_transcripts_step1.py dpl_good.gff stringtie.gff3 > good_transcripts_dpl_tophat &
python2 select_good_transcripts_step1.py hme_good.gff pasa.gff > good_transcripts_hme_pasa &
python2 select_good_transcripts_step1.py hme_good.gff stringtie.gff3 > good_transcripts_hme_tophat &
python2 select_good_transcripts_step1.py lac_good.gff pasa.gff > good_transcripts_lac_pasa &
python2 select_good_transcripts_step1.py lac_good.gff stringtie.gff3 > good_transcripts_lac_tophat &
python2 select_good_transcripts_step1.py pgl_good.gff pasa.gff > good_transcripts_pgl_pasa &
python2 select_good_transcripts_step1.py pgl_good.gff stringtie.gff3 > good_transcripts_pgl_tophat &
python2 select_good_transcripts_step1.py ppo_good.gff pasa.gff > good_transcripts_ppo_pasa &
python2 select_good_transcripts_step1.py ppo_good.gff stringtie.gff3 > good_transcripts_ppo_tophat &
python2 select_good_transcripts_step1.py pxu_good.gff pasa.gff > good_transcripts_pxu_pasa &
python2 select_good_transcripts_step1.py pxu_good.gff stringtie.gff3 > good_transcripts_pxu_tophat &

cat good_transcripts_???_* > good_transcript_step1.gff
sed -i 's/StringTie/Cufflinks/g' good_transcript_step1.gff
python2 select_good_transcripts_step2.py
python2 select_good_transcripts_step3.py ../genome.fa
python2 select_good_transcripts_step4.py ../genome.fa
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/12_mask_repeats/3303_repeats.gff ./repeats.gff
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/12_mask_repeats/3318_repeats.gff ./repeats.gff
python2 select_good_transcripts_step5.py repeats.gff
python2 select_good_transcripts_step6.py 
python2 select_good_transcripts_step7.py 

```

### train denovo

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/17_train_denovo
cp ../16_prepare_train_data/good_transcript_genome_homology.gff ./train_data_ori.gff

# train genemark first, this can be done without a training set
mkdir train_genmark 
cd train_genmark
/home/xcao/p/GeneMark/gm_et_linux_64/gmes_petap/gmes_petap.pl --ES --cores 32  --sequence ../../8_improve_assembly/wenlin3303min200filter.fa >& train.log & disown -h
# it produces: /export/home10/congqian/3303/17_train_denovo/train_genemark/mod/es.mod, which is a soft link point to /export/home10/congqian/3303/17_train_denovo/train_genemark/run/S7/org_S7_bp.0mtx
cp /export/home10/congqian/3303/17_train_denovo/train_genemark/run/S7/org_S7_bp.0mtx /export/home10/congqian/3303/17_train_denovo/train_genemark/3318.mod

# train augustus after getting the trainning data
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/17_train_denovo
mkdir train_augustus
cd train_augustus/
export AUGUSTUS_CONFIG_PATH=/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/config
export PATH=/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin:$PATH
/home/xcao/p/gffread/gffread-0.9.12.Linux_x86_64/gffread ../train_data.gff  -g ../../8_improve_assembly/wenlin3303min200filter.fa -w train_data.fa
python /home/xcao/p/xiaolongTools/utils/annotation/changeGFF2oldNaming.py /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/17_train_denovo/train_data_ori.gff >/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/17_train_denovo/train_data.gff
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/gff2gbSmallDNA.pl ../train_data.gff ../../8_improve_assembly/wenlin3303min200filter.fa 1000 train_data.gb
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/randomSplit.pl train_data.gb 100
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/new_species.pl --species=3303
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/etraining --species=3303 train_data.gb.train
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/augustus --species=3303 train_data.gb.test | tee firsttest.out
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/optimize_augustus.pl --cpus=32 --species=3303 train_data.gb >& optimize.log & disown -h
# last step will take long. let in run in the background and train other programs
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/etraining --species=3303 train_data.gb

# train snap while the optimizing step for augustus is still running
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/17_train_denovo
mkdir train_snap
cd train_snap/
sort -k 1,1 -s ../train_data.gff  > train_data.gff
sed -i 's/cds/exon/g' train_data.gff
cp /alea/home5/qc_4/protein_annotation_accius/train_snap/gff2zff.pl ./
perl gff2zff.pl < train_data.gff > 3303.ann
grep '>' 3303.ann | awk '{split($1,a,">");printf "%s\n",a[2]}' > scaflist
cp /alea/home5/qc_4/protein_annotation_accius/train_snap/extract_seqs_in_order.py ./
python extract_seqs_in_order.py scaflist ../../8_improve_assembly/wenlin3303min200filter.fa 3303.dna
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/fathom 3303.ann 3303.dna -gene-stats > gene-stats.log 2>&1
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/fathom 3303.ann 3303.dna -validate > validate.log 2>&1
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/fathom 3303.ann 3303.dna -categorize 1000 > categorize.log 2>&1
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/fathom uni.ann uni.dna -export 1000 -plus > uni-plus.log 2>&1
mkdir params
cd params/
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/forge ../export.ann ../export.dna > ../forge.log 2>&1
cd ../
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/hmm-assembler.pl 3303 params/ > 3303.hmm

# train glimmer while the optimizing step for augustus is still running
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/17_train_denovo
mkdir train_glimmer
cd train_glimmer
cp /alea/home5/qc_4/protein_annotation_accius/train_glimmer/zff2glim.py ./
cp ../train_snap/3303.dna ./
python2 zff2glim.py ../train_snap/3303.ann > 3303.ann
/home/xcao/p/GlimmerHMM/GlimmerHMM/train/trainGlimmerHMM 3303.dna 3303.ann -b 2 -d 3303 >& train.log
```

#### 3318
```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/17_train_denovo
cp ../16_prepare_train_data/good_transcript_genome_homology.gff ./train_data_ori.gff
python /home/xcao/p/xiaolongTools/utils/annotation/changeGFF2oldNaming.py train_data_ori.gff >train_data.gff

# train genemark first, this can be done without a training set
mkdir train_genmark 
cd train_genmark
/home/xcao/p/GeneMark/gm_et_linux_64/gmes_petap/gmes_petap.pl --ES --cores 32  --sequence ../../8_improve_assembly/3318_genomefilter1.fa >& train.log & disown -h
# it produces: /export/home10/congqian/3303/17_train_denovo/train_genemark/mod/es.mod, which is a soft link point to /export/home10/congqian/3303/17_train_denovo/train_genemark/run/S7/org_S7_bp.0mtx
cp /export/home10/congqian/3303/17_train_denovo/train_genemark/run/S7/org_S7_bp.0mtx /export/home10/congqian/3303/17_train_denovo/train_genemark/3318.mod


# train augustus after getting the trainning data
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/17_train_denovo
mkdir train_augustus
cd train_augustus/
export AUGUSTUS_CONFIG_PATH=/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/config
export PATH=/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin:$PATH
/home/xcao/p/gffread/gffread-0.9.12.Linux_x86_64/gffread ../train_data.gff  -g ../../8_improve_assembly/3318_genomefilter1.fa -w train_data.fa
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/gff2gbSmallDNA.pl ../train_data.gff ../../8_improve_assembly/3318_genomefilter1.fa 1000 train_data.gb
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/randomSplit.pl train_data.gb 100
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/new_species.pl --species=3318
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/etraining --species=3318 train_data.gb.train
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/augustus --species=3318 train_data.gb.test | tee firsttest.out
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/optimize_augustus.pl --cpus=32 --species=3318 train_data.gb >& optimize.log & disown -h
# last step will take long. let in run in the background and train other programs
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/etraining --species=3318 train_data.gb

# train snap while the optimizing step for augustus is still running
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/17_train_denovo
mkdir train_snap
cd train_snap/
sort -k 1,1 -s ../train_data.gff  > train_data.gff
sed -i 's/cds/exon/g' train_data.gff
cp /alea/home5/qc_4/protein_annotation_accius/train_snap/gff2zff.pl ./
perl gff2zff.pl < train_data.gff > 3318.ann
grep '>' 3318.ann | awk '{split($1,a,">");printf "%s\n",a[2]}' > scaflist
cp /alea/home5/qc_4/protein_annotation_accius/train_snap/extract_seqs_in_order.py ./
python extract_seqs_in_order.py scaflist ../../8_improve_assembly/3318_genomefilter1.fa 3318.dna
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/fathom 3318.ann 3318.dna -gene-stats > gene-stats.log 2>&1
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/fathom 3318.ann 3318.dna -validate > validate.log 2>&1
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/fathom 3318.ann 3318.dna -categorize 1000 > categorize.log 2>&1
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/fathom uni.ann uni.dna -export 1000 -plus > uni-plus.log 2>&1
mkdir params
cd params/
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/forge ../export.ann ../export.dna > ../forge.log 2>&1
cd ../
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/hmm-assembler.pl 3318 params/ > 3318.hmm

# train glimmer while the optimizing step for augustus is still running
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/17_train_denovo
mkdir train_glimmer
cd train_glimmer
cp /alea/home5/qc_4/protein_annotation_accius/train_glimmer/zff2glim.py ./
cp ../train_snap/3318.dna ./
python2 zff2glim.py ../train_snap/3318.ann > 3318.ann
/home/xcao/p/GlimmerHMM/GlimmerHMM/train/trainGlimmerHMM 3318.dna 3318.ann -b 2 -d 3318 >& train.log
```

#### rerun 20190123
```
cd train_augustus/
export AUGUSTUS_CONFIG_PATH=/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/config
export PATH=/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin:$PATH
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/gff2gbSmallDNA.pl ../train_data.gff ../../genome.fa 1000 train_data.gb
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/randomSplit.pl train_data.gb 100


perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/new_species.pl --species=3303
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/etraining --species=3303 train_data.gb.train
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/augustus --species=3303 train_data.gb.test | tee firsttest.out
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/optimize_augustus.pl --cpus=32 --species=3303 train_data.gb >& optimize.log & disown -h
# last step will take long. let in run in the background and train other programs
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/etraining --species=3303 train_data.gb


perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/new_species.pl --species=3318
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/etraining --species=3318 train_data.gb.train
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/augustus --species=3318 train_data.gb.test | tee firsttest.out
perl /home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/scripts/optimize_augustus.pl --cpus=32 --species=3318 train_data.gb >& optimize.log & disown -h
# last step will take long. let in run in the background and train other programs
/home/xcao/p/quast/quast-5.0.2/quast_libs/augustus3.2.3/bin/etraining --species=3318 train_data.gb
```

### step 18_annotate_denovo

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/
mkdir maker
cd maker
source activate bio
maker -CTL
#edit generated file in the folder
maker
```

#### split the genome to 100 parts and run to increase speed 3303


```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/
cp ../8_improve_assembly/wenlin3303min200filter.fa ./genome.fa
source activate bio
gffcompare  ../14_annotate_transcript/stringtie.merged.gtf ../14_annotate_transcript/sample_mydb_pasa.sqlite.pasa_assemblies.gtf -o genomeBased -s genome.fa
gffread genomeBased.combined.gtf -o genomeBased.combined.gff3

gffcompare ../15_annotate_homolog/*.gff -o homologyBased -p Spaln -X
gffread homologyBased.combined.gtf -o homologyBased.combined.gff3

grep '>' genome.fa | awk '{split($1,a,">");printf "%s\n",a[2]}' > scaflist
python2 prepare_repeat_for_maker.py  ../12_mask_repeats/3303_repeats.gff repeats.gff3
```

```
import os
import glob
from Bio import SeqIO
import numpy as np

workfolder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo'
folder_makers = os.path.join(workfolder, 'maker_split')
folder_cmds = os.path.join(workfolder, 'cmds')
folder_ref = os.path.join(workfolder,'maker')
file_maker_bopts = os.path.join(folder_ref,'maker_bopts.ctl')
file_maker_exe = os.path.join(folder_ref,'maker_exe.ctl')#add missing programs to this file
file_maker_opts = os.path.join(folder_ref,'maker_opts.ctl')#edit this file so that it works for all. 
#Basically, since all individual runs have similar input, set 
## genome=genome_each.fa
## est_gff=est_each.gff
## protein_gff=protein_each.gff
## rm_gff=repeats_each.gff
## est2genome=1
## protein2genome=1
## trna=1 # or 0 depending on whether to find tRNAs
# other trained data file need to be changed, too
file_genome = os.path.join(workfolder,'genome.fa')
gff3_genomeBased = os.path.join(workfolder, 'genomeBased.combined.gff3')
gff3_homologyBased = os.path.join(workfolder, 'homologyBased.combined.gff3')
gff3_repeats = os.path.join(workfolder,'repeats.gff3')

qsub_common = '''#!/bin/bash
#$ -S /bin/sh
#$ -cwd
#$ -j y
source /home/xcao/.bashrc
source /home/xcao/.bash_profile
source /home/xcao/p/anaconda3_5.2.0/bin/activate bio
cd {workfolder}
maker --ignore_nfs_tmp
'''



def makefolder(*paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

def writeFasta(seqs,outfile):
    '''
    seqs is a list of SeqIO sequence, write sequence to outfile
    '''
    with open(outfile,'w') as f:
        for s in seqs:
            f.write('>'+s.id+'\n'+str(s.seq)+'\n')
    return None

def getGffEach(gffall, scaffolds, outfile):
    '''
    gffall is a filename or a list by reading the gffall file with readlines()
    scaffolds is scaffolds to keep in gffall
    outfile is where to save the output file
    filter the gffall with scaffolds, and save output to outfile
    '''
    if isinstance(gffall, str):
        print('input gffall is a filename. Read it to list')
        gffall = open(gffall).readlines()
    elif isinstance(gffall,list):
        print('input gffall is a list')
    else:
        print('unknown format for input')
        return None
    
    if isinstance(scaffolds, list):
        scaffolds = set(scaffolds)
    if not isinstance(scaffolds,set):
        print('scaffolds should be a list or set')
        return None
    
    with open(outfile,'w') as f:
        for line in gffall:
            elements = line.split('\t')
            if len(elements) >8:
                if elements[0] in scaffolds:
                    f.write(line)
    return None

N = 1000

genome_seqs = list(SeqIO.parse(file_genome,'fasta'))
ls_gff3_genomeBased = open(gff3_genomeBased).readlines()
ls_gff3_homologyBased = open(gff3_homologyBased).readlines()
ls_gff3_repeats = open(gff3_repeats).readlines()

#split genome_seqs to N parts
ls_genomeIDs = np.array_split(range(len(genome_seqs)),N)
ls_genomeSeqs = [[genome_seqs[i] for i in e] for e in ls_genomeIDs]

#split genome sequences to each folder
# for each folder, create genome_each.fa est_each.gff protein_each.gff repeats_each.gff

def processOne(i):
    outfolder = os.path.join(folder_makers, str(i))
    makefolder(outfolder)
    file_each_genome = os.path.join(outfolder,'genome_each.fa')
    file_each_est = os.path.join(outfolder,'est_each.gff')
    file_each_protein = os.path.join(outfolder,'protein_each.gff')
    file_each_repeats = os.path.join(outfolder,'repeats_each.gff')
    writeFasta(ls_genomeSeqs[i], file_each_genome)
    scaffolds = [e.id for e in ls_genomeSeqs[i]]
    getGffEach(gffall=ls_gff3_genomeBased, scaffolds=scaffolds, outfile=file_each_est)
    getGffEach(gffall=ls_gff3_homologyBased, scaffolds=scaffolds, outfile=file_each_protein)
    getGffEach(gffall=ls_gff3_repeats, scaffolds=scaffolds, outfile=file_each_repeats)
    
    os.system('cp ' + file_maker_bopts + ' ' + outfolder + '/')
    os.system('cp ' + file_maker_exe + ' ' + outfolder + '/')
    os.system('cp ' + file_maker_opts + ' ' + outfolder + '/')
    
    open(os.path.join(folder_cmds,'qsub'+str(i)),'w').write(qsub_common.format(workfolder=outfolder))

#for i in range(N):
#    processOne(i)
#    break

from multiprocessing import Pool
pool = Pool(32)
pool.map(processOne,list(range(N)))
pool.close()





```

```
workfolder=/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/cmds
tosub=`ls $workfolder`
for onefile in $tosub; do qsub "$workfolder/$onefile"; done
```

maker_opts.ctl looks like
```
#-----Genome (these are always required)
genome=genome_each.fa #genome sequence (fasta file or fasta embeded in GFF3 file)
organism_type=eukaryotic #eukaryotic or prokaryotic. Default is eukaryotic

#-----Re-annotation Using MAKER Derived GFF3
maker_gff= #MAKER derived GFF3 file
est_pass=0 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=0 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no
protein_pass=0 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=0 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

#-----EST Evidence (for best results provide a file for at least one)
est= #set of ESTs or assembled mRNA-seq in fasta format
altest= #EST/cDNA sequence file in fasta format from an alternate organism
est_gff=est_each.gff #aligned ESTs or mRNA-seq from an external GFF3 file
altest_gff= #aligned ESTs from a closly relate species in GFF3 format

#-----Protein Homology Evidence (for best results provide a file for at least one)
protein=  #protein sequence file in fasta format (i.e. from mutiple oransisms)
protein_gff=protein_each.gff  #aligned protein homology evidence from an external GFF3 file

#-----Repeat Masking (leave values blank to skip repeat masking)
model_org=all #select a model organism for RepBase masking in RepeatMasker
rmlib= #provide an organism specific repeat library in fasta format for RepeatMasker
repeat_protein=/home/xcao/p/maker/maker-3.01.02/data/te_proteins.fasta #provide a fasta file of transposable element proteins for RepeatRunner
rm_gff=repeats_each.gff #pre-identified repeat elements from an external GFF3 file
prok_rm=0 #forces MAKER to repeatmask prokaryotes (no reason to change this), 1 = yes, 0 = no
softmask=1 #use soft-masking rather than hard-masking in BLAST (i.e. seg and dust filtering)

#-----Gene Prediction
snaphmm=/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/17_train_denovo/train_snap/3303.hmm #SNAP HMM file
gmhmm=/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/17_train_denovo/train_genmark/output/gmhmm.mod #GeneMark HMM file
augustus_species= #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no
trna=0 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no

#-----Other Annotation Feature Types (features MAKER doesn't recognize)
other_gff= #extra features to pass-through to final MAKER generated GFF3 file

#-----External Application Behavior Options
alt_peptide=C #amino acid used to replace non-standard amino acids in BLAST databases
cpus=32 #max number of cpus to use in BLAST and RepeatMasker (not for MPI, leave 1 when using MPI)

#-----MAKER Behavior Options
max_dna_len=2000000 #length for dividing up contigs into chunks (increases/decreases memory usage)
min_contig=100 #skip genome contigs below this length (under 10kb are often useless)

pred_flank=200 #flank for extending evidence clusters sent to gene predictors
pred_stats=1 #report AED and QI statistics for all predictions as well as models
AED_threshold=1 #Maximum Annotation Edit Distance allowed (bound by 0 and 1)
min_protein=30 #require at least this many amino acids in predicted proteins
alt_splice=0 #Take extra steps to try and find alternative splicing, 1 = yes, 0 = no
always_complete=0 #extra steps to force start and stop codons, 1 = yes, 0 = no
map_forward=0 #map names and attributes forward from old GFF3 genes, 1 = yes, 0 = no
keep_preds=0 #Concordance threshold to add unsupported gene prediction (bound by 0 and 1)

split_hit=10000 #length for the splitting of hits (expected max intron size for evidence alignments)
single_exon=0 #consider single exon EST evidence when generating annotations, 1 = yes, 0 = no
single_length=250 #min length required for single exon ESTs if 'single_exon is enabled'
correct_est_fusion=0 #limits use of ESTs in annotation to avoid fusion genes

tries=2 #number of times to try a contig if there is a failure for some reason
clean_try=0 #remove all data from previous run before retrying, 1 = yes, 0 = no
clean_up=0 #removes theVoid directory with individual analysis files, 1 = yes, 0 = no
TMP=/tmp #specify a directory other than the system default temporary directory for temporary files

```

#### collect data

```
import glob
folder = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/maker_split'
files = glob.glob(folder+'/*/genome_each.maker.output/genome_each_datastore/**/scaffold*.gff', recursive=True)
fout = open(folder+'/../maker_all.gff3','w')
for f in files:
    fout.write(open(f).read())
fout.close()

import os
os.system('cd '+folder+'/../ && '+ ''' grep '^scaf' maker_all.gff3 | grep -P '\tgenemark\t' > maker_genemark.gff
grep '^scaf' maker_all.gff3 | grep -P '\taugustus_masked\t' > maker_augustus.gff
grep '^scaf' maker_all.gff3 | grep -P '\tsnap_masked\t' > maker_snap.gff
grep '^scaf' maker_all.gff3 | grep -P '\tmaker\t' > maker_maker.gff
grep '^scaf' maker_all.gff3 | grep -P '\trepeatrunner\t' > maker_repeatrunner.gff ''')

import glob
folder = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/maker_split/'
files = glob.glob(folder+'/*/genome_each.maker.output/genome_each_datastore/**/scaffold*.gff', recursive=True)
fout = open(folder+'/../maker_all.gff3','w')
for f in files:
    fout.write(open(f).read())
fout.close()

```
It seems snap and augustus are not working here. ingore them.

#### 3318

```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/
cp ../8_improve_assembly/3318_genomefilter1.fa ./genome.fa
source activate bio
gffcompare  ../14_annotate_transcript/stringtie.merged.gtf ../14_annotate_transcript/3318_genomefilter1.fa.sqlite.pasa_assemblies.gtf -o genomeBased -s genome.fa
gffread genomeBased.combined.gtf -o genomeBased.combined.gff3

gffcompare ../15_annotate_homolog/*.gff -o homologyBased -p Spaln -X
gffread homologyBased.combined.gtf -o homologyBased.combined.gff3

grep '>' genome.fa | awk '{split($1,a,">");printf "%s\n",a[2]}' > scaflist
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/prepare_repeat_for_maker.py ./
python2 prepare_repeat_for_maker.py  ../12_mask_repeats/3318_repeats.gff repeats.gff3
```

```
import os
import glob
from Bio import SeqIO
import numpy as np

workfolder = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/'
folder_makers = os.path.join(workfolder, 'maker_split')
folder_cmds = os.path.join(workfolder, 'cmds')
folder_ref = os.path.join(workfolder,'maker')
file_maker_bopts = os.path.join(folder_ref,'maker_bopts.ctl')
file_maker_exe = os.path.join(folder_ref,'maker_exe.ctl')#add missing programs to this file
file_maker_opts = os.path.join(folder_ref,'maker_opts.ctl')#edit this file so that it works for all. 
#Basically, since all individual runs have similar input, set 
## genome=genome_each.fa
## est_gff=est_each.gff
## protein_gff=protein_each.gff
## rm_gff=repeats_each.gff
## est2genome=1
## protein2genome=1
## trna=1 # or 0 depending on whether to find tRNAs
# other trained data file need to be changed, too
file_genome = os.path.join(workfolder,'genome.fa')
gff3_genomeBased = os.path.join(workfolder, 'genomeBased.combined.gff3')
gff3_homologyBased = os.path.join(workfolder, 'homologyBased.combined.gff3')
gff3_repeats = os.path.join(workfolder,'repeats.gff3')

qsub_common = '''#!/bin/bash
#$ -S /bin/sh
#$ -cwd
#$ -j y
source /home/xcao/.bashrc
source /home/xcao/.bash_profile
source /home/xcao/p/anaconda3_5.2.0/bin/activate bio
cd {workfolder}
maker --ignore_nfs_tmp
'''



def makefolder(*paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

def writeFasta(seqs,outfile):
    '''
    seqs is a list of SeqIO sequence, write sequence to outfile
    '''
    with open(outfile,'w') as f:
        for s in seqs:
            f.write('>'+s.id+'\n'+str(s.seq)+'\n')
    return None

def getGffEach(gffall, scaffolds, outfile):
    '''
    gffall is a filename or a list by reading the gffall file with readlines()
    scaffolds is scaffolds to keep in gffall
    outfile is where to save the output file
    filter the gffall with scaffolds, and save output to outfile
    '''
    if isinstance(gffall, str):
        print('input gffall is a filename. Read it to list')
        gffall = open(gffall).readlines()
    elif isinstance(gffall,list):
        print('input gffall is a list')
    else:
        print('unknown format for input')
        return None
    
    if isinstance(scaffolds, list):
        scaffolds = set(scaffolds)
    if not isinstance(scaffolds,set):
        print('scaffolds should be a list or set')
        return None
    
    with open(outfile,'w') as f:
        for line in gffall:
            elements = line.split('\t')
            if len(elements) >8:
                if elements[0] in scaffolds:
                    f.write(line)
    return None

N = 1000
makefolder(folder_cmds)
makefolder(folder_makers)
genome_seqs = list(SeqIO.parse(file_genome,'fasta'))
ls_gff3_genomeBased = open(gff3_genomeBased).readlines()
ls_gff3_homologyBased = open(gff3_homologyBased).readlines()
ls_gff3_repeats = open(gff3_repeats).readlines()

#split genome_seqs to N parts
ls_genomeIDs = np.array_split(range(len(genome_seqs)),N)
ls_genomeSeqs = [[genome_seqs[i] for i in e] for e in ls_genomeIDs]

#split genome sequences to each folder
# for each folder, create genome_each.fa est_each.gff protein_each.gff repeats_each.gff

def processOne(i):
    outfolder = os.path.join(folder_makers, str(i))
    makefolder(outfolder)
    file_each_genome = os.path.join(outfolder,'genome_each.fa')
    file_each_est = os.path.join(outfolder,'est_each.gff')
    file_each_protein = os.path.join(outfolder,'protein_each.gff')
    file_each_repeats = os.path.join(outfolder,'repeats_each.gff')
    writeFasta(ls_genomeSeqs[i], file_each_genome)
    scaffolds = [e.id for e in ls_genomeSeqs[i]]
    getGffEach(gffall=ls_gff3_genomeBased, scaffolds=scaffolds, outfile=file_each_est)
    getGffEach(gffall=ls_gff3_homologyBased, scaffolds=scaffolds, outfile=file_each_protein)
    getGffEach(gffall=ls_gff3_repeats, scaffolds=scaffolds, outfile=file_each_repeats)
    
    os.system('cp ' + file_maker_bopts + ' ' + outfolder + '/')
    os.system('cp ' + file_maker_exe + ' ' + outfolder + '/')
    os.system('cp ' + file_maker_opts + ' ' + outfolder + '/')
    
    open(os.path.join(folder_cmds,'qsub'+str(i)),'w').write(qsub_common.format(workfolder=outfolder))

#for i in range(N):
#    processOne(i)
#    break

from multiprocessing import Pool
pool = Pool(32)
pool.map(processOne,list(range(N)))
pool.close()





```

```
workfolder=/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/cmds
tosub=`ls $workfolder`
for onefile in $tosub; do qsub "$workfolder/$onefile"; done
```

#### glimmer
```
python /home/xcao/p/xiaolongTools/utils/annotation/runGlimmer.py -g /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/genome.fa -d /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/17_train_denovo/train_glimmer/TrainGlimm/ -G /home/xcao/p/GlimmerHMM/GlimmerHMM/bin/glimmerhmm_linux_x86_64 -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/glimmer.gtf -t 32

cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/
source activate bio
gffread glimmer.gtf -o glimmer.gff3
```

3318
```
python /home/xcao/p/xiaolongTools/utils/annotation/runGlimmer.py -g /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/genome.fa -d /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/17_train_denovo/train_glimmer/3318/ -G /home/xcao/p/GlimmerHMM/GlimmerHMM/bin/glimmerhmm_linux_x86_64 -o /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/glimmer.gtf -t 32

cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/
source activate bio
gffread glimmer.gtf -o glimmer.gff3
```

#### maker does not work well on alea. Transfer files to bioHPC

```
#transfer files
scp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/* s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/2018Hermeuptychia/20190106maker/3303/
scp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318//18_annotate_denovo/* s185491@nucleus.biohpc.swmed.edu:/work/biophysics/s185491/2018Hermeuptychia/20190106maker/3318/


```

```
import os
import glob
from Bio import SeqIO
import numpy as np

workfolder = '/work/biophysics/s185491/2018Hermeuptychia/20190106maker/3303/'
folder_makers = os.path.join(workfolder, 'maker_split')
folder_cmds = os.path.join(workfolder, 'cmds')
folder_ref = os.path.join(workfolder,'maker')
file_maker_bopts = os.path.join(folder_ref,'maker_bopts.ctl')
file_maker_exe = os.path.join(folder_ref,'maker_exe.ctl')#add missing programs to this file
file_maker_opts = os.path.join(folder_ref,'maker_opts.ctl')#edit this file so that it works for all. 
#Basically, since all individual runs have similar input, set 
## genome=genome_each.fa
## est_gff=est_each.gff
## protein_gff=protein_each.gff
## rm_gff=repeats_each.gff
## est2genome=1
## protein2genome=1
## trna=1 # or 0 depending on whether to find tRNAs
# other trained data file need to be changed, too
file_genome = os.path.join(workfolder,'genome.fa')
gff3_genomeBased = os.path.join(workfolder, 'genomeBased.combined.gff3')
gff3_homologyBased = os.path.join(workfolder, 'homologyBased.combined.gff3')
gff3_repeats = os.path.join(workfolder,'repeats.gff3')

qsub_common = '''#!/bin/bash
#SBATCH --job-name=maker
#SBATCH --partition=32GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --time=10-00:00:00
#SBATCH --output=0.%j.out
#####SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL


cd {workfolder}

/home2/jshen3/apps/maker/bin/maker --ignore_nfs_tmp
'''



def makefolder(*paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

def writeFasta(seqs,outfile):
    '''
    seqs is a list of SeqIO sequence, write sequence to outfile
    '''
    with open(outfile,'w') as f:
        for s in seqs:
            f.write('>'+s.id+'\n'+str(s.seq)+'\n')
    return None

def getGffEach(gffall, scaffolds, outfile):
    '''
    gffall is a filename or a list by reading the gffall file with readlines()
    scaffolds is scaffolds to keep in gffall
    outfile is where to save the output file
    filter the gffall with scaffolds, and save output to outfile
    '''
    if isinstance(gffall, str):
        print('input gffall is a filename. Read it to list')
        gffall = open(gffall).readlines()
    elif isinstance(gffall,list):
        print('input gffall is a list')
    else:
        print('unknown format for input')
        return None

    if isinstance(scaffolds, list):
        scaffolds = set(scaffolds)
    if not isinstance(scaffolds,set):
        print('scaffolds should be a list or set')
        return None

    with open(outfile,'w') as f:
        for line in gffall:
            elements = line.split('\t')
            if len(elements) >8:
                if elements[0] in scaffolds:
                    f.write(line)
    return None

N = 1000
makefolder(folder_cmds)
makefolder(folder_makers)
genome_seqs = list(SeqIO.parse(file_genome,'fasta'))
ls_gff3_genomeBased = open(gff3_genomeBased).readlines()
ls_gff3_homologyBased = open(gff3_homologyBased).readlines()
ls_gff3_repeats = open(gff3_repeats).readlines()

#split genome_seqs to N parts
ls_genomeIDs = np.array_split(range(len(genome_seqs)),N)
ls_genomeSeqs = [[genome_seqs[i] for i in e] for e in ls_genomeIDs]

#split genome sequences to each folder
# for each folder, create genome_each.fa est_each.gff protein_each.gff repeats_each.gff

def processOne(i):
    outfolder = os.path.join(folder_makers, str(i))
    makefolder(outfolder)
    file_each_genome = os.path.join(outfolder,'genome_each.fa')
    file_each_est = os.path.join(outfolder,'est_each.gff')
    file_each_protein = os.path.join(outfolder,'protein_each.gff')
    file_each_repeats = os.path.join(outfolder,'repeats_each.gff')
    writeFasta(ls_genomeSeqs[i], file_each_genome)
    scaffolds = [e.id for e in ls_genomeSeqs[i]]
    getGffEach(gffall=ls_gff3_genomeBased, scaffolds=scaffolds, outfile=file_each_est)
    getGffEach(gffall=ls_gff3_homologyBased, scaffolds=scaffolds, outfile=file_each_protein)
    getGffEach(gffall=ls_gff3_repeats, scaffolds=scaffolds, outfile=file_each_repeats)

    os.system('cp ' + file_maker_bopts + ' ' + outfolder + '/')
    os.system('cp ' + file_maker_exe + ' ' + outfolder + '/')
    os.system('cp ' + file_maker_opts + ' ' + outfolder + '/')

    open(os.path.join(folder_cmds,'qsub'+str(i)),'w').write(qsub_common.format(workfolder=outfolder))

#for i in range(N):
#    processOne(i)
#    break

from multiprocessing import Pool
pool = Pool(32)
pool.map(processOne,list(range(N)))
pool.close()



#for BioHPC
folders = '/work/biophysics/s185491/2018Hermeuptychia/20190106maker/*/maker_split/*'
foldercmd = '/work/biophysics/s185491/2018Hermeuptychia/20190106maker/cmds/'
import glob
import numpy as np
workfolders = glob.glob(folders)
commandlines = ['''cd {workfolder} && /home2/jshen3/apps/maker/bin/maker --ignore_nfs_tmp\n'''.format(workfolder=e) for e in workfolders]
N=20
qsub_common = '''#!/bin/bash
#SBATCH --job-name=maker
#SBATCH --partition=32GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --time=10-00:00:00
#SBATCH --output=0.%j.out
#####SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL



python /home2/s185491/p/xiaolongTools/multiThread.py 32 {file_commands}
'''

ls_commands = np.array_split(commandlines,N)
for i,commands in zip(range(N),ls_commands):
    file_commands = foldercmd+str(i)
    file_qsub = foldercmd+'qsub_'+str(i)
    open(file_commands,'w').write(''.join(list(commands)))
    open(file_qsub,'w').write(qsub_common.format(file_commands=file_commands))





```

collect data
```
import glob
folder = '/work/biophysics/s185491/2018Hermeuptychia/20190106maker/3303/maker_split/'
files = glob.glob(folder+'/*/genome_each.maker.output/genome_each_datastore/*/*/*/scaffold*.gff')
fout = open(folder+'/../maker_all.gff3','w')
for f in files:
    fout.write(open(f).read())
fout.close()

import glob
folder = '/work/biophysics/s185491/2018Hermeuptychia/20190106maker/3318/maker_split/'
files = glob.glob(folder+'/*/genome_each.maker.output/genome_each_datastore/*/*/*/scaffold*.gff')
fout = open(folder+'/../maker_all.gff3','w')
for f in files:
    fout.write(open(f).read())
fout.close()
```

Transfer the data to alea. extract different part the same as previous job. 

#### bioHPC maker, change a new augustus so that it works
in previous step, augustus does not work

```
import os
import glob
from Bio import SeqIO
import numpy as np

workfolder = '/work/biophysics/s185491/2018Hermeuptychia/20190106maker/3303/'
folder_makers = os.path.join(workfolder, 'maker_split')
folder_cmds = os.path.join(workfolder, 'cmds')
folder_ref = os.path.join(workfolder,'maker')
file_maker_bopts = os.path.join(folder_ref,'maker_bopts.ctl')
file_maker_exe = os.path.join(folder_ref,'maker_exe.ctl')#add missing programs to this file
file_maker_opts = os.path.join(folder_ref,'maker_opts.ctl')#edit this file so that it works for all. 
#Basically, since all individual runs have similar input, set 
## genome=genome_each.fa
## est_gff=est_each.gff
## protein_gff=protein_each.gff
## rm_gff=repeats_each.gff
## est2genome=1
## protein2genome=1
## trna=1 # or 0 depending on whether to find tRNAs
# other trained data file need to be changed, too
file_genome = os.path.join(workfolder,'genome.fa')
gff3_genomeBased = os.path.join(workfolder, 'genomeBased.combined.gff3')
gff3_homologyBased = os.path.join(workfolder, 'homologyBased.combined.gff3')
gff3_repeats = os.path.join(workfolder,'repeats.gff3')

qsub_common = '''#!/bin/bash
#SBATCH --job-name=maker
#SBATCH --partition=32GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --time=10-00:00:00
#SBATCH --output=0.%j.out
#####SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL


cd {workfolder}
source /home2/s185491/p/anaconda3/anaconda520/bin/activate bio
/home2/jshen3/apps/maker/bin/maker --ignore_nfs_tmp
'''



def makefolder(*paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

def writeFasta(seqs,outfile):
    '''
    seqs is a list of SeqIO sequence, write sequence to outfile
    '''
    with open(outfile,'w') as f:
        for s in seqs:
            f.write('>'+s.id+'\n'+str(s.seq)+'\n')
    return None

def getGffEach(gffall, scaffolds, outfile):
    '''
    gffall is a filename or a list by reading the gffall file with readlines()
    scaffolds is scaffolds to keep in gffall
    outfile is where to save the output file
    filter the gffall with scaffolds, and save output to outfile
    '''
    if isinstance(gffall, str):
        print('input gffall is a filename. Read it to list')
        gffall = open(gffall).readlines()
    elif isinstance(gffall,list):
        print('input gffall is a list')
    else:
        print('unknown format for input')
        return None

    if isinstance(scaffolds, list):
        scaffolds = set(scaffolds)
    if not isinstance(scaffolds,set):
        print('scaffolds should be a list or set')
        return None

    with open(outfile,'w') as f:
        for line in gffall:
            elements = line.split('\t')
            if len(elements) >8:
                if elements[0] in scaffolds:
                    f.write(line)
    return None

N = 1000
makefolder(folder_cmds)
makefolder(folder_makers)
genome_seqs = list(SeqIO.parse(file_genome,'fasta'))
ls_gff3_genomeBased = open(gff3_genomeBased).readlines()
ls_gff3_homologyBased = open(gff3_homologyBased).readlines()
ls_gff3_repeats = open(gff3_repeats).readlines()

#split genome_seqs to N parts
ls_genomeIDs = np.array_split(range(len(genome_seqs)),N)
ls_genomeSeqs = [[genome_seqs[i] for i in e] for e in ls_genomeIDs]

#split genome sequences to each folder
# for each folder, create genome_each.fa est_each.gff protein_each.gff repeats_each.gff

def processOne(i):
    outfolder = os.path.join(folder_makers, str(i))
    makefolder(outfolder)
    file_each_genome = os.path.join(outfolder,'genome_each.fa')
    file_each_est = os.path.join(outfolder,'est_each.gff')
    file_each_protein = os.path.join(outfolder,'protein_each.gff')
    file_each_repeats = os.path.join(outfolder,'repeats_each.gff')
    writeFasta(ls_genomeSeqs[i], file_each_genome)
    scaffolds = [e.id for e in ls_genomeSeqs[i]]
    getGffEach(gffall=ls_gff3_genomeBased, scaffolds=scaffolds, outfile=file_each_est)
    getGffEach(gffall=ls_gff3_homologyBased, scaffolds=scaffolds, outfile=file_each_protein)
    getGffEach(gffall=ls_gff3_repeats, scaffolds=scaffolds, outfile=file_each_repeats)

    os.system('ln -s ' + file_maker_bopts + ' ' + outfolder + '/')
    os.system('ln -s ' + file_maker_exe + ' ' + outfolder + '/')
    os.system('ln -s ' + file_maker_opts + ' ' + outfolder + '/')

    open(os.path.join(folder_cmds,'qsub'+str(i)),'w').write(qsub_common.format(workfolder=outfolder))

#for i in range(N):
#    processOne(i)
#    break

from multiprocessing import Pool
pool = Pool(48)
pool.map(processOne,list(range(N)))
pool.close()




folders = '/work/biophysics/s185491/2018Hermeuptychia/20190106maker/*/maker_split/*'
foldercmd = '/work/biophysics/s185491/2018Hermeuptychia/20190106maker/cmds/'
import glob
import numpy as np
workfolders = glob.glob(folders)
commandlines = ['''cd {workfolder} && /home2/jshen3/apps/maker/bin/maker --ignore_nfs_tmp &\n'''.format(workfolder=e) for e in workfolders]
N=50
qsub_common = '''#!/bin/bash
#SBATCH --job-name=maker
#SBATCH --partition=32GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --time=10-00:00:00
#SBATCH --output=0.%j.out
#####SBATCH --mail-user=xiaolong.cao@utsouthwestern.edu
#SBATCH --mail-type=ALL


source /home2/s185491/p/anaconda3/anaconda520/bin/activate bio

{runninglines}

wait

'''

ls_commands = np.array_split(commandlines,N)
for i,commands in zip(range(N),ls_commands):
    file_commands = foldercmd+str(i)
    file_qsub = foldercmd+'qsub_'+str(i)
    runninglines = ''.join(list(commands))
    open(file_qsub,'w').write(qsub_common.format(runninglines=runninglines))


import glob
folder = '/work/biophysics/s185491/2018Hermeuptychia/20190106maker/'
files = glob.glob(folder+'/*/*/*/genome_each.maker.output/genome_each_datastore/**/scaffold*.gff', recursive=True)
fout = open(folder+'/maker_test.gff3','w')
for f in files:
    fout.write(open(f).read())
fout.close()
```

### step 19_annotate_consensus

```
mkdir /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/
cp /alea/home11/congqian/3314/19_annotate_consensus/*.py ./
source activate bio

python2 prepare_repeatmasker_for_EVM.py ../12_mask_repeats/3303_repeats.gff repeats.gff3
python2 prepare_repeatrunner_for_EVM.py ../18_annotate_denovo/maker_repeatrunner.gff maker_repeatrunner.gff3
cat repeats.gff3 maker_repeatrunner.gff3 >all_repeats.gff3

python prepare_glimmer_for_EVM.py ../18_annotate_denovo/glimmer.gff3 glimmer.gff3
python prepare_genemark_for_EVM.py ../18_annotate_denovo/maker_genemark.gff maker_genemark.gff3 
python prepare_augustus_for_EVM.py ../18_annotate_denovo/maker_augustus.gff maker_augustus.gff3
python prepare_maker_for_EVM.py ../18_annotate_denovo/maker_maker.gff maker_maker.gff3
cat maker_maker.gff3 glimmer.gff3 maker_genemark.gff3   maker_augustus.gff3 >denovo.gff3

python2 prepare_cufflinks_for_EVM.py ../14_annotate_transcript/stringtie.merged.gtf.gff3 stringtie.gff3
python2 prepare_pasa_for_EVM.py ../14_annotate_transcript/sample_mydb_pasa.sqlite.pasa_assemblies.gff3 pasa.gff3
cat stringtie.gff3 pasa.gff3 > transcript.gff3

python2 prepare_spaln_gff3_for_EVM.py ../18_annotate_denovo/homologyBased.combined.gff3 homolog.gff3 

mkdir EVM
cd EVM

/alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/partition_EVM_inputs.pl --genome ../../18_annotate_denovo/genome.fa --gene_predictions ../denovo.gff3 --protein_alignments ../homolog.gff3 --transcript_alignments ../transcript.gff3 --repeats ../all_repeats.gff3 --segmentSize 100000000 --overlapSize 10000 --partition_listing partitions_list.out 

perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/write_EVM_commands.pl --genome ../../18_annotate_denovo/genome.fa --gene_predictions ../denovo.gff3 --protein_alignments ../homolog.gff3 --transcript_alignments ../transcript.gff3 --repeats ../all_repeats.gff3 --output_file_name evm.out --partitions partitions_list.out --weights /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/weights.txt --search_long_introns 1 > commands

cp commands ../commands
cd ..
split commands commands_split_ -d -n 4 -a 2
python /home/xcao/p/xiaolongTools/multiThread.py 32 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/commands_split_00
python /home/xcao/p/xiaolongTools/multiThread.py 32 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/commands_split_01
python /home/xcao/p/xiaolongTools/multiThread.py 32 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/commands_split_02
python /home/xcao/p/xiaolongTools/multiThread.py 32 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/commands_split_03


perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/convert_EVM_outputs_to_GFF3.pl  --partitions partitions_list.out --output evm.out  --genome ../../18_annotate_denovo/genome.fa
#cat */evm.out.gff3 > ../EVM.gff3 #does not work. argument list too long
python /home/xcao/p/xiaolongTools/utils/annotation/combine_files.py -i */evm.out.gff3 -o ../EVM.gff3
perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/gff3_file_to_proteins.pl ../EVM.gff3 ../../18_annotate_denovo/genome.fa prot > ../EVM.fa
cd ..
cat EVM.fa | awk '{printf "%s\n",$1}' > EVM.protein.fa

```

3318
```
mkdir /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus
cp /alea/home11/congqian/3314/19_annotate_consensus/*.py ./
source activate bio


python2 prepare_repeatmasker_for_EVM.py ../12_mask_repeats/3318_repeats.gff repeats.gff3
python2 prepare_repeatrunner_for_EVM.py ../18_annotate_denovo/maker_repeatrunner.gff maker_repeatrunner.gff3
cat repeats.gff3 maker_repeatrunner.gff3 >all_repeats.gff3

python prepare_glimmer_for_EVM.py ../18_annotate_denovo/glimmer.gff3 glimmer.gff3
python prepare_genemark_for_EVM.py ../18_annotate_denovo/maker_genemark.gff maker_genemark.gff3 
python prepare_augustus_for_EVM.py ../18_annotate_denovo/maker_augustus.gff maker_augustus.gff3
python prepare_maker_for_EVM.py ../18_annotate_denovo/maker_maker.gff maker_maker.gff3
cat maker_maker.gff3 glimmer.gff3 maker_genemark.gff3   maker_augustus.gff3 >denovo.gff3

python2 prepare_cufflinks_for_EVM.py ../14_annotate_transcript/stringtie.merged.gtf.gff3 stringtie.gff3
python2 prepare_pasa_for_EVM.py ../14_annotate_transcript/3318_genomefilter1.fa.sqlite.pasa_assemblies.gff3 pasa.gff3
cat stringtie.gff3 pasa.gff3 > transcript.gff3

python2 prepare_spaln_gff3_for_EVM.py ../18_annotate_denovo/homologyBased.combined.gff3 homolog.gff3 

mkdir EVM
cd EVM

/alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/partition_EVM_inputs.pl --genome ../../18_annotate_denovo/genome.fa --gene_predictions ../denovo.gff3 --protein_alignments ../homolog.gff3 --transcript_alignments ../transcript.gff3 --repeats ../all_repeats.gff3 --segmentSize 100000000 --overlapSize 10000 --partition_listing partitions_list.out 

perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/write_EVM_commands.pl --genome ../../18_annotate_denovo/genome.fa --gene_predictions ../denovo.gff3 --protein_alignments ../homolog.gff3 --transcript_alignments ../transcript.gff3 --repeats ../all_repeats.gff3 --output_file_name evm.out --partitions partitions_list.out --weights /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/weights.txt --search_long_introns 1 > commands

cp commands ../commands
cd ..
split commands commands_split_ -d -n 2 -a 2
python /home/xcao/p/xiaolongTools/multiThread.py 32 /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/commands_split_00
python /home/xcao/p/xiaolongTools/multiThread.py 32 /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/commands_split_01

perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/convert_EVM_outputs_to_GFF3.pl  --partitions partitions_list.out --output evm.out  --genome ../../18_annotate_denovo/genome.fa

cat */evm.out.gff3 > ../EVM.gff3
perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/gff3_file_to_proteins.pl EVM.gff3 ../../18_annotate_denovo/genome.fa prot > ../EVM.fa
cd ..
cat EVM.fa | awk '{printf "%s\n",$1}' > EVM.protein.fa
```

#### EVM is slow. store files in memory
```
import os
os.chdir('/dev/shm/work/')
os.rename('/dev/shm/work/EVM/partitions_list.out','/dev/shm/work/partitions_list.out')
fout = open('EVM/partitions_list.out','w')
lines = open('partitions_list.out').readlines()
for l in lines:
    f = l.split()[1]+'/evm.out'
    if os.path.exists(f):
        if len(open(f).readlines()) != 0:
            fout.write(l)
fout.close()



import os,glob
os.chdir('/dev/shm/work/')
folders = glob.glob('/dev/shm/work/EVM/*')
os.mkdir('/dev/shm/work/EVM2/')
from collections import Counter
from Bio import SeqIO
print(Counter([len(glob.glob(e+'/*')) for e in folders]))
for f in folders:
    if os.path.isdir(f):
        seqlen = len(SeqIO.read(f+'/genome.fa','fasta'))
        if seqlen<=1000:
            os.rename(f,f.replace('EVM','EVM2'))



file1 = 'commands'
file2 = 'EVM/partitions_list.out'

ls1 = open(file1).readlines()
ls2 = open(file2).readlines()
ls1_keep = []
ls2_keep = []
from Bio import SeqIO
for l1,l2 in zip(ls1,ls2):
    fastafile = l2.split()[1] + '/genome.fa'
    if len(SeqIO.read(fastafile,'fasta').seq) > 1000:
        ls1_keep.append(l1)
        ls2_keep.append(l2)

open(file1,'w').write(''.join(ls1_keep))
open(file2,'w').write(''.join(ls2_keep))


```

```
cd /dev/shm
mkdir work
cd work
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/* ./
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/genome.fa ./
source activate bio
mkdir EVM
cd EVM


/alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/partition_EVM_inputs.pl --genome ../genome.fa --gene_predictions ../denovo.gff3 --protein_alignments ../homolog.gff3 --transcript_alignments ../transcript.gff3 --repeats ../all_repeats.gff3 --segmentSize 100000000 --overlapSize 10000 --partition_listing partitions_list.out 

perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/write_EVM_commands.pl --genome ../genome.fa --gene_predictions ../denovo.gff3 --protein_alignments ../homolog.gff3 --transcript_alignments ../transcript.gff3 --repeats ../all_repeats.gff3 --output_file_name evm.out --partitions partitions_list.out --weights /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/weights.txt --search_long_introns 1 > commands
mv commands ../commands
cd ..
python /home/xcao/p/xiaolongTools/multiThread.py 48 commands
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/genome.fa* ./
perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/convert_EVM_outputs_to_GFF3.pl  --partitions EVM/partitions_list.out --output evm.out  --genome genome.fa

cd /dev/shm
mkdir work
cd work
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/* ./
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/genome.fa ./
source activate bio
mkdir EVM
cd EVM
/alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/partition_EVM_inputs.pl --genome ../genome.fa --gene_predictions ../denovo.gff3 --protein_alignments ../homolog.gff3 --transcript_alignments ../transcript.gff3 --repeats ../all_repeats.gff3 --segmentSize 100000000 --overlapSize 10000 --partition_listing partitions_list.out 

perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/write_EVM_commands.pl --genome ../genome.fa --gene_predictions ../denovo.gff3 --protein_alignments ../homolog.gff3 --transcript_alignments ../transcript.gff3 --repeats ../all_repeats.gff3 --output_file_name evm.out --partitions partitions_list.out --weights /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/weights.txt --search_long_introns 1 > commands
cp commands ../commands
cd ..
python /home/xcao/p/xiaolongTools/multiThread_shortSleep.py 48 commands
perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/convert_EVM_outputs_to_GFF3.pl  --partitions EVM/partitions_list.out --output evm.out  --genome /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/EVM/../../18_annotate_denovo/genome.fa
cat EVM/*/evm.out.gff3 >EVM.gff3.20190112
cp EVM.gff3.20190112 /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/
perl /alea/home5/qc_4/software/evidencemodeler/EVM_r2012-06-25/EvmUtils/gff3_file_to_proteins.pl EVM.gff3.20190112 ../18_annotate_denovo/genome.fa prot > EVM20190112.fa

```

#### combine PASA and EVM to make the final version
EVM does not work well, as maker never work well. Combine EVM and PASA result.

```
gffread /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/sample_mydb_pasa.sqlite.pasa_assemblies.gtf -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/3303.pasa_assemblies.gff3
 
gffcompare /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/EVM.gff3.20190110 /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/3303.pasa_assemblies.gff3 -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/20190111_3303  -p 3303 -C -X
gffread /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/20190111_3303.combined.gtf -o /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/20190111_3303.combined.gff3



gffread /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/3318_genomefilter1.fa.sqlite.pasa_assemblies.gtf -o /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/3318.pasa_assemblies.gff3

gffcompare /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/EVM.gff3.20190110 /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/3318.pasa_assemblies.gff3 -o /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/20190111_3318  -p 3318 -C -X
gffread /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/20190111_3318.combined.gtf -o /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/20190111_3318.combined.gff3

cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/20190111_3303.combined.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/20190111_3318.combined.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/

```

### step 21_identify_snp_in_others
identify heterozygous rate

```
cd /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/21_identify_snp_in_others/
ln -s /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa genome.fa
ln -s /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/13_align_reads_v1/2_filter3303_250s.TrimP.fq.bam input.bam
java -jar /home/xcao/p/picard/picard2.18.20.jar CreateSequenceDictionary R=genome.fa O=genome.dict
mkdir tmp
samtools faidx genome.fa
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/picard/picard2.18.20.jar SortSam INPUT=input.bam OUTPUT=sorted.bam SORT_ORDER=coordinate >& sortBAM.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/picard/picard2.18.20.jar MarkDuplicates MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=1000 INPUT=sorted.bam OUTPUT=sorted_dedup.bam METRICS_FILE=sorted.metrics.txt >& markdup.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/picard/picard2.18.20.jar  AddOrReplaceReadGroups  INPUT=sorted_dedup.bam  OUTPUT=sorted_addrg.bam SORT_ORDER=coordinate RGID=group1 RGLB=lib1 RGPL=illumina RGPU=unit1 RGSM=4285 CREATE_INDEX=True >& add_readgroup.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/picard/picard2.18.20.jar  BuildBamIndex INPUT=sorted_addrg.bam >& index.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -T RealignerTargetCreator -R genome.fa -I sorted_addrg.bam -o sorted_realignment_targets.list >& prepare_realign.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -T IndelRealigner -R genome.fa -I sorted_addrg.bam -targetIntervals sorted_realignment_targets.list -o  sorted_realigned.bam >& realign.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -nct 32 -R genome.fa -T  UnifiedGenotyper -stand_call_conf 30  -I sorted_realigned.bam -o sorted_snp.vcf

```

3318
```
cd /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/21_identify_snp_in_others/
ln -s /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/3318_genomefilter1.fa genome.fa
ln -s /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/13_align_reads_v1/250bp.bam input.bam
samtools faidx genome.fa
java -jar /home/xcao/p/picard/picard2.18.20.jar CreateSequenceDictionary R=genome.fa O=genome.dict
mkdir tmp
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/picard/picard2.18.20.jar SortSam INPUT=input.bam OUTPUT=sorted.bam SORT_ORDER=coordinate >& sortBAM.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/picard/picard2.18.20.jar MarkDuplicates INPUT=sorted.bam MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=1000 OUTPUT=sorted_dedup.bam METRICS_FILE=sorted.metrics.txt >& markdup.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/picard/picard2.18.20.jar  AddOrReplaceReadGroups  INPUT=sorted_dedup.bam  OUTPUT=sorted_addrg.bam SORT_ORDER=coordinate RGID=group1 RGLB=lib1 RGPL=illumina RGPU=unit1 RGSM=4285 CREATE_INDEX=True >& add_readgroup.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/picard/picard2.18.20.jar  BuildBamIndex INPUT=sorted_addrg.bam >& index.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -T RealignerTargetCreator -R genome.fa -I sorted_addrg.bam -o sorted_realignment_targets.list >& prepare_realign.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -T IndelRealigner -R genome.fa -I sorted_addrg.bam -targetIntervals sorted_realignment_targets.list -o  sorted_realigned.bam >& realign.log
java -Djava.io.tmpdir=tmp -jar /home/xcao/p/gatk/GenomeAnalysisTK-3.8-1-0-gf15c1c3ef/GenomeAnalysisTK.jar -nct 32 -R genome.fa -T  UnifiedGenotyper -stand_call_conf 30  -I sorted_realigned.bam -o sorted_snp.vcf

```

### step 23 predict function

```
# get the fasta file contains all protein sequences, like protein.fasta
cd /work/biophysics/s185491/2018Hermeuptychia/20190114functionPrediction/3318/
cp /work/archive/biophysics/Nick_lab/jshen3/annotation/3639/23_predict_function/*.py .
cp /work/biophysics/s185491/2018Hermeuptychia/20190114functionPrediction/3318/20190111_3318.fa.transdecoder.pep.dedup protein.fasta
sed -i 's/\*//g' protein.fasta


# get the fasta file contains all protein sequences, like 3303.fasta
cd /work/biophysics/s185491/2018Hermeuptychia/20190114functionPrediction/3303/
cp /work/archive/biophysics/Nick_lab/jshen3/annotation/3639/23_predict_function/*.py .
cp /work/biophysics/s185491/2018Hermeuptychia/20190114functionPrediction/3303/20190111_3303.fa.transdecoder.pep.dedup protein.fasta




mkdir protein_prots
mkdir protein_blastsp
mkdir protein_blastdm
cd protein_prots
python2 ../split_fasta.py ../protein.fasta
ls -1 *.fa | awk '{split($1,a,".fa");printf "%s\n",a[1]}' > ../protein.list
cd ..

/home2/s185491/p/anaconda3/anaconda520/envs/bio/bin/blastp -query protein.fasta -out all.blastsp -evalue 0.00001 -db /work/archive/biophysics/Nick_lab/jshen3/database/swissprot -outfmt "6 qseqid sseqid pident evalue qlen qstart qend slen sstart send qseq sseq stitle" -task blastp-fast -num_threads 48 -num_alignments 1 
/home2/s185491/p/anaconda3/anaconda520/envs/bio/bin/blastp -query protein.fasta -out all.blastdm -evalue 0.00001 -db /work/archive/biophysics/Nick_lab/jshen3/database/Dm_prots -outfmt "6 qseqid sseqid pident evalue qlen qstart qend slen sstart send qseq sseq stitle" -task blastp-fast -num_threads 48 -num_alignments 1 
python blast_keep1.py all.blastsp
python blast_keep1.py all.blastdm


cp /work/archive/biophysics/Nick_lab/jshen3/annotation/4190/23_predict_function/Dm_def_map .
cp /work/archive/biophysics/Nick_lab/jshen3/annotation/4190/23_predict_function/Dm_go_map .
cp /work/archive/biophysics/Nick_lab/jshen3/annotation/4190/23_predict_function/swissprot_gos .
python2 get_annotation_from_blastdm.py protein.list > protein_flybase_annotation
python2 get_annotation_from_blastsp.py protein.list > protein_swissprot_annotation



mkdir protein_blocks
mkdir protein_interpro
python /home2/s185491/p/xiaolongTools/utils/seq/splitFasta2Nparts.py -s protein.fasta -o protein_blocks/ -N 15
#run interpro: scripts not shown here
cat ./protein_interpro/*.tsv >protein_interpro.tsv
python2 get_go_from_interpro.py protein_interpro.tsv > protein_interpro_go
python2 get_domain_from_interpro.py protein_interpro.tsv > protein_interpro_domain
python2 get_pathway_from_interpro.py protein_interpro.tsv > protein_interpro_pathway
python2 get_signal_from_interpro.py protein_interpro.tsv > protein_interpro_signal
cut -f 1,4 protein_flybase_annotation > protein_flybase_go
cut -f 1,4 protein_swissprot_annotation > protein_swissprot_go
cut -f 1,3 protein_swissprot_annotation > protein_func
cut -f 1,2 protein_flybase_annotation > protein_flybase_id
python2 integrate_go.py protein_interpro_go protein_flybase_go protein_swissprot_go > protein_gos
python2 integrate_all.py protein.list protein.fasta protein_func protein_gos protein_flybase_id protein_interpro_domain protein_interpro_pathway protein_interpro_signal > protein_table



```

#### keep only those in final kept proteins

```

file_result = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/23_predict_function/protein_table'
fasta_proteins = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/summary/20190111_3303.fa.transdecoder.pep.dedup.GeneGroup.quality_Xlen100repeats50overlap50'

file_result = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/23_predict_function/protein_table'
fasta_proteins = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/summary/20190111_3318.fa.transdecoder.pep.dedup.GeneGroup.quality_Xlen100repeats50overlap50'


import pandas as pd
import numpy as np
from Bio import SeqIO
df = pd.read_csv(file_result,sep='\t')
df2 = df.copy()
def getSeq(row):
    if row['Sequence'] is np.nan:
        return row['SignalP_EUK']
    else:
        return row['Sequence']
df['Sequence'] = df2.apply(getSeq, axis='columns')
def getSignalP(row):
    if len(row['SignalP_EUK']) > 30:
        return 'None'
    return row['SignalP_EUK']
df['SignalP_EUK'] = df.apply(getSignalP, axis='columns')

seq_ids = [s.id for s in SeqIO.parse(fasta_proteins,'fasta')]
seq_ids = set(seq_ids)
df_keep = df[df['proteinID'].apply(lambda x:x in seq_ids)]


df_keep.to_excel('/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/23_predict_function/3303filter_table.xlsx')

df_keep.to_excel('/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/23_predict_function/3318filter_table.xlsx')
```

## data processing for paper

### get genome information

```

import os
from multiprocessing import Pool
import pandas as pd

species = ['Ldi', 'Pra', 'Pse', 'Pgl', 'Ppo', 'Pxu', 'Dpl', 'Hme', 'Mci', 'Cce', 'Lac', 'Mse', 'Bmo', 'Pxy', 'Obr']
dc_genomes = {e:'/local/jzhang/genome_comparison_data/genomes/'+e.lower() for e in species}
dc_genomes['Hme'] = '/local/jzhang/genome_comparison_data/genomes/hm2'
dc_genomes['3303'] = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/8_improve_assembly/wenlin3303min200filter.fa'
dc_genomes['3318'] = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/8_improve_assembly/3318_genomefilter1.fa'

outfolder = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/genomes/'
for k,v in dc_genomes.items():
    os.system('cp '+v+' '+outfolder+k+'&')

species = list(dc_genomes.keys())
genomes = [outfolder + e for e in species]

def get_information_genome(filename):
    '''
    filename is a location of genome file. calculate 
        genome size with N
            and without N
        scaffold number
        Scaffold N50
        GC percentage
    return a dictionary. keys are 'genome_size_all', 'genome_size_noN', 'scf_count','N50', 'GC_ratio','N_ratio'
    '''
    from Bio import SeqIO
    dc = {}
    dc['genome_size_all'] = 0
    dc['genome_size_noN'] = 0
    dc['scf_count'] = 0
    dc['N50'] = 0
    dc['GC_ratio'] = 0
    ls_seqlen = []
    GC = 0
    for s in SeqIO.parse(filename,'fasta'):
        seq = str(s.seq)
        seq = seq.upper()
        seqlen = len(seq)
        seqlen_noN = sum(seq.count(e) for e in 'ATCG')
        dc['scf_count'] += 1
        ls_seqlen.append(seqlen)
        dc['genome_size_all'] += seqlen
        dc['genome_size_noN'] += seqlen_noN
        GC += sum(seq.count(e) for e in 'GC')
    dc['GC_ratio'] = GC / dc['genome_size_noN']
    ls_seqlen.sort()
    current = 0
    for l in ls_seqlen:
        current += l
        if current >= dc['genome_size_all'] * 0.5:
            dc['N50'] = l
            break
    dc['N_ratio'] = 1 - dc['genome_size_noN'] / dc['genome_size_all']
    dc['species'] = os.path.basename(filename)
    return dc

pool = Pool(17)
ls_info = pool.map(get_information_genome,genomes)
pool.close()

df = pd.DataFrame(ls_info)
df = df.set_index('species')
df = df.T
df.to_excel('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/genome_sum.xlsx')



```

### proteins

```
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/EVM.protein.fa /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins/3303pr
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/EVM.protein.fa /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins/3318pr
```

#### get transcript and protein sequence for the combined EVM and PASA result
20190111
```
gffread -w /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3303.fa -g /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/genome.fa /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.combined.gff3
gffread -w /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3318.combined.fa -g /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/genome.fa /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.gff3

gffread /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.combined.gff3 -o /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.combined.gtf -T

gffread /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.combined.gff3 -o /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.combined.gtf -T

/home/xcao/p/transdecoder/TransDecoder-TransDecoder-v5.5.0/util/gtf_to_alignment_gff3.pl /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.combined.gtf >/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.combined.gtf.gff3
/home/xcao/p/transdecoder/TransDecoder-TransDecoder-v5.5.0/util/gtf_to_alignment_gff3.pl /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.combined.gtf >/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.combined.gtf.gff3
```

#### get transcript_id to gene_id
```
#get transcript_id to gene_id
files = '''/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.combined.gff3
/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.combined.gff3
'''.split()

def get_transcript2gene(filename):
    '''
    filename is a gff3 file. get and write transcript_id to gene_id
    
    '''
    ls = open(filename).readlines()
    fout = open(filename.replace('combined.gff3','tr2gene'),'w')
    for l in ls:
        l = l.strip()
        if len(l)==0:
            continue
        if l[0]=='#':
            continue
        es = l.split('\t')
        if len(es)>8:
            if es[8].startswith('ID='):
                ess = es[8].split(';')
                tr_id = ess[0][3:]
                gene_id = ess[1][7:]
                fout.write(tr_id+'\t'+gene_id+'\n')
    fout.close()
    return None

for f in files:
    get_transcript2gene(f)
```

#### genome-guided translation

```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/
source activate bio

/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.LongOrfs -m 60 -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3303.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.Predict -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3303.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/cdna_alignment_orf_to_genome_orf.pl 20190111_3303.fa.transdecoder.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.combined.gtf.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3303.fa > 20190111_3303.fa.transdecoder.genome.gff3

/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.LongOrfs -m 60 -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3318.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.Predict -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3318.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/cdna_alignment_orf_to_genome_orf.pl 20190111_3318.fa.transdecoder.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.combined.gtf.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3318.fa > 20190111_3318.fa.transdecoder.genome.gff3
```

### improve proteins

#### combine genes with overlap of 10% sites or over 100bp

```

filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.combined.gff3'

lines = open(filename).readlines()
dc_scf2gene2loc = {}
for line in lines:
    line = line.strip()
    es = line.split('\t')
    if len(es) > 8:
        if 'geneID' in es[8]:
            geneID = es[8].split(';')[1][7:]
            scf_id = es[0]
            if scf_id not in dc_scf2gene2loc:
                dc_scf2gene2loc[scf_id] = {}
            if geneID not in dc_scf2gene2loc[scf_id]:
                dc_scf2gene2loc[scf_id][geneID] = set()
            dc_scf2gene2loc[scf_id][geneID].update(range(int(es[3]), int(es[4])+1))


#if two genes shares over 10% or 100bp overlap, merge them
import itertools
dc_scf2groups = {}
ratio_min = 0.1
overlap_min = 100
for scf, gene2loc in dc_scf2gene2loc.items():
    dc_scf2groups[scf] = set()
    genes = list(gene2loc.keys())
    if len(genes) == 1:
        dc_scf2groups.add((genes[0],))
    for gene1, gene2 in itertools.combinations(genes, 2):
        gene1_len = len(gene2loc[gene1])
        gene2_len = len(gene2loc[gene2])
        overlap = len(gene2loc[gene1] & gene2loc[gene2])
        ratio = overlap / min(gene1_len, gene2_len)
        if overlap > overlap_min or ratio > ratio_min:
            dc_scf2groups[scf].add((gene1, gene2))
        else:
            dc_scf2groups[scf].add((gene1,))
            dc_scf2groups[scf].add((gene2,))

dc_scf2lsgroup = {}
def groupingPairs(L):
    L = [list(e) for e in L]
    LL = set(itertools.chain.from_iterable(L)) 
    # LL is {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'k', 'o', 'p'}
    
    for each in LL:
        components = [x for x in L if each in x]
        for i in components:
            L.remove(i)
        L += [list(set(itertools.chain.from_iterable(components)))]
    return L
      
for scf, groups in dc_scf2groups.items():
    dc_scf2lsgroup[scf] = groupingPairs(groups)

dc_gene2group = {}
for n, genes in enumerate(itertools.chain.from_iterable(dc_scf2lsgroup.values())):
    for gene in genes:
        dc_gene2group[gene] = n

fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.gene2group','w')
for gene, group in dc_gene2group.items():
    fout.write('{gene}\t{group}\n'.format(gene=gene,group=group))
fout.close()
```

#### dealing with transdecoder for final version

* get transcritps with proteins.
* get genes with protein
* keep the longest protein for each gene (20190114 after checking BUSCO, if only keep the longest protein for each gene, some good proteins will be removed. For each gene, if a protein shares over 80% identical amino acids with the longer one, remove it.)
* calculate CDS length for all genes
* calculate intron length



```

file_3303 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.genome.gff3'
file_3318 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.genome.gff3'

dc_files = {'3303':file_3303, '3318':file_3318}
dc_fileinfo = {}
for k,filename in dc_files.items():
    l = open(filename).readlines()
    lines = [e.strip().split('\t') for e in l if len(e.split('\t'))>8]
    dc_fileinfo[k] = lines

def calculateLength(lines,keywords = 'gene'):
    l = [e for e in lines if e[2]==keywords]
    dc_scf = {}
    for e in l:
        if e[0] not in dc_scf:
            dc_scf[e[0]] = set()
        start = int(e[3])-1
        end = int(e[4])
        if start > end:
            print(e,'start > end')
        dc_scf[e[0]].update(range(start,end))
    return sum(len(v) for v in dc_scf.values())

print('3303 gene len',calculateLength(dc_fileinfo['3303'],'gene'))
print('3303 CDS len', calculateLength(dc_fileinfo['3303'],'CDS'))
print('3318 gene len',calculateLength(dc_fileinfo['3318'],'gene'))
print('3318 CDS len', calculateLength(dc_fileinfo['3318'],'CDS'))

def getCodingTranscripts(line):
    coding = set()
    for l in line:
        if l[2] == 'CDS':
            coding.add(l[8].split(';')[0].split('.')[1])
    print('coding transcripts',len(coding))
    return coding

dc_coding={}#store coding transcripts
for k,v in dc_fileinfo.items():
    print(k)
    dc_coding[k] = getCodingTranscripts(v)

import pandas as pd
df_t2g_3303 = pd.read_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.tr2gene',header=None,index_col=0,sep='\t')
df_t2g_3318 = pd.read_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.tr2gene',header=None,index_col=0,sep='\t')
dc_t2g_3303 = df_t2g_3303.to_dict()[1]
dc_t2g_3318 = df_t2g_3318.to_dict()[1]

dc_coding_genes = {}
dc_coding_genes['3303'] = set([dc_t2g_3303[e] for e in dc_coding['3303']])
dc_coding_genes['3318'] = set([dc_t2g_3318[e] for e in dc_coding['3318']])
for k,v in dc_coding_genes.items():
    print(k,'coding gene',len(v))
    
def calculateLengthCodingGene(lines,keywords = 'gene',genes=dc_coding_genes['3303']):
    l = [e for e in lines if e[2]==keywords and e[8].split(';')[0][3:] in genes]
    dc_scf = {}
    for e in l:
        if e[0] not in dc_scf:
            dc_scf[e[0]] = set()
        start = int(e[3])-1
        end = int(e[4])
        if start > end:
            print(e,'start > end')
        dc_scf[e[0]].update(range(start,end))
    return sum(len(v) for v in dc_scf.values())

for k,v in dc_fileinfo.items():
    print(k,'coding gene length', calculateLengthCodingGene(v,'gene',dc_coding_genes[k]))

#keep longest protein for each gene
filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep'
genes = dc_coding_genes['3303']
from Bio import SeqIO
proteins = list(SeqIO.parse(filename,'fasta'))
dc_proteins = {}
for p in proteins:
    gene = dc_t2g_3303[p.id.split('.')[0]]
    if gene not in dc_proteins:
        dc_proteins[gene] = []
    dc_proteins[gene].append(p)

pr_3303 = []
for seqs in dc_proteins.values():
    if len(seqs) == 1:
        pr_3303.append(seqs[0])
    else:
        seqs.sort(key=lambda x:len(x.seq))
        pr_3303.append(seqs[-1])

fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep.longestPerGene','w')
for seq in pr_3303:
    fout.write('>'+seq.description+'\n'+str(seq.seq)+'\n')
fout.close()

#keep longest protein for each gene
filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep'
genes = dc_coding_genes['3318']
from Bio import SeqIO
proteins = list(SeqIO.parse(filename,'fasta'))
dc_proteins = {}
for p in proteins:
    gene = dc_t2g_3318[p.id.split('.')[0]]
    if gene not in dc_proteins:
        dc_proteins[gene] = []
    dc_proteins[gene].append(p)

pr_3318 = []
for seqs in dc_proteins.values():
    if len(seqs) == 1:
        pr_3318.append(seqs[0])
    else:
        seqs.sort(key=lambda x:len(x.seq))
        pr_3318.append(seqs[-1])

fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.longestPerGene','w')
for seq in pr_3318:
    fout.write('>'+seq.description+'\n'+str(seq.seq)+'\n')
fout.close()

#keep less redundant protein for each gene
filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep'
genes = dc_coding_genes['3303']
from Bio import SeqIO
proteins = list(SeqIO.parse(filename,'fasta'))
dc_proteins = {}
for p in proteins:
    gene = dc_t2g_3303[p.id.split('.')[0]]
    if gene not in dc_proteins:
        dc_proteins[gene] = []
    dc_proteins[gene].append(p)

import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/seq')
import sequenceComparison

pr_3303 = []
pr_3303_longest = []
for seqs in dc_proteins.values():
    if len(seqs) == 1:
        pr_3303.append(seqs[0])
    else:
        seqs.sort(key=lambda x:len(x.seq),reverse=True)
        pr_3303.append(seqs[0])
        pr_3303_longest.append(seqs[0])
        seqmaxlen = len(seqs[0].seq)
        for seq in seqs[1:]:
            matchlen = sequenceComparison.getProteinAlignLength([seqs[0],seq], muscle_exe='/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/muscle')
            if matchlen / len(seq.seq) < 0.6:
                pr_3303.append(seq)
                print(seqs[0].id, seqmaxlen, seq.id, len(seq.seq), matchlen, 'keep')
            else:
                print(seqs[0].id, seqmaxlen, seq.id, len(seq.seq), matchlen, 'remove')

def get_unique_per_gene(seqs):
    to_return = []
    if len(seqs) == 1:
        to_return.append(seqs[0])
    else:
        seqs.sort(key=lambda x:len(x.seq),reverse=True)
        to_return.append(seqs[0])
        seqmaxlen = len(seqs[0].seq)
        for seq in seqs[1:]:
            matchlen = sequenceComparison.getProteinAlignLength([seqs[0],seq], muscle_exe='/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/muscle')
            if matchlen / len(seq.seq) < 0.6:
                to_return.append(seq)
                print(seqs[0].id, seqmaxlen, seq.id, len(seq.seq), matchlen, 'keep')
            else:
                print(seqs[0].id, seqmaxlen, seq.id, len(seq.seq), matchlen, 'remove')
    return to_return

from multiprocessing import Pool
pool = Pool(32)
pr_3303_chain = pool.map(get_unique_per_gene, dc_proteins.values())
pool.close()
import itertools
pr_3303 = list(itertools.chain.from_iterable(pr_3303_chain))


fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep.dedup','w')
for seq in pr_3303:
    fout.write('>'+seq.description+'\n'+str(seq.seq)+'\n')
fout.close()


#3318
filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep'
genes = dc_coding_genes['3318']
from Bio import SeqIO
proteins = list(SeqIO.parse(filename,'fasta'))
dc_proteins = {}
for p in proteins:
    gene = dc_t2g_3318[p.id.split('.')[0]]
    if gene not in dc_proteins:
        dc_proteins[gene] = []
    dc_proteins[gene].append(p)

import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/seq')
import sequenceComparison

pr_3318 = []
for seqs in dc_proteins.values():
    if len(seqs) == 1:
        pr_3318.append(seqs[0])
    else:
        seqs.sort(key=lambda x:len(x.seq),reverse=True)
        pr_3318.append(seqs[0])
        seqmaxlen = len(seqs[0].seq)
        for seq in seqs[1:]:
            matchlen = sequenceComparison.getProteinAlignLength([seqs[0],seq], muscle_exe='/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/muscle')
            if matchlen / len(seq.seq) < 0.6:
                pr_3318.append(seq)
                print(seqs[0].id, seqmaxlen, seq.id, len(seq.seq), matchlen, 'keep')
            else:
                print(seqs[0].id, seqmaxlen, seq.id, len(seq.seq), matchlen, 'remove')
                
fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.dedup','w')
for seq in pr_3318:
    fout.write('>'+seq.description+'\n'+str(seq.seq)+'\n')
fout.close()

```


#### dealing with transdecoder, with gene groups

some genes overlap with each other. use gene group to present genes

the same as previous step. genes were further grouped. Use group instead of gene.
```

import pandas as pd
df_t2g_3303 = pd.read_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.tr2gene',header=None,index_col=0,sep='\t')
df_t2g_3318 = pd.read_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.tr2gene',header=None,index_col=0,sep='\t')
df_g2G_3303 = pd.read_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.gene2group',header=None,index_col=0,sep='\t')
df_g2G_3318 = pd.read_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.gene2group',header=None,index_col=0,sep='\t')
dc_t2g_3303 = df_t2g_3303.to_dict()[1]
dc_t2g_3318 = df_t2g_3318.to_dict()[1]
dc_g2G_3303 = df_g2G_3303.to_dict()[1]
dc_g2G_3318 = df_g2G_3318.to_dict()[1]
dc_t2G_3303 = {k:dc_g2G_3303[v] for k,v in dc_t2g_3303.items()}
dc_t2G_3318 = {k:dc_g2G_3318[v] for k,v in dc_t2g_3318.items()}


#keep less redundant protein for each gene
filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep'
from Bio import SeqIO
proteins = list(SeqIO.parse(filename,'fasta'))
dc_proteins = {}
for p in proteins:
    gene = dc_t2G_3303[p.id.split('.')[0]]
    if gene not in dc_proteins:
        dc_proteins[gene] = []
    dc_proteins[gene].append(p)

import sys
sys.path.append('/home/xcao/p/xiaolongTools/utils/seq')
import sequenceComparison
import itertools

def get_unique_per_gene(seqs):
    seqlen = len(seqs)
    seqs = sequenceComparison.fasta_uni_keepone(seqs)
    if len(seqs) == 1:
        return seqs
    else:
        seqs.sort(key=lambda x:len(x.seq),reverse=True)
        to_remove = set()
        for seq1,seq2 in itertools.combinations(seqs,2):
            if len(seq1.seq) < len(seq2.seq):
                seq1, seq2 = seq2, seq1
            matchlen = sequenceComparison.getProteinAlignLength([seq1,seq2], muscle_exe='/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/muscle')
            if matchlen / len(seq2.seq) > 0.6:
                to_remove.add(seq2.id)
    print('input', seqlen, 'seqs, with', seqlen -len(seqs),'seqs identical and ', len(to_remove), 'seqs further removed')
    return [seq for seq in seqs if seq.id not in to_remove]

from multiprocessing import Pool
pool = Pool(32)
pr_3303_chain = pool.map(get_unique_per_gene, dc_proteins.values())
pool.close()
pr_3303 = list(itertools.chain.from_iterable(pr_3303_chain))


fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep.dedup.GeneGroup','w')
for seq in pr_3303:
    fout.write('>'+seq.description+'\n'+str(seq.seq)+'\n')
fout.close()


#3318
filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep'
from Bio import SeqIO
proteins = list(SeqIO.parse(filename,'fasta'))
dc_proteins = {}
for p in proteins:
    gene = dc_t2G_3318[p.id.split('.')[0]]
    if gene not in dc_proteins:
        dc_proteins[gene] = []
    dc_proteins[gene].append(p)

pool = Pool(32)
pr_3318_chain = pool.map(get_unique_per_gene, dc_proteins.values())
pool.close()
pr_3318 = list(itertools.chain.from_iterable(pr_3318_chain))

fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.dedup.GeneGroup','w')
for seq in pr_3318:
    fout.write('>'+seq.description+'\n'+str(seq.seq)+'\n')
fout.close()


filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.dedup.GeneGroup'
file_transcript_gff = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.combined.gff3'
file_repeat_gff = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/all_repeats.gff3'
file_transcript = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3318.fa'

filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep.dedup.GeneGroup'
file_transcript_gff = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3303.combined.gff3'
file_repeat_gff = '/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/all_repeats.gff3'
file_transcript = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3303.fa'


from Bio import SeqIO
import pandas as pd
seqs = list(SeqIO.parse(filename,'fasta'))

df = pd.DataFrame()
df['seqid'] = [e.id for e in seqs]
df['seqlen'] = [len(str(e.seq).strip('*')) for e in seqs]
df['seqlenNoX'] = [len(str(e.seq).strip('*')) - str(e.seq).upper().count('X') for e in seqs]
df['orf_type'] = [e.description.split()[3].split(':')[1] for e in seqs]
df['transcript_id'] = df['seqid'].apply(lambda x:x.split('.')[0])
seq_transcript = list(SeqIO.parse(file_transcript,'fasta'))
dc_transcriptlen = {e.id:len(e.seq) for e in seq_transcript}

from collections import defaultdict
dc_repeat = defaultdict(set) #key is scaffold, value is a set of sites belong to repeats
for line in open(file_repeat_gff):
    es = line.split()
    if len(es)>8:
        dc_repeat[es[0]].update(list(range(int(es[3]), int(es[4])+1)))

dc_transcript = {}#key is protein_id, value [scaffold_id, set of exon sites in that scaffold] 
for line in open(file_transcript_gff):
    line = line.strip()
    es = line.split()
    if len(es)>8:
        if es[2] == 'exon':
            transcript_id = es[8].split(';')[0].split('=')[1]
            if transcript_id not in dc_transcript:
                dc_transcript[transcript_id] =[es[0], set(list(range(int(es[3]), int(es[4])+1)))]

dc_transcript2gene = {}
for line in open(file_transcript_gff):
    line = line.strip()
    es = line.split()
    if len(es)>8:
        if es[2] == 'transcript':
            transcript_id = es[8].split(';')[0].split('=')[1]
            gene_id = es[8].split(';')[1].split('=')[1]
            dc_transcript2gene[transcript_id] = gene_id

df['scf_id'] = df['transcript_id'].apply(lambda x:dc_transcript[x][0])
df['repeats_count'] = df.apply(lambda x: len(dc_repeat[x['scf_id']] & dc_transcript[x['transcript_id']][1]), axis='columns')
df['transcript_len'] = df['transcript_id'].apply(lambda x:dc_transcriptlen[x])
df['repeats_ratio'] = df['repeats_count'] / df['transcript_len']
df['gene_id'] = df['transcript_id'].apply(lambda x: dc_transcript2gene[x])

df['quality_Xlen'] = df.apply(lambda x: (x['seqlenNoX']>100) or (x['orf_type'] != 'internal') ,axis='columns')
df.to_csv(filename + '.info',sep='\t')

df_filter_qualityXlen = df[df['quality_Xlen']]
df_filter_qualityXlen_repeat05 = df[(df['repeats_ratio'] < 0.05) & df['quality_Xlen']  ]
df_filter_qualityXlen_repeat25 = df[(df['repeats_ratio'] < 0.25) & df['quality_Xlen']  ]
df_filter_qualityXlen_repeat50 = df[(df['repeats_ratio'] < 0.50) & df['quality_Xlen']  ]
df_filter_qualityXlen_repeat75 = df[(df['repeats_ratio'] < 0.75) & df['quality_Xlen']  ]


def writeSeqs(dfuse, outfilename):
    seqids = set(dfuse['seqid'])
    fout = open(outfilename,'w')
    for s in seqs:
        if s.id in seqids:
            fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
    fout.close()
writeSeqs(df_filter_qualityXlen,filename+'.quality_Xlen100')
writeSeqs(df_filter_qualityXlen_repeat05,filename+'.quality_Xlen100repeats05')
writeSeqs(df_filter_qualityXlen_repeat25,filename+'.quality_Xlen100repeats25')
writeSeqs(df_filter_qualityXlen_repeat50,filename+'.quality_Xlen100repeats50')
writeSeqs(df_filter_qualityXlen_repeat75,filename+'.quality_Xlen100repeats75')


print(len(df_filter_qualityXlen.gene_id.unique()))
print(len(df_filter_qualityXlen_repeat05.gene_id.unique()))
print(len(df_filter_qualityXlen_repeat25.gene_id.unique()))
print(len(df_filter_qualityXlen_repeat50.gene_id.unique()))
print(len(df_filter_qualityXlen_repeat75.gene_id.unique()))

txt = 'source activate bio && cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins && python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/{file} -o {file} -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ & '

files = '''20190111_3303.fa.transdecoder.pep.dedup.GeneGroup
20190111_3303.fa.transdecoder.pep.dedup.GeneGroup.quality_Xlen100
20190111_3303.fa.transdecoder.pep.dedup.GeneGroup.quality_Xlen100repeats05
20190111_3303.fa.transdecoder.pep.dedup.GeneGroup.quality_Xlen100repeats25
20190111_3303.fa.transdecoder.pep.dedup.GeneGroup.quality_Xlen100repeats50
20190111_3303.fa.transdecoder.pep.dedup.GeneGroup.quality_Xlen100repeats75
'''.split()

for file in files:
    print(txt.format(file=file))
```

#### remove overlapped proteins

```
file1 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep.dedup.quality_Xlen100repeats50'
file2 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.dedup.quality_Xlen100repeats50'

gff1 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.genome.gff3'
gff2 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.genome.gff3'



def getDC2Pr2loc(filename):
    lines = open(filename).readlines()
    dc_scf2gene2loc = {}
    for line in lines:
        line = line.strip()
        es = line.split('\t')
        if len(es) > 8:
            if es[2] == 'CDS':
                geneID = es[8].split(';')[0][3:]
                scf_id = es[0]
                if scf_id not in dc_scf2gene2loc:
                    dc_scf2gene2loc[scf_id] = {}
                if geneID not in dc_scf2gene2loc[scf_id]:
                    dc_scf2gene2loc[scf_id][geneID] = set()
                dc_scf2gene2loc[scf_id][geneID].update(range(int(es[3]), int(es[4])+1))
    return dc_scf2gene2loc

dc_3303scf2pr2loc = getDC2Pr2loc(gff1)
dc_3318scf2pr2loc = getDC2Pr2loc(gff2)

#if two proteins shares over 50% remove the shorter one
def getPr2group(dc_scf2gene2loc):
    import itertools
    dc_scf2groups = {}
    ratio_min = 0.5
    for scf, gene2loc in dc_scf2gene2loc.items():
        dc_scf2groups[scf] = set()
        genes = list(gene2loc.keys())
        if len(genes) == 1:
            dc_scf2groups[scf].add((genes[0],))
        for gene1, gene2 in itertools.combinations(genes, 2):
            gene1_len = len(gene2loc[gene1])
            gene2_len = len(gene2loc[gene2])
            overlap = len(gene2loc[gene1] & gene2loc[gene2])
            ratio = overlap / min(gene1_len, gene2_len)
            if  ratio > ratio_min:
                dc_scf2groups[scf].add((gene1, gene2))
            else:
                dc_scf2groups[scf].add((gene1,))
                dc_scf2groups[scf].add((gene2,))

    dc_scf2lsgroup = {}
    def groupingPairs(L):
        L = [list(e) for e in L]
        LL = set(itertools.chain.from_iterable(L)) 
        # LL is {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'k', 'o', 'p'}

        for each in LL:
            components = [x for x in L if each in x]
            for i in components:
                L.remove(i)
            L += [list(set(itertools.chain.from_iterable(components)))]
        return L

    for scf, groups in dc_scf2groups.items():
        dc_scf2lsgroup[scf] = groupingPairs(groups)

    dc_gene2group = {}
    for n, genes in enumerate(itertools.chain.from_iterable(dc_scf2lsgroup.values())):
        for gene in genes:
            dc_gene2group[gene] = n
    return dc_gene2group

dc_3303_pr2group = getPr2group(dc_3303scf2pr2loc)
dc_3318_pr2group = getPr2group(dc_3318scf2pr2loc)
dc_3303_pr2group = {k.replace('cds.',''):v for k,v in dc_3303_pr2group.items()}
dc_3318_pr2group = {k.replace('cds.',''):v for k,v in dc_3318_pr2group.items()}

from Bio import SeqIO
seqs1 = list(SeqIO.parse(file1,'fasta'))
seqs2 = list(SeqIO.parse(file2, 'fasta'))
dc_group2pr ={}
seqs_nogroup = []
for s in seqs1:
    if s.id in dc_3303_pr2group:
        group = dc_3303_pr2group[s.id]
        if group not in dc_group2pr:
            dc_group2pr[group] = []
        dc_group2pr[group].append(s)
    else:
        seqs_nogroup.append(s)
fout = open(file1+'overlap50','w')
for k,s in dc_group2pr.items():
    s.sort(key=lambda x:len(x))
    s = s[-1]
    fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
fout.close()
fout = open(file1+'ovelap50_outgenome','w')
for s in seqs_nogroup:
    fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
fout.close()

dc_group2pr ={}
seqs_nogroup = []
for s in seqs2:
    if s.id in dc_3318_pr2group:
        group = dc_3318_pr2group[s.id]
        if group not in dc_group2pr:
            dc_group2pr[group] = []
        dc_group2pr[group].append(s)
    else:
        seqs_nogroup.append(s)
fout = open(file2+'overlap50','w')
for k,s in dc_group2pr.items():
    s.sort(key=lambda x:len(x))
    s = s[-1]
    fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
fout.close()
fout = open(file2+'ovelap50_outgenome','w')
for s in seqs_nogroup:
    fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
fout.close()



```

```
file1 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep.dedup.quality_Xlen100repeats50'
file2 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.dedup.quality_Xlen100repeats50'

gff1 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.genome.gff3'
gff2 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.genome.gff3'



def getDC2Pr2loc(filename):
    lines = open(filename).readlines()
    dc_scf2gene2loc = {}
    for line in lines:
        line = line.strip()
        es = line.split('\t')
        if len(es) > 8:
            if es[2] == 'CDS':
                geneID = es[8].split(';')[0][3:]
                scf_id = es[0]
                if scf_id not in dc_scf2gene2loc:
                    dc_scf2gene2loc[scf_id] = {}
                if geneID not in dc_scf2gene2loc[scf_id]:
                    dc_scf2gene2loc[scf_id][geneID] = set()
                dc_scf2gene2loc[scf_id][geneID].update(range(int(es[3]), int(es[4])+1))
    return dc_scf2gene2loc

dc_3303scf2pr2loc = getDC2Pr2loc(gff1)
dc_3318scf2pr2loc = getDC2Pr2loc(gff2)

#if two proteins shares over 50% remove the shorter one
def getPr2group(dc_scf2gene2loc):
    import itertools
    dc_scf2groups = {}
    ratio_min = 0.5
    for scf, gene2loc in dc_scf2gene2loc.items():
        dc_scf2groups[scf] = set()
        genes = list(gene2loc.keys())
        if len(genes) == 1:
            dc_scf2groups[scf].add((genes[0],))
        for gene1, gene2 in itertools.combinations(genes, 2):
            gene1_len = len(gene2loc[gene1])
            gene2_len = len(gene2loc[gene2])
            overlap = len(gene2loc[gene1] & gene2loc[gene2])
            ratio = overlap / min(gene1_len, gene2_len)
            if  ratio > ratio_min:
                dc_scf2groups[scf].add((gene1, gene2))
            else:
                dc_scf2groups[scf].add((gene1,))
                dc_scf2groups[scf].add((gene2,))
    
    dc_scf2lsgroup = {}
    def groupingPairs(L):
        L = [list(e) for e in L]
        LL = set(itertools.chain.from_iterable(L)) 
        # LL is {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'k', 'o', 'p'}
        
        for each in LL:
            components = [x for x in L if each in x]
            for i in components:
                L.remove(i)
            L += [list(set(itertools.chain.from_iterable(components)))]
        return L
          
    for scf, groups in dc_scf2groups.items():
        dc_scf2lsgroup[scf] = groupingPairs(groups)
    
    dc_gene2group = {}
    for n, genes in enumerate(itertools.chain.from_iterable(dc_scf2lsgroup.values())):
        for gene in genes:
            dc_gene2group[gene] = n
    return dc_gene2group

dc_3303_pr2group = getPr2group(dc_3303scf2pr2loc)
dc_3318_pr2group = getPr2group(dc_3318scf2pr2loc)
dc_3303_pr2group = {k.replace('cds.',''):v for k,v in dc_3303_pr2group.items()}
dc_3318_pr2group = {k.replace('cds.',''):v for k,v in dc_3318_pr2group.items()}

from Bio import SeqIO
seqs1 = list(SeqIO.parse(file1,'fasta'))
seqs2 = list(SeqIO.parse(file2, 'fasta'))
dc_group2pr ={}
for s in seqs1:
    if s.id in dc_3303_pr2group:
        group = dc_3303_pr2group[s.id]
        if group not in dc_group2pr:
            dc_group2pr[group] = []
        dc_group2pr[group].append(s)
    else:
        dc_group2pr[s.id] = [s]
fout = open(file1+'overlap50','w')
for k,s in dc_group2pr.items():
    s.sort(key=lambda x:len(x))
    s = s[-1]
    fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
fout.close()

dc_group2pr ={}
for s in seqs2:
    if s.id in dc_3318_pr2group:
        group = dc_3318_pr2group[s.id]
        if group not in dc_group2pr:
            dc_group2pr[group] = []
        dc_group2pr[group].append(s)
    else:
        dc_group2pr[s.id] = [s]
fout = open(file2+'overlap50','w')
for k,s in dc_group2pr.items():
    s.sort(key=lambda x:len(x))
    s = s[-1]
    fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
fout.close()
```

#### 20190115 further improve protein
remove those overlap with repetitive regions


```
filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.dedup'

from Bio import SeqIO
import pandas as pd
seqs = list(SeqIO.parse(filename,'fasta'))

df = pd.DataFrame()
df['seqid'] = [e.id for e in seqs]
df['seqlen'] = [len(str(e.seq).strip('*')) for e in seqs]
df['seqlenNoX'] = [len(str(e.seq).strip('*')) - str(e.seq).upper().count('X') for e in seqs]
df['orf_type'] = [e.description.split()[3].split(':')[1] for e in seqs]
df['transcript_id'] = df['seqid'].apply(lambda x:x.split('.')[0])

file_transcript_gff = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/20190111_3318.combined.gff3'
file_repeat_gff = '/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/all_repeats.gff3'
file_transcript = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/20190111_3318.fa'
seq_transcript = list(SeqIO.parse(file_transcript,'fasta'))
dc_transcriptlen = {e.id:len(e.seq) for e in seq_transcript}

from collections import defaultdict
dc_repeat = defaultdict(set) #key is scaffold, value is a set of sites belong to repeats
for line in open(file_repeat_gff):
    es = line.split()
    if len(es)>8:
        dc_repeat[es[0]].update(list(range(int(es[3]), int(es[4])+1)))

dc_transcript = {}#key is protein_id, value [scaffold_id, set of exon sites in that scaffold] 
for line in open(file_transcript_gff):
    line = line.strip()
    es = line.split()
    if len(es)>8:
        if es[2] == 'exon':
            transcript_id = es[8].split(';')[0].split('=')[1]
            if transcript_id not in dc_transcript:
                dc_transcript[transcript_id] =[es[0], set(list(range(int(es[3]), int(es[4])+1)))]

dc_transcript2gene = {}
for line in open(file_transcript_gff):
    line = line.strip()
    es = line.split()
    if len(es)>8:
        if es[2] == 'transcript':
            transcript_id = es[8].split(';')[0].split('=')[1]
            gene_id = es[8].split(';')[1].split('=')[1]
            dc_transcript2gene[transcript_id] = gene_id

df['scf_id'] = df['transcript_id'].apply(lambda x:dc_transcript[x][0])
df['repeats_count'] = df.apply(lambda x: len(dc_repeat[x['scf_id']] & dc_transcript[x['transcript_id']][1]), axis='columns')
df['transcript_len'] = df['transcript_id'].apply(lambda x:dc_transcriptlen[x])
df['repeats_ratio'] = df['repeats_count'] / df['transcript_len']
df['gene_id'] = df['transcript_id'].apply(lambda x: dc_transcript2gene[x])

df['quality_Xlen'] = df.apply(lambda x: (x['seqlenNoX']>100) or (x['orf_type'] != 'internal') ,axis='columns')
df.to_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/summary/20190111_3318.fa.info',sep='\t')

df_filter_qualityXlen = df[df['quality_Xlen']]
df_filter_qualityXlen_repeat05 = df[(df['repeats_ratio'] < 0.05) & df['quality_Xlen']  ]
df_filter_qualityXlen_repeat25 = df[(df['repeats_ratio'] < 0.25) & df['quality_Xlen']  ]
df_filter_qualityXlen_repeat50 = df[(df['repeats_ratio'] < 0.50) & df['quality_Xlen']  ]
df_filter_qualityXlen_repeat75 = df[(df['repeats_ratio'] < 0.75) & df['quality_Xlen']  ]


def writeSeqs(dfuse, outfilename):
    seqids = set(dfuse['seqid'])
    fout = open(outfilename,'w')
    for s in seqs:
        if s.id in seqids:
            fout.write('>'+s.description+'\n'+str(s.seq)+'\n')
    fout.close()
writeSeqs(df_filter_qualityXlen,filename+'.quality_Xlen100')
writeSeqs(df_filter_qualityXlen_repeat05,filename+'.quality_Xlen100repeats05')
writeSeqs(df_filter_qualityXlen_repeat25,filename+'.quality_Xlen100repeats25')
writeSeqs(df_filter_qualityXlen_repeat50,filename+'.quality_Xlen100repeats50')
writeSeqs(df_filter_qualityXlen_repeat75,filename+'.quality_Xlen100repeats75')


print(len(df_filter_qualityXlen.gene_id.unique()))
print(len(df_filter_qualityXlen_repeat05.gene_id.unique()))
print(len(df_filter_qualityXlen_repeat25.gene_id.unique()))
print(len(df_filter_qualityXlen_repeat50.gene_id.unique()))
print(len(df_filter_qualityXlen_repeat75.gene_id.unique()))

txt = 'source activate bio && cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins && python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/{file} -o {file} -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ & '

files = '''20190111_3303.fa.transdecoder.pep.dedup.quality_Xlen100
20190111_3303.fa.transdecoder.pep.dedup.quality_Xlen100repeats05
20190111_3303.fa.transdecoder.pep.dedup.quality_Xlen100repeats25
20190111_3303.fa.transdecoder.pep.dedup.quality_Xlen100repeats50
20190111_3303.fa.transdecoder.pep.dedup.quality_Xlen100repeats75
20190111_3318.fa.transdecoder.pep.dedup.quality_Xlen100
20190111_3318.fa.transdecoder.pep.dedup.quality_Xlen100repeats05
20190111_3318.fa.transdecoder.pep.dedup.quality_Xlen100repeats25
20190111_3318.fa.transdecoder.pep.dedup.quality_Xlen100repeats50
20190111_3318.fa.transdecoder.pep.dedup.quality_Xlen100repeats75
'''.split()

for file in files:
    print(txt.format(file=file))
```

#### BUSCO for kept proteins
also for other transdecoder results
```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep.longestPerGene -o 20190111_3303.fa.transdecoder.pep.longestPerGene -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.longestPerGene -o 20190111_3318.fa.transdecoder.pep.longestPerGene -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/

python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep -o 20190111_3303.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep -o 20190111_3318.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/

cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins && python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/3303.pasa.fa.transdecoder.pep -o 3303.pasa.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ & 
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins && python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/3303.stringtie.fa.transdecoder.pep -o 3303.stringtie.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ & 
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins && python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/3318.pasa.fa.transdecoder.pep -o 3318.pasa.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ & 
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins && python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/3318.stringtie.fa.transdecoder.pep -o 3318.stringtie.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ & 

cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins && python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep.dedup -o 20190111_3303.fa.transdecoder.pep.dedup -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ & 
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins && python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.dedup -o 20190111_3318.fa.transdecoder.pep.dedup -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ & 

```

### quast

```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/ && /home/xcao/p/quast/quast-5.0.2/quast.py -t 32 --eukaryote --large  --no-gzip ./genomes/* -o ./quast -k -m 1 --conserved-genes-finding &

```

```
cd /home/xcao/w/genomes/Bombyx_mori/ && /home/xcao/p/quast/quast-5.0.2/quast.py -t 32 --eukaryote --large  --no-gzip Bomo_genome_assembly.fa -o ./quast -k -m 1 --conserved-genes-finding &

```

### BUSCO

#### EVM proteins

for proteins, BUSCO
```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins/ 
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3303pr -o 3303pr_BUSCO -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ 
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3318pr -o 3318pr_BUSCO -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/

cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/19_annotate_consensus/EVM.fa 3303evm.20190110
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/19_annotate_consensus/EVM.fa 3318evm.20190110
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3303evm.20190110 -o 3303evm.20190110_BUSCO -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3318evm.20190110 -o 3318evm.20190110_BUSCO -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/

python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3303evm.20190112 -o 3303evm.20190112_BUSCO -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3318evm.20190112 -o 3318evm.20190112_BUSCO -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/
```

#### PASA, stringtie, Trinity, Trinity-GG maker transcripts

get sequences
```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie.merged.gtf.fasta 3303.stringtie.fa
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/Trinity.fasta Trinity.fa
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/Trinity-GG.fasta Trinity-GG.fa
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/sample_mydb_pasa.sqlite.assemblies.fasta 3303.pasa.fa
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie.merged.gtf.fasta 3318.stringtie.fa
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/3318_genomefilter1.fa.sqlite.assemblies.fasta 3318.pasa.fa
gffread -w 3303.maker.fa -g /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/genome.fa /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/18_annotate_denovo/maker_maker.gff
gffread -w 3318.maker.fa -g /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/genome.fa /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/18_annotate_denovo/maker_maker.gff


```

#### genomes
```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/genomes
source activate bio
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3303 -o 3303_BUSCO2 -c 32 -m geno -l /home/xcao/p/BUSCO/db/insecta_odb9/ --long
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3318 -o 3318_BUSCO2 -c 32 -m geno -l /home/xcao/p/BUSCO/db/insecta_odb9/ --long
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i Ldi -o Ldi_BUSCO -c 32 -m geno -l /home/xcao/p/BUSCO/db/insecta_odb9/

```

run Bombyx with new genome
```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/genomes
cp /home/xcao/w/genomes/Bombyx_mori/Bomo_genome_assembly.fa ./
source activate bio
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i Bomo_genome_assembly.fa -c 48 -m geno -l /home/xcao/p/BUSCO/db/insecta_odb9/ -o Bomo_genome_BUSCO
```

run BUSCO

```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3303.maker.fa -o 3303.maker.fa_BUSCO -c 32 -m tran -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3303.pasa.fa -o 3303.pasa.fa_BUSCO -c 32 -m tran -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3303.stringtie.fa -o 3303.stringtie.fa_BUSCO -c 32 -m tran -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3318.maker.fa -o 3318.maker.fa_BUSCO -c 32 -m tran -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3318.pasa.fa -o 3318.pasa.fa_BUSCO -c 32 -m tran -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3318.stringtie.fa -o 3318.stringtie.fa_BUSCO -c 32 -m tran -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i Trinity.fa -o Trinity.fa_BUSCO -c 32 -m tran -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i Trinity-GG.fa -o Trinity-GG.fa_BUSCO -c 32 -m tran -l /home/xcao/p/BUSCO/db/insecta_odb9/


```

#### proteins to get species Tree

```
txt_species = '''/home/xcao/w/genomes/proteins_ref/all_pr/aly  Achalarus_lyciades
/home/xcao/w/genomes/proteins_ref/all_pr/atr  Amyelois_transitella
/home/xcao/w/genomes/proteins_ref/all_pr/bmo  Bombyx_mori
/home/xcao/w/genomes/proteins_ref/all_pr/cce  Calycopis_cecrops
/home/xcao/w/genomes/proteins_ref/all_pr/cne  Calephelis_nemesis
/home/xcao/w/genomes/proteins_ref/all_pr/csu  Chilo_suppressalis
/home/xcao/w/genomes/proteins_ref/all_pr/dpl  Danaus plexippus
/home/xcao/w/genomes/proteins_ref/all_pr/GCF_000836215.1_Ppol_1.0_protein.faa  Papilio_polytes
/home/xcao/w/genomes/proteins_ref/all_pr/GCF_002938995.1_ASM293899v1_protein_Vanessa_tameamea.faa  Vanessa_tameamea
/home/xcao/w/genomes/proteins_ref/all_pr/GCF_003589595.1_ASM358959v1_protein_Hyposmocoma_kahamanoa.faa  Hyposmocoma_kahamanoa
/home/xcao/w/genomes/proteins_ref/all_pr/GCF_900239965.1_Bicyclus_anynana_v1.2_protein.faa  Bicyclus_anynana
/home/xcao/w/genomes/proteins_ref/all_pr/hm2  Heliconius_melpomene_melpomene
/home/xcao/w/genomes/proteins_ref/all_pr/lac  Lerema_accius
/home/xcao/w/genomes/proteins_ref/all_pr/ldi  Lymantria_dispar
/home/xcao/w/genomes/proteins_ref/all_pr/mci  Melitaea_cinxia
/home/xcao/w/genomes/proteins_ref/all_pr/mse  Manduca_sexta
/home/xcao/w/genomes/proteins_ref/all_pr/obr  Operophtera_brumata
/home/xcao/w/genomes/proteins_ref/all_pr/pgl  Pterourus_glaucus
/home/xcao/w/genomes/proteins_ref/all_pr/pin  Plodia_interpunctella
/home/xcao/w/genomes/proteins_ref/all_pr/pra  Pieris_rapae
/home/xcao/w/genomes/proteins_ref/all_pr/pse  Phoebis_sennae
/home/xcao/w/genomes/proteins_ref/all_pr/pxu  Papilio_xuthus
/home/xcao/w/genomes/proteins_ref/all_pr/pxy  Plutella_xylostella
/home/xcao/w/genomes/proteins_ref/all_pr/tni  Trichoplusia_ni
/home/xcao/w/genomes/proteins_ref/all_pr/mvi  Megathymus_ursus_violae
/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/3303.pasa.fa.transdecoder.pep  3303_pasa
/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/3303.stringtie.fa.transdecoder.pep  3303_stringtie
/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/3318.pasa.fa.transdecoder.pep  3318_pasa
/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/3318.stringtie.fa.transdecoder.pep  3318_stringtie
/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/Trinity.fa.transdecoder.pep  satyr_trinity
'''

dc_species ={}
_lsinfo = txt_species.strip().split('\n')
for e in _lsinfo:
    es = e.split()
    dc_species[es[1]] = es[0]

import os
outfolder = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins/'
for k,v in dc_species.items():
    os.system('cd {outfolder} && python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i {filein} -o {fileout} -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/ '.format(filein = v, fileout = 'BUSCO_'+k, outfolder=outfolder))

#get complete sequences for each species
    
def getCompleteIDs(filename):
    '''
    filename is a BUSCo full table
    return dictionary of protein IDs that is Complete or Duplicated. If Duplicated, return the last one
    '''
    dc = {}
    for line in open(filename):
        if len(line)<1:
            continue
        if line[0] == '#':
            continue
        es = line.split()
        if len(es) <5:
            continue
        if es[1] == 'Complete' or es[1] == 'Duplicated':
            dc[es[0]] = es[2]
    return dc

file_BUSCO = [outfolder+'run_'+'BUSCO_'+k+'/full_table_BUSCO_'+k+'.tsv' for k in dc_species.keys()]
import pandas as pd
results = [getCompleteIDs(e) for e in file_BUSCO]
df_complete = pd.DataFrame(results)
df_complete.index = dc_species.keys()
df_complete = df_complete.T
#allow 3 missing values per protein
missing_allowed = 3
df_good = df_complete[df_complete.isna().sum(axis=1) <= missing_allowed]

from Bio import SeqIO
dc_species_seq = {}
for k in dc_species:
    dc_species_seq[k] = SeqIO.to_dict(SeqIO.parse(dc_species[k],'fasta'))

for n, line in df_good.iterrows():
    #break
    fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins/BUSCO_complete_fasta/'+n,'w')
    for k,v in line.items():
        try:
            seq = dc_species_seq[k][v]
            seq='>'+k+'\n'+str(seq.seq)+'\n'
        except:
            seq=''
        fout.write(seq)
    fout.close()
#run muscle
import os
import glob
folder_fasta = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins/BUSCO_complete_fasta/'
folder_out = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins/BUSCO_complete_fastaAlign/'
files = os.listdir(folder_fasta)
for f in files:
    os.system('muscle -in {filein} -out {fileout} -maxiters 1000 &'.format(filein = folder_fasta+f, fileout=folder_out+f))
#combine the alignments
file_out = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins/BUSCO_complete_combine/combined.fa'
dc_aligns = {k:'' for k in dc_species}
for f in files:
    _dc = SeqIO.to_dict(SeqIO.parse(folder_out+f,'fasta'))
    for s in _dc.values():
        break
    seqlen = len(s.seq)
    for s in dc_aligns:
        if s in _dc:
            dc_aligns[s] = dc_aligns[s] + str(_dc[s].seq)
        else:
            dc_aligns[s] = dc_aligns[s] + '-'*seqlen
fout = open(file_out,'w')
for k,v in dc_aligns.items():
    fout.write('>'+k+'\n'+v+'\n')
fout.close()

outfolder = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/PrOfSpecies/'
for k,v in dc_species.items():
    os.system('cp '+v+' '+outfolder+k)

l = '''3303_pasa
3303_stringtie
3318_pasa
3318_stringtie
Achalarus_lyciades
Amyelois_transitella
Bicyclus_anynana
Bombyx_mori
Calephelis_nemesis
Calycopis_cecrops
Chilo_suppressalis
Danaus
Heliconius_melpomene_melpomene
Hyposmocoma_kahamanoa
Lerema_accius
Lymantria_dispar
Manduca_sexta
Megathymus_ursus_violae
Melitaea_cinxia
Operophtera_brumata
Papilio_polytes
Papilio_xuthus
Phoebis_sennae
Pieris_rapae
Plodia_interpunctella
Plutella_xylostella
Pterourus_glaucus
satyr_trinity
Trichoplusia_ni
Vanessa_tameamea
'''.split()
txt = '/export/home5/qc_4/software/orthomcl/orthomclSoftware-v2.0.9/bin/orthomclAdjustFasta {key} ../PrOfSpecies/{key} 1'
for e in l:
    print(txt.format(key=e))


cmd = 'blastp -num_threads 1 -task blastp-fast -query ./blast/seq{n} -db goodProteins -evalue 0.00001 -max_target_seqs 10000 -outfmt 6 -out ./blast/seq{n}.blast &'
for n in range(32):
    os.system(cmd.format(n=n))
```

#### species Tree

run RAxML

```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/proteins/BUSCO_complete_combine &&\
/home/xcao/p/RAxML/standard-RAxML-master/raxmlHPC-PTHREADS-SSE3 -m PROTGAMMAGTR -p 234 -s combined30.fa -x 12345 -n combined30 -T 32 -N 100 -f a

```

### ribosome proteins
```
cat /export/home10/congqian/3306/24_compare_genomes/ribosome/Dmel_ribosome_prots/CG* >/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/ribosome/Dmel_ribosome_prots.fa
#make blastDB
source activate bio
makeblastdb -in /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/genomes/3303 -dbtype nucl -out /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/blastDB/3303nt
makeblastdb -in /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/genomes/3318 -dbtype nucl -out /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/blastDB/3318nt
makeblastdb -in /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/summary/20190111_3303.fa.transdecoder.pep.dedup -out /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/ribosome/20190111_3303.fa.transdecoder.pep.dedup -dbtype prot
makeblastdb -in /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/summary/20190111_3318.fa.transdecoder.pep.dedup -out /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/ribosome/20190111_3318.fa.transdecoder.pep.dedup -dbtype prot



```

```

file_query = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/ribosome/Dmel_ribosome_prots.fa'
file_check = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/summary/20190111_3318.fa.transdecoder.pep.dedup'
#file_check = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep'
#file_check = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/PrOfSpecies/Bombyx_mori'
MAKEBLASTDB = '/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/makeblastdb'
workfolder = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/ribosome/'
BLASTP = '/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/blastp'
import os
#make blastdb
basename_query = os.path.basename(file_check)
cmd = '{MAKEBLASTDB} -in {file_check} -out {workfolder}/{basename_query}.blastDB -dbtype prot'.format(MAKEBLASTDB=MAKEBLASTDB, file_check=file_check, workfolder=workfolder,basename_query=basename_query)
os.system(cmd)

#run blastp
cmd = '''{BLASTP} -db {workfolder}/{basename_query}.blastDB -query {file_query} -out {workfolder}/{basename_query}.blast6 -outfmt "6 qseqid sseqid pident evalue qlen qstart qend slen sstart send"  -num_threads 48 -evalue 0.0000000001'''.format(BLASTP=BLASTP, workfolder=workfolder, file_check=file_check,basename_query=basename_query,file_query=file_query)
os.system(cmd)

import pandas as pd
df = pd.read_csv('{workfolder}/{basename_query}.blast6'.format(workfolder=workfolder, basename_query=basename_query), sep='\t',header=None)
df.columns = ['qseqid', 'sseqid', 'pident', 'evalue', 'qlen', 'qstart', 'qend', 'slen', 'sstart', 'send']
#df.drop_duplicates(subset='qseqid')
df = df.sort_values(by=['sseqid','evalue'])
df_result = df.drop_duplicates(subset=['sseqid'])
df_result=df_result.sort_values(by=['qseqid','evalue'])
df_result = df_result.drop_duplicates(subset=['qseqid'])

#based on genome
file_check = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/genomes/3318'
TBLASTN = '/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/tblastn'
#make blastdb
basename_query = os.path.basename(file_check)
cmd = '{MAKEBLASTDB} -in {file_check} -out {workfolder}/{basename_query}.blastDB -dbtype nucl'.format(MAKEBLASTDB=MAKEBLASTDB, file_check=file_check, workfolder=workfolder,basename_query=basename_query)
os.system(cmd)
#run tblastn
cmd = '''{TBLASTN} -db {workfolder}/{basename_query}.blastDB -query {file_query} -out {workfolder}/{basename_query}.blast6 -outfmt "6 qseqid sseqid pident evalue qlen qstart qend slen sstart send"  -num_threads 48 -evalue 0.0000000001'''.format(TBLASTN=TBLASTN, workfolder=workfolder, file_check=file_check,basename_query=basename_query,file_query=file_query)
os.system(cmd)
import pandas as pd
df = pd.read_csv('{workfolder}/{basename_query}.blast6'.format(workfolder=workfolder, basename_query=basename_query), sep='\t',header=None)
df.columns = ['qseqid', 'sseqid', 'pident', 'evalue', 'qlen', 'qstart', 'qend', 'slen', 'sstart', 'send']
df = df[(df['evalue'] < 1e-5) & (df['pident']>50)]
dc_query2position = {}
dc_query2len = {}
for n,row in df.iterrows():
    if row['qseqid'] not in dc_query2position:
        dc_query2position[row['qseqid']] = set()
    dc_query2position[row['qseqid']].update(range(row['qstart'], row['qend']+1))
    dc_query2len[row['qseqid']] = row['qlen']
dc_query2position = {k:len(v) for k,v in dc_query2position.items()}
df_result = pd.DataFrame([dc_query2len,dc_query2position])
df_result = df_result.T
df_result.columns = ['qlen','match_len']
df_result['cov'] = df_result['match_len'] / df_result['qlen']
df_result.to_csv('{workfolder}/{basename_query}.ribosome.result'.format(workfolder=workfolder, basename_query=basename_query), sep='\t')
df_result.shape

df2 = pd.read_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/ribosome/3318.blast6',sep='\t',header=None)
df2.columns = ['qseqid', 'sseqid', 'pident', 'evalue', 'qlen', 'qstart', 'qend', 'slen', 'sstart', 'send']
df3 = pd.concat([df,df2])
df3 = df3.sort_values(by=['qseqid','evalue'])
df4 =df3.drop_duplicates(subset=['qseqid'])
df4.shape


df1 = pd.read_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/ribosome/3303.blast6', sep='\t',header=None)
df2 = pd.read_csv('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/ribosome/20190111_3303.fa.transdecoder.pep.dedup.blast6', sep='\t',header=None)
df3 = pd.concat([df1,df2],ignore_index=True)
df3.columns = ['qseqid', 'sseqid', 'pident', 'evalue', 'qlen', 'qstart', 'qend', 'slen', 'sstart', 'send']
df3 = df3.sort_values(by=['qseqid','evalue'])
df4 =df3.drop_duplicates(subset=['qseqid'])
df4.shape


```

### repeats summary

```
python2 /alea/home10/congqian/3306/12_mask_repeats/cleanup_table.py /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/12_mask_repeats/3303_repeats.sum >/home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/12_mask_repeats/3303_repeats.table

python2 /alea/home10/congqian/3306/12_mask_repeats/cleanup_table.py /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/12_mask_repeats/3318_repeats.sum >/home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/12_mask_repeats/3318_repeats.sum.table
```

### gff files

```
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/stringtie.merged.gtf.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/3303.stringtie.gff3
cp /home/xcao/w/20181205Hermeuptychia/20181208QianPipeline/14_annotate_transcript/sample_mydb_pasa.sqlite.pasa_assemblies.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/3303.PASA.gff3
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/stringtie.merged.gtf.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/3318.stringtie.gff3
cp /home/xcao/w/20181205Hermeuptychia/20181224QianPipleline_3318/14_annotate_transcript/3318_genomefilter1.fa.sqlite.pasa_assemblies.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/3318.PASA.gff3

```

### translate proteins

```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/
source activate bio

/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.LongOrfs -m 60 -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/Trinity.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.Predict -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/Trinity.fa

/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.LongOrfs -m 60 -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/Trinity-GG.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.Predict -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/Trinity-GG.fa

/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.LongOrfs -m 60 -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3303.pasa.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.Predict -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3303.pasa.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/cdna_alignment_orf_to_genome_orf.pl 3303.pasa.fa.transdecoder.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/3303.PASA.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3303.pasa.fa > 3303.pasa.fa.transdecoder.genome.gff3

/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.LongOrfs -m 60 -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3303.stringtie.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.Predict -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3303.stringtie.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/cdna_alignment_orf_to_genome_orf.pl 3303.stringtie.fa.transdecoder.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/3303.stringtie.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3303.stringtie.fa > 3303.stringtie.fa.transdecoder.genome.gff3


/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.LongOrfs -m 60 -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3318.pasa.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.Predict -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3318.pasa.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/cdna_alignment_orf_to_genome_orf.pl 3318.pasa.fa.transdecoder.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/3318.PASA.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3318.pasa.fa > 3318.pasa.fa.transdecoder.genome.gff3

/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.LongOrfs -m 60 -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3318.stringtie.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/TransDecoder.Predict -t /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3318.stringtie.fa
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/cdna_alignment_orf_to_genome_orf.pl 3318.stringtie.fa.transdecoder.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/gff/3318.stringtie.gff3 /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transcripts/3318.stringtie.fa > 3318.stringtie.fa.transdecoder.genome.gff3

```

#### translated protein BUSCO

```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder
source activate bio
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3303.pasa.fa.transdecoder.pep -o 3303.pasa.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3318.pasa.fa.transdecoder.pep -o 3318.pasa.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3303.stringtie.fa.transdecoder.pep -o 3303.stringtie.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/
python /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/run_BUSCO.py -i 3318.stringtie.fa.transdecoder.pep -o 3318.stringtie.fa.transdecoder.pep -c 32 -m prot -l /home/xcao/p/BUSCO/db/insecta_odb9/
```

### orthomcl

```

txt_species = '''/home/xcao/w/genomes/proteins_ref/all_pr/aly  aly
/home/xcao/w/genomes/proteins_ref/all_pr/atr  atr
/home/xcao/w/genomes/proteins_ref/all_pr/bmo  bmo
/home/xcao/w/genomes/proteins_ref/all_pr/cce  cce
/home/xcao/w/genomes/proteins_ref/all_pr/cne  cne
/home/xcao/w/genomes/proteins_ref/all_pr/csu  csu
/home/xcao/w/genomes/proteins_ref/all_pr/dpl  dpl
/home/xcao/w/genomes/proteins_ref/all_pr/GCF_000836215.1_Ppol_1.0_protein.faa  ppo
/home/xcao/w/genomes/proteins_ref/all_pr/GCF_002938995.1_ASM293899v1_protein_Vanessa_tameamea.faa  vta
/home/xcao/w/genomes/proteins_ref/all_pr/GCF_003589595.1_ASM358959v1_protein_Hyposmocoma_kahamanoa.faa  hka
/home/xcao/w/genomes/proteins_ref/all_pr/GCF_900239965.1_Bicyclus_anynana_v1.2_protein.faa  ban
/home/xcao/w/genomes/proteins_ref/all_pr/hm2  hm2
/home/xcao/w/genomes/proteins_ref/all_pr/lac  lac
/home/xcao/w/genomes/proteins_ref/all_pr/ldi  ldi
/home/xcao/w/genomes/proteins_ref/all_pr/mci  mci
/home/xcao/w/genomes/proteins_ref/all_pr/mse  mse
/home/xcao/w/genomes/proteins_ref/all_pr/obr  obr
/home/xcao/w/genomes/proteins_ref/all_pr/pgl  pgl
/home/xcao/w/genomes/proteins_ref/all_pr/pin  pin
/home/xcao/w/genomes/proteins_ref/all_pr/pra  pra
/home/xcao/w/genomes/proteins_ref/all_pr/pse  pse
/home/xcao/w/genomes/proteins_ref/all_pr/pxu  pxu
/home/xcao/w/genomes/proteins_ref/all_pr/pxy  pxy
/home/xcao/w/genomes/proteins_ref/all_pr/tni  tni
/home/xcao/w/genomes/proteins_ref/all_pr/mvi  mvi
/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3303.fa.transdecoder.pep.dedup  3303
/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/transdecoder/20190111_3318.fa.transdecoder.pep.dedup  3318
'''
dc_species ={}
_lsinfo = txt_species.strip().split('\n')
for e in _lsinfo:
    es = e.split()
    dc_species[es[1]] = es[0]

outfolder = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/pr_all/'
import os
from Bio import SeqIO
def renamePr(filename,species):
    '''
    for proteins in pr, reanme protein names to 6digit number P000001
    '''
    fout = open(outfolder+species,'w')
    for n, seq in enumerate(SeqIO.parse(filename,'fasta')):
        fout.write('>P{n:06d}\t{description}\n{sequence}\n'.format(n=n+1, description=seq.description, sequence = str(seq.seq)))
    fout.close()

for k,v in dc_species.items():
    renamePr(v,k)

orthofolder = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/PrOfSpecies/'
os.chdir(orthofolder)
cmd = '/alea/home5/qc_4/software/orthomcl/orthomclSoftware-v2.0.9/bin/orthomclAdjustFasta  {k} {filein} 1'
for k in dc_species:
    os.system(cmd.format(filein = outfolder+k, fileout = orthofolder+k, k=k))
```



```
cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl
/alea/home5/qc_4/software/orthomcl/orthomclSoftware-v2.0.9/bin/orthomclFilterFasta ./PrOfSpecies 10 20
mv goodProteins.fasta goodProteins
/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/makeblastdb -in goodProteins -dbtype prot

#split to 32 parts
python /home/xcao/p/xiaolongTools/utils/seq/splitFasta2Nparts.py -s /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/goodProteins -N 320 -o /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/blast/seq
#run blastp
#combine the result
cat /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/blast/seq*.blast > /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/goodProteins.blast

```

```
cmd = 'cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/blast/ && /home/xcao/p/anaconda3_5.2.0/envs/bio/bin/blastp -query seq{n} -db ../goodProteins -max_target_seqs 1000 -outfmt 6 -out seq{n}.blast -evalue 0.00001 '

qsub = '''#!/bin/bash
#$ -S /bin/sh
#$ -cwd
#$ -j y
source /home/xcao/.bashrc
{cmd}
'''
for n in range(240):
    open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/blastcmds/qsub'+str(n),'w').write(qsub.format(cmd=cmd.format(n=n)))


```
morpho qsub jobs with bash

for i in {0..239}; do qsub /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/blastcmds/qsub$i; done;


```
cat /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/blast/seq*.blast >/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/goodProteins.blast

cd /home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/
/alea/home5/qc_4/software/orthomcl/orthomclSoftware-v2.0.9/bin/orthomclBlastParser goodProteins.blast ./compliantFasta/ >> similarSequences.txt

/alea/home5/qc_4/software/orthomcl/orthomclSoftware-v2.0.9/bin/orthomclInstallSchema ./orthomcl.config install_schema.log
/alea/home5/qc_4/software/orthomcl/orthomclSoftware-v2.0.9/bin/orthomclLoadBlast ./orthomcl.config ./similarSequences.txt
/alea/home5/qc_4/software/orthomcl/orthomclSoftware-v2.0.9/bin/orthomclPairs ./orthomcl.config ./orthomcl_pairs.log -cleanup=no
/alea/home5/qc_4/software/orthomcl/orthomclSoftware-v2.0.9/bin/orthomclDumpPairsFiles ./orthomcl.config

/home/jshen/apps/mcl-14-137/bin/mcl ./mclInput --abc -I 1.5 -o ./mclOutput

/alea/home5/qc_4/software/orthomcl/orthomclSoftware-v2.0.9/bin/orthomclMclToGroups LEP 10000 < mclOutput > groups.txt
python2 select_groups_for_tree.py > groups_for_tree
mkdir proteins_for_tree
python2 get_proteins.py groups_for_tree
cat list | awk '{printf "mkdir %s_prots\n",$1}' | bash
cat list | awk '{printf "cd %s_prots\npython ../split_fasta.py ../%s.fasta &\ncd ..\n",$1,$1}' | bash
cat id_mapping  | awk '{printf "cd %s_transcripts\npython ../split_fasta.py ../../all_transcriptomes/%s &\ncd ..\n",$1,$2}' | bash
python get_transcripts.py groups_for_tree
cd transcripts_for_tree/
wc -l *.fa | sort -k 1 -g | head -145 | awk '{printf "rm %s\n",$2}' | bash # remove the files that does not contain all the species
ls -1 *.fa | awk '{split($1,a,".");printf "%s\n",a[1]}' > list
cd ../proteins_for_tree/
ls -1 *.fa | awk '{split($1,a,".");printf "%s\n",a[1]}' > list

```

#### further process, remove proteins

```
folder = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/compliantFasta/'
outfolder = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/blast2Dm/'
import glob
from Bio import SeqIO
import os
files = glob.glob(folder+'*')
for f in files:
    fout = open(outfolder+os.path.basename(f),'w')
    for s in SeqIO.parse(f,'fasta'):
        species, pr_id = s.id.split('|')
        fout.write('>'+species+'|'+species+pr_id+'\n'+str(s.seq)+'\n')
    fout.close()

files = [outfolder+os.path.basename(f) for f in files]
for f in files:
    fout = open(outfolder+'../'+os.path.basename(f).split('.')[0]+'.len','w')
    for s in SeqIO.parse(f,'fasta'):
        fout.write(s.id+'\t'+str(len(s.seq))+'\n')
    fout.close()

filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/groups.txt'
ls_info = open(filename).readlines()
fout = open(filename,'w')
def helpfun(x):
    if '|' in x:
        s,i = x.split('|')
        return s+'|'+s+i
    return x
for l in ls_info:
    es = l.split()
    es = [helpfun(e) for e in es]
    fout.write(' '.join(es)+'\n')
fout.close()
    


BLASTP = '/home/xcao/p/anaconda3_5.2.0/envs/bio/bin/blastp'
BLASTDB = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/blast2Dm/Dm_prots'
for f in files:
    cmd = '''{BLASTP} -db {BLASTDB} -query {file_query} -out {file_out} -outfmt "6 qseqid sseqid pident evalue qlen qstart qend slen sstart send"  -num_threads 48 -evalue 0.0000000001 '''.format(BLASTP=BLASTP, BLASTDB=BLASTDB, file_query=f, file_out=f+'.blast_Dm')
    print(cmd)

fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/all_map_to_flybase','w')
ids = set()
for f in files:
    for l in open(f+'.blast_Dm'):
        es = l.split()
        if es[0] not in ids:
            ids.add(es[0])
            pr_id = es[0].split('|')[1]
            fout.write(pr_id+'\t'+es[1]+'\n')
fout.close()


# get dc_newName2oldname
dc_pr_n2o ={}
files_pr_ori = glob.glob('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/pr_all/*')
for f in files_pr_ori:
    species = os.path.basename(f)
    for s in SeqIO.parse(f,'fasta'):
        des = s.description.split()
        newname = species + des[0]
        oldname = species + des[1]
        dc_pr_n2o[newname] = oldname
dc_pr_o2n ={v:k for k,v in dc_pr_n2o.items()}



fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/speciesOldName2NewName','w')
for k,v in dc_pr_n2o.items():
    fout.write(k+'\t'+v+'\n')
fout.close()

file_fun1 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/3303_fun_ori'
file_fun2 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/3318_fun_ori'
fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/3303_func','w')
for l in open(file_fun1):
    e1,e2 = l.split('\t')
    e1 = dc_pr_o2n['3303'+e1]
    fout.write(e1+'\t'+e2)
fout.close()
fout = open('/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/3318_func','w')
for l in open(file_fun2):
    e1,e2 = l.split('\t')
    e1 = dc_pr_o2n['3318'+e1]
    fout.write(e1+'\t'+e2)
fout.close()

filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/duplicate_groups_step4_3303'
ls_info = open(filename).readlines()
os.rename(filename,filename+'.backup')
fout = open(filename,'w')
for l in ls_info:
    names, info = l.split('\t',maxsplit=1)
    names = names.split(',')
    newnames = [dc_pr_n2o[e] for e in names]
    newnames = [e[4:] for e in newnames]
    newnames = ','.join(newnames)
    fout.write(newnames+'\t'+info)
fout.close()
filename = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/duplicate_groups_step4_3318'
ls_info = open(filename).readlines()
os.rename(filename,filename+'.backup')
fout = open(filename,'w')
for l in ls_info:
    names, info = l.split('\t',maxsplit=1)
    names = names.split(',')
    newnames = [dc_pr_n2o[e] for e in names]
    newnames = [e[4:] for e in newnames]
    newnames = ','.join(newnames)
    fout.write(newnames+'\t'+info)
fout.close()


#for groups.txt, all_map_to_flybase, change sequence names back to their original name. didn't do it
file_groups_txt = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/groups.txt'
ls_groups_txt = open(file_groups_txt).readlines()
os.rename(file_groups_txt,file_groups_txt+'.backup')
def helpfun(x):
    if '|' in x:
        s,i = x.split('|')
        return s+'|'+dc_pr_n2o[i]
    return x
fout = open(file_groups_txt)
for l in ls_groups_txt:
    es = l.split()
    es = [helpfun(e) for e in es]
    fout.write(' '.join(es)+'\n')
fout.close()


#for group.txt, for 3303 and 3318, only keep those in filtered fasta file
file_pr_o2n = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/speciesOldName2NewName'
file_3303 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/summary/20190111_3303.fa.transdecoder.pep.dedup.quality_Xlen100repeats50overlap50'
file_3318 = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/summary/20190111_3318.fa.transdecoder.pep.dedup.quality_Xlen100repeats50overlap50'
file_group = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/groups.txt'

import pandas as pd
from Bio import SeqIO
df = pd.read_csv(file_pr_o2n, sep='\t', header=None)
df.columns = ['newName','oldName']
def oldname2name(x):
    if x[0]== '3':
        return x[4:]
    return x[3:]
df['ori_name'] = df['oldName'].apply(oldname2name)

dc_oriName2newName = dict(zip(df['ori_name'], df['newName']))

keep_ids = [e.id for e in SeqIO.parse(file_3303,'fasta')] + [e.id for e in SeqIO.parse(file_3318,'fasta')]
keep_idsNewName = [dc_oriName2newName[e] for e in keep_ids]
keep_idsNewName = set(keep_idsNewName)

def helpfun(x):
    if '|' not in x:
        return x
    species = x.split('|')[0]
    if species not in ['3303','3318']:
        return x
    pr_id = x.split('|')[1]
    if pr_id not in keep_idsNewName:
        print('remove',x)
        return ''
    return x

lines = open(file_group+'.backup').readlines()
fout = open(file_group,'w')
for line in lines:
    es = line.split()
    es =[helpfun(e) for e in es]
    fout.write(' '.join(es)+'\n')
fout.close()

file_map2filybase = '/home/xcao/w/20181205Hermeuptychia/20190105CompareGenomes/orthomcl/all_map_to_flybase'
lines = open(file_map2filybase+'.backup').readlines()
fout = open(file_map2filybase,'w')
for line in lines:
    es = line.split()
    if es[0].startswith('33'):
        if es[0] not in keep_idsNewName:
            print('remove',es[0])
            continue
    fout.write(line)
fout.close()
```